## <center>Avaliação regular do perfil do corpo docente do PPCS IRR/Fiocruz <br /> Programa de Pós-graduação em Ciências da Saúde <br /> Instituto René Rachou – Fiocruz Minas</center>

    Rubens Lima do Monte Neto – Instituto René Rachou
    Antonio Marcos Aires Barbosa – Fiocruz Ceará
    Equipe do PPGCS

**Objetivo:**

    Reavaliação de meio termo do corpo docente do programa, a fim de acompanhar a manutenção dos parâmetros exigidos pela CAPES – Área de Medicina II e readequar a composição do grupo. São considerados os docentes permanentes (DP) e docentes colaboradores (DC), com base nos mesmos parâmetros

# Apuração do Indicadores de Gestão do PPGCS
### 1. Índice de Produção Conjunta dos docentes com discentes (IPC >= 50%):
    Indicador: Índice de publicações com discentes por orientador (IPC)
     Objetivo: Em, pelo menos, 50 % dos artigos publicados deve constar discentes do programa
         Meta: IPC >= 50,00
      
      Cálculo:

$$\sum_{k=1}^{n}\, 100 * \frac{QPCD}{QPAT}$$

        onde:
               n = Artigos completos publicado em periódicos indexados
            QPCD = Qte. Publicação de Artigos com discentes do Programa na lista de autores
            QPAT = Qte. Publicação de Artigos Total no período avaliado

### 2. Pontuação por Fator de Impacto (PFI >= 600 em pelo menos 70% dos docentes):

    Indicador: Total de pontos conforme periódicos das publicações no período
     Objetivo: Publicar trabalhos em periódicos de elevado impacto
               
         Meta: 70% dos docentes permanentes com PFI >= 600 pontos no quadriênio
               (150/ano e ao menos 03 artigos A, no mínimo 02 em A1, ou 04 artigos A2);
 
      Cálculo: Soma ponderada pela estratificação Qualis de acordo com o que segue:
        A1 = 100 pontos
        A2 = 80 pontos
        B1 = 60 pontos
        B2 = 40 pontos
        B3 = 20 pontos
        B4 = 10 pontos
        B5 = 2 pontos.


Parâmetro para classificaçao do periódico:

        A1: Periódicos com FI ou CPD >= 4,300
        A2: Periódicos com FI ou CPD entre 2,950 e 4,299
        B1: Periódicos com FI ou CPD entre 1,800 e 2,949
        B2: Periódicos com FI ou CPD entre 1,100 e 1,799
        B3: Periódicos com FI ou CPD entre 0,300 e 1,099
        B4: Periódicos com FI ou CPD entre 0,001 e 0,299, (ou Scielo, Scimago, PubMed ou Web of Science)
        B5: Periódicos sem FI ou CPD e indexado nas lases Lilacs ou Latindex    

# <b>F00: Preparar Ambiente</b>

#### Instalar Git, WSL, ambiente virtual ou Contêiner Docker

In [1]:
## Instalar o WSL e integrar ao VSCode, no terminal rodar:
## Verificar versões já instaladas
#wsl -l -v
## Atualizar para WSL2 para que as instalações de distros já rodem em WSL2 por padrão
#wsl --set-default-version 2
## Reiniciar a máquina e instalar distribuição Linux Ubuntu LTS pelo store do windows
## Configurar VScode: iniciar VScode e instalar a extensão "Remote - WSL" para desenvolver diretamente no VSCode dentro do ambiente do WSL.
# Após a instalação, na parte inferior esquerda da janela do VSCode, aparecerá o ícone verde.
# Clicar no ícone verde e selecionar "New WSL Window" ou "Reopen in WSL" se o projeto já estiver aberto.
# Agora, o VSCode estará rodando dentro do contexto do seu WSL, pode-se abrir terminais dentro do VSCode que acessarão diretamente o Linux.

## Instalar Python e o Pip:
# sudo apt update
# sudo apt install python3-pip

## Inserir dados do Git no VScode, no terminal rodar:
# git config --global user.name "nome_usuario_gi"
# git config --global user.email "email_git"

#### Instalar gerenciadores de pacotes e aplicativos necessários

In [2]:
## Intalar os gerenciador de pacotes PIP  no terminal do WSL executar:
# sudo apt update
# sudo apt upgrade
# sudo apt install python3-pip
# python3 -m pip install --upgrade pip

## Atualizar o gerenciador de pacotes e ferramentas de compilação
# %pip install --upgrade pip setuptools wheel

## Instalar primeiro o GraphViz antes do ygraphviz, no terminal do WSL rodar e depois reiniciar o kernel:
## Para Linux e WSL instalar a partir do terminal:
# sudo apt-get install graphviz graphviz-dev

## Para Windows, atualizar o gerenciador de pacotes e ferramentas de compilação:
# %pip install --upgrade pip setuptools wheel

## Instalar o Chocolatey (Só para Windows, não no WSL)
## Baixar e instalar o gerenciador chocolatey em https://jcutrer.com/windows/install-chocolatey-choco-windows10
# Set-ExecutionPolicy Bypass -Scope Process -Force; [System.Net.ServicePointManager]::SecurityProtocol = [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://chocolatey.org/install.ps1'))

## Baixar e instalar o Graphviz em https://www.graphviz.org/download/
## https://savleen307.medium.com/pygraphviz-installation-in-windows-f45cc6fed981
## Seguir as instruções em https://pygraphviz.github.io/documentation/stable/install.html#id1
# O comando na página acima está errado por conter duas aspas onde não deve, executar o comando abaixo:
# python -m pip install --use-pep517 --config-settings="--global-option=build_ext" --config-settings="--global-option=-IC:\Program Files\Graphviz\include" --config-settings=--global-option="-LC:\Program Files\Graphviz\lib" pygraphviz

## Instalar o pacote no ambiente local
# %pip install pygraphviz

## Para instalar o Conda (Miniconda ou Anaconda) no WSL, baixar o script de instalação diretamente do site oficial e executá-lo manualmente.
## Passos para instalar o Miniconda como exemplo:
## Baixar script de instalação do Miniconda para Linux:
# wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# bash Miniconda3-latest-Linux-x86_64.sh
# conda --version

#### Instalar pacotes do requirements.txt e importar bibliotecas

In [3]:
## Localizar requirements.txt para instalar bibliotecas no ambente
# os.listdir('./../../../../')
# %pip install -r ./../../../../requirements.txt

## Outras instalações pontuais quando necessário, por exemplo:
# import os, tqdm
# %pip install chardet
# print(tqdm.__version__)
# %pip3 install --upgrade plotly
# %pip3 install omegaconf --upgrade

#### Instalar e configurar GPU e CUDA

In [4]:
## Instalar drivers da GPU e compilador CUDA Nvidia
## Obter comandos adequados para cada sistema em: https://developer.nvidia.com/cuda-downloads
## Exemplo para Linux com Ubuntu
# wget https://developer.download.nvidia.com/compute/cuda/repos/wsl-ubuntu/x86_64/cuda-wsl-ubuntu.pin
# sudo mv cuda-wsl-ubuntu.pin /etc/apt/preferences.d/cuda-repository-pin-600
# wget https://developer.download.nvidia.com/compute/cuda/12.4.0/local_installers/cuda-repo-wsl-ubuntu-12-4-local_12.4.0-1_amd64.deb
# sudo dpkg -i cuda-repo-wsl-ubuntu-12-4-local_12.4.0-1_amd64.deb
# sudo cp /var/cuda-repo-wsl-ubuntu-12-4-local/cuda-*-keyring.gpg /usr/share/keyrings/
# sudo apt-get update
# sudo apt-get -y install cuda-toolkit-12-4

## Instruções para instalação do PyTorch para usar a GPU em Windows
# https://sh-tsang.medium.com/tutorial-cuda-cudnn-anaconda-jupyter-pytorch-installation-in-windows-10-96b2a2f0ac57

## Para máquinas com apenas CPU
# !pip install torch torchvision

## Testar cálculo na GPU
# import torch
# x = torch.rand(5, 3)
# print(x)

#### Implementar classes para obter e preparar dados:

    LattesScraper  (Extrair currículos do HTML para JSON)
    SoupParser (Extrair currículos do Objeto Soup)
    Neo4jPersister (Persistir em Neo4j)
    DatasetArticlesGenerator (Gerar Datasets)
    DictToHDF5 (converter dicionários para formato HDF5)
    ArticlesCounter (Monta dataframe com quantidades de artigos e tempo de defasagem da atualização)

# <b>F01: Preparar para extrair e montar datasets</b>

#### Abrir Neo4j e depois preparar pastas locais e classes de processamento

In [5]:
from pathlib import Path
from getpass import getpass
from datetime import datetime
from IPython.display import clear_output
import pandas as pd, os, re, sys, time, json, subprocess

## Configurar exibição do pandas para melhor visualizar os dados
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 600)

def find_repo_root(path='.', depth=10):
    ''' 
    Busca o arquivo .git e retorna string com a pasta raiz do repositório
    '''
    # Prevent infinite recursion by limiting depth
    if depth < 0:
        return None
    path = Path(path).absolute()
    if (path / '.git').is_dir():
        return path
    return find_repo_root(path.parent, depth-1)

delay = 10

## Definir a pasta de base do repositório local
base_repo_dir = find_repo_root()

## Sempre construir os caminhos usando os.path.join para compatibilidade WxL
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, '_data', 'in_csv')
folder_data_output = os.path.join(base_repo_dir, '_data', 'out_json')

## Adicionar pastas locais ao sys.path para importar pacotes criados localmente
sys.path.append(folder_utils)
sys.path.append(folder_domain)
from scraper_pasteur import PasteurScraper
from environment_setup import EnvironmentSetup
from scraper_sucupira import SucupiraScraper
from scraper_sucupira_edge import SucupiraScraperEdge
from chromedriver_manager import ChromeDriverManager
from lattes_scrapper import JSONFileManager, LattesScraper, HTMLParser, SoupParser, GetQualis, ArticlesCounter, DictToHDF5, attribute_to_be_non_empty

### Checar/atualizar Chromedriver e GoogleChrome

In [6]:
# Cria instância da classe ChromeDriverManager e verifica compatibilidade entre versões do Chrome e Chromedriver
actualizer = ChromeDriverManager()
actualizer.main()

Versões 123 Chrome e 123 Chromedriver estão compatíveis


### Obter lista Qualis de Periódicos da Plataforma Sucupira

In [7]:
# scraper = SucupiraScraperEdge()
# scraper.scrape()

# <b>F02: Extrair dados de Currículos Lattes</b>

## Rodar testes de ambiente e definir pastas

### Preparar pastas locais para processamento de dados

In [8]:
# Cria instância da classe EnvironmenSetup e preparar pastas
preparer = EnvironmentSetup()
folder_name = input("Digite o nome da pasta principal: ")
preparer.set_root_path(folder_name)
preparer.try_cpu()
preparer.try_gpu()
preparer.try_amb()
# preparer.try_browser()
preparer.preparar_pastas()

## Remover diretórios no linux
# !rm -rf /home/mak/fioce
# os.listdir('/home/mak/fioce')

All necessary directories are ensured.
Processador em uso: 
Arquitetura modelo: AMD64 Family 25 Model 33 Stepping 0, AuthenticAMD
Arquitetura em uso: 64bit
Frequência das CPU: 3801.0 MHz
  Qte CPUs físicas: 8
  Qte CPUs lógicas: 16
Carga total na CPU: 1.4%
Ocupação atual CPU: user=5.4%, system=1.3%, idle=92.8%

Espaço Total em disco: 465.15 GB
Espaço em disco usado: 447.43 GB 96.2%
Espaço em disco livre: 17.72 GB 3.8%

Capacidade memórias RAM: 63.94 GB
Utilização atual da RAM: 12.81 GB

VERSÕES DOS DRIVERS CUDA, PYTORCH E GPU
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:30:42_Pacific_Standard_Time_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0

    PyTorch: 2.2.0+cu121
Dispositivo: cuda
Disponível: cuda True  | Inicializado: False | Capacidade: (7, 5)
Nome GPU: NVIDIA GeForce RTX 2060  | Quantidade: 1

VERSÕES DAS PRINCIPAIS BIBLIOTECAS INSTALADAS NO ENVIROMENT
Ambiente Conda ativ

## Montar lista_nomes de arquivo .csv para extrair currículos

In [9]:
import os, pandas as pd
pathfilename = os.path.join(folder_data_input,'nomesdocentes.csv')
lista_busca = pd.read_csv(pathfilename,header=None)[0].values
print(f'{len(lista_busca)} currículos a extrair da plataforma Lattes')

55 currículos a extrair da plataforma Lattes


## Montar lista_nomes de planilha para extrair currículos

In [10]:
# # Ler dados do arquivo Excel do Setor de Recursos Humanos
# pathdata = './../data/'
# file_persons = 'fioce_colaboradores-2023.xls'

# # Ler apenas os cabeçalhos do arquivo Excel
# headers = pd.read_excel(pathdata+file_persons, skiprows=3, header=0, nrows=0).columns
# # headers

# # Usar função para indicar quais colunas devem ser eliminadas na leitura
# def cols_to_keep(col_name):
#     return col_name not in ['QUANT','Unnamed: 3','Unnamed: 6','Unnamed: 9','ADICIONAL OCUPACIONAL',
#                             'EMPRESA/BOLSA/PROGRAMA','GESTOR','ADI','POSSE NA FIOCRUZ',
#                             'VIGÊNCIA BOLSA/ENCERRAMENTO DO CONTRATO','Unnamed: 17',
#                             'EMAIL INSTITUCIONAL','EMAIL PESSOAL','GENERO','DATA NASCIMENTO',
#                             'Unnamed: 22','FORMAÇÃO','ENDEREÇO RESIDENCIAL']

# # Filtrar cabeçalhos com base na função
# selected_columns = [col for col in headers if cols_to_keep(col)]

# # Ler dados do arquivo Excel do Setor de Recursos Humanos
# fioce_pessoal = pd.read_excel(pathdata+file_persons, skiprows=3, header=0, usecols=selected_columns)
# print(f'{len(fioce_pessoal.index)} nomes de colaboradores no total, todos vínculos e status')
# print(f'{len(fioce_pessoal["VÍNCULO"].unique()):3} tipos de vínculos')
# print('\nTipos de vínculos',list(fioce_pessoal['VÍNCULO'].unique()))
# print('  Tipos de status',list(fioce_pessoal['STATUS'].unique()))
# filtro1 = fioce_pessoal.VÍNCULO == 'SERVIDOR'
# filtro2 = fioce_pessoal['STATUS'].isin(['ATIVO', 'AFASTADO'])
# lista_nomes = fioce_pessoal[(filtro1) & (filtro2)]['NOME'].tolist()
# print(f'\n{len(lista_nomes)} nomes para extrair currículos')
# for i,nome in enumerate(lista_nomes):
#     print(f'{i+1:2}. {nome}')

In [11]:
# grupos_internos = fioce_pessoal['ÁREA'].unique()
# coordenacoes=[]
# grupos_tematicos=[]
# for i in grupos_internos:
#     # print(i)
#     if 'coordenação' in str(i).lower():
#         coordenacoes.append(i)
#     elif str(i) != 'nan':
#         grupos_tematicos.append(str(i))

In [12]:
# coordenacoes.sort()
# coordenacoes

In [13]:
# grupos_tematicos.sort()
# grupos_tematicos

In [14]:
# # Outras atividades não relacionadas diretamente com pesquisa
# remover = ['Angela Christina De Moraes Ostritz',
#            'Bruno Bezerra Carvalho',
#            'Carlos Jose Araujo Pinheiro',
#            'Charles Cerqueira De Abreu',
#            'Ezequiel Valentim De Melo',
#            'Ivanildo Lopes Farias',
#            'João Baptista Estabile Neto',
#            'Kamila Matos Albuquerque',
#            'Luciana Coelho Serafim',
#            'Luciano Pinto Zorzanelli',
#            'Luciana Silvério Alleluia Higino Da Silva',
#            'Luis Fernando Pessoa De Andrade',
#            'Marcelo Jorge Lopes Coutinho',
#            'Nilton Luiz Costa Machado',
#            'Patricia Maria Ferreira da Silva',
#            'Renato Caldeira De Souza',
#            'Sergio Dos Santos Reis',
#            'Clarice Gomes E Souza Dabés',
#            'Luciana Pereira Lindenmeyer',
#            'Rodrigo Carvalho Nogueira',
           
#         #    'Ana Camila Oliveira Alves',
#         #    'Antonio Marcos Aires Barbosa',
#         #    'Venúcia Bruna Magalhães Pereira',
#            ]

# trocar = {
#     'Maximiliano Loiola Ponte De Souza': 'Maximiliano Ponte',
#     'Raphael Trevizani Roque': 'Raphael Trevizani',
#     }

# lista_busca=[]
# for i in lista_nomes:
#     if i.strip() in trocar.keys():
#         lista_busca.append(trocar.get(i))
#     elif i.strip() not in remover:
#         lista_busca.append(i)

# # Limitando quantidade para testes
# # lista_busca = lista_busca[:5]

# print(f'{len(lista_busca)} currículos a extrair da plataforma Lattes')

## <b>Processar com classe de extração</b> 
    (10-15min/40nomes)

### Extrair currículos da plataforma Lattes

In [15]:
from environment_setup import EnvironmentSetup
preparer = EnvironmentSetup()
repo_root = preparer.find_repo_root(os.getcwd())
pasta_dados = os.path.join(repo_root,'data','output')
instituicao = 'cruz'
termo1 = 'cpqrr'
termo2 = 'minas'
termo3 = 'rachou'

In [16]:
# # Teste de estração de corrículos com homônimos
# t1 = time.time()
# lista_homonimos = [
#     'Tania Maria Alves de Almeida',
#     'Rodrigo Corrêa de Oliveira',
# ]

# scraper = LattesScraper(instituicao, termo1, termo2, 'bolt://localhost:7687', 'neo4j', 'password')
# dom_dict_homonimos = scraper.scrape(lista_homonimos, instituicao, termo1, termo2, termo3)

In [17]:
# Iniciar a extração completa de todos currículos
t1 = time.time()
scraper = LattesScraper(instituicao, termo1, termo2, termo3, 'bolt://localhost:7687', 'neo4j', 'password')

# Extrai e monta JSON com a lista de dicionários
dom_dict_list = scraper.scrape(lista_busca, instituicao, termo1, termo2, termo3)
print(f'\n{preparer.tempo(t1,time.time())} para busca de {len(lista_busca)} nomes com extração de dados de {len(dom_dict_list)} dicionários')

 1/55: Alessandra Aparecida Guarneri
       057 artigos extraídos
       DOI indisponível em 02 artigos extraídos
       Extração bem-sucedida
 2/55: Alexandre de Magalhães Vieira Machado
       034 artigos extraídos
       DOI indisponível para 01 artigo extraído
       Extração bem-sucedida
 3/55: Ana Lúcia Teles Rabello
       149 artigos extraídos
       DOI indisponível para 01 artigo extraído
       Extração bem-sucedida
 4/55: Andrea Teixeira de Carvalho
       268 artigos extraídos
       DOI indisponível para 01 artigo extraído
       Extração bem-sucedida
 5/55: Carlos Eduardo Calzavara Silva
       040 artigos extraídos
       DOI indisponível em 04 artigos extraídos
       Extração bem-sucedida
 6/55: Caroline Furtado Junqueira
       030 artigos extraídos
       Extração bem-sucedida
 7/55: Célia Maria Ferreira Gontijo
       107 artigos extraídos
       DOI indisponível em 04 artigos extraídos
       Extração bem-sucedida
 8/55: Cristiana Ferreira Alves de Brito
       08

In [18]:
print('\nAdicionando Estratos Qualis Periódicos')
from lattes_scrapper import GetQualis
stratifier = GetQualis()
stratifier.buscar_qualis(dom_dict_list)


Adicionando Estratos Qualis Periódicos
19352735 | A1
19326203 | A1
00311820 | A1
19352735 | A1
09651748 | A1
20492618 | A1
17563305 | A2
00311820 | A1
22352988 | A2
00311820 | A1
20010370 | A2
19352735 | A1
00221910 | A1
22352988 | A2
20010370 | A2
00311820 | A1
19352735 | A1
19326203 | A1
0001706X | A2
16788060 | A4
01666851 | A4
09320113 | A1
20452322 | A1
20452322 | A1
0950382X | A2
00311820 | A1
00221910 | A1
03009084 | A2
00311820 | A1
00311820 | A1
19352735 | A1
19352735 | A1
03009084 | A2
19352735 | A1
19326203 | A1
00278424 | A1
19326203 | A1
09651748 | A1
19352735 | A1
00740276 | A4
17560500 | B1
15303667 | A4
00222011 | A2
00221910 | A1
17563305 | A2
14714922 | A1
03076962 | A3
0001706X | A2
03076962 | A3
03076962 | A3
00221910 | A1
0102311X | A1
0102311X | A1
13602276 | A1
00222585 | A1
00740276 | A4
00740276 | A4
20411723 | A1
10436618 | A1
15178382 | A2
10400605 | A1
07302312 | A3
19352735 | A1
16643224 | A2
23275081 | B4
1743422X | A3
01663542 | A2
22352988 | A2
16643224

In [19]:
# Função para salvar a lista de dicionários em um arquivo .json
def save_to_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
       
dict_list=dom_dict_list
filepath = os.path.join(folder_data_input,'dict_list.json')
save_to_json(dict_list, filepath)

print(f"Arquivo JSON salvo em: {filepath}")

Arquivo JSON salvo em: c:\Users\marco\ppgcs\_data\in_csv\dict_list.json


In [20]:
lista_restante = lista_busca[:]
print(f'{len(lista_busca)} currículos buscados')
print(f'{len(dom_dict_list)} currículos extraídos com sucesso')
total_extraidos = 0
total_nao_extraidos = 0

# Função para normalizar os nomes
def normalizar_nome(nome):
    # Normaliza o nome para comparar de forma mais flexível
    nome_normalizado = nome.lower().replace('  ', ' ').replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('â', 'a').replace('ê', 'e').replace('ô', 'o').replace('ã', 'a').replace('õ', 'o').replace('ç', 'c').replace('ü', 'u')
    return nome_normalizado

# Lista para armazenar nomes extraídos com sucesso
nomes_extraidos = []

for i in dom_dict_list:
    nome = i.get('Identificação').get('Nome')
    nome_normalizado = normalizar_nome(nome)
    encontrado = False
    lista_restante = list(lista_restante)

    # Verifica se o nome ou uma forma similar já foi extraído
    for nome_original in lista_restante:
        nome_original_normalizado = normalizar_nome(nome_original)
        if nome_original_normalizado == nome_normalizado:
            lista_restante.remove(nome_original)
            encontrado = True
            break

    # Incrementa o contador correspondente
    if encontrado:
        total_extraidos += 1
    else:
        total_nao_extraidos += 1

    # Imprime o status da extração
    if encontrado:
        print(f'({total_extraidos:>02}) {nome}')
    else:
        print(f'({total_nao_extraidos}) Não extraído: {nome}')
    # Adiciona à lista de nomes extraídos apenas se não for exatamente igual ao buscado
    if encontrado and nome_original_normalizado != nome_normalizado:
        nomes_extraidos.append(nome)

print(f'\n{len(lista_restante)} currículos não extraídos')

for i in lista_restante:
    print(f'   {i}')

55 currículos buscados
55 currículos extraídos com sucesso
(01) Alessandra Aparecida Guarneri
(02) Alexandre de Magalhaes Vieira Machado
(03) Ana Lúcia Teles Rabello
(04) Andréa Teixeira de Carvalho
(05) Carlos Eduardo Calzavara Silva
(06) Caroline Furtado Junqueira
(07) Celia Maria Ferreira Gontijo
(08) Cristiana Ferreira Alves de Brito
(09) Cristina Toscano Fonseca
(10) Edelberto Santos Dias
(11) Edward José de Oliveira
(12) Flora Satiko Kano
(13) Glaucia Fernandes Cota
(14) Jaquelline Germano de Oliveira
(15) Jeronimo Conceição Ruiz
(16) José Dilermando Andrade Filho
(17) Lileia Gonçalves Diotaiuti
(18) Lis Ribeiro do Valle Antonelli
(19) Luciano Andrade Moreira
(20) Luzia Helena Carvalho
(21) Marcelo Antonio Pascoal Xavier
(22) Marcelo Gustavo Lorenzo
(23) Márcio Sobreira Silva Araújo
(24) Marco Antônio da Silva Campos
(25) Marina de Moraes Mourão
(26) Nagila Francinete Costa Secundino
(27) Olindo Assis Martins Filho
(28) Paulo Filemon Paolucci Pimenta
(29) Paulo Marcos Zech Coelho

### Visualizar resumo quantitativo de artigos extraídos

In [21]:
## Contagem de artigos para simples confererência
print(f'{len(dict_list)} dicionários montados')
qte_artigos=0
qte_titulos=0
for k,i in enumerate(dict_list):
    try:
        qte_jcr = len(i.get('Produções').get('Artigos completos publicados em periódicos'))
    except:
        qte_jcr = 0
    try:
       qte_jcr2 = len(i['JCR2'])
    except:
       qte_jcr2 = 0
    qte_artigos+=qte_jcr
    qte_titulos+=qte_jcr2
    status=qte_jcr2-qte_jcr
    print(f"{k:>2}C {qte_jcr:>03}A {qte_jcr2:>03}T Dif:{status:>03} {i.get('Identificação').get('name')} ")

print(f'\nTotal de artigos em todos períodos: {qte_artigos}')
print(f'Total de títulos em todos períodos: {qte_titulos}')

55 dicionários montados
 0C 057A 057T Dif:000 None 
 1C 034A 034T Dif:000 None 
 2C 149A 149T Dif:000 None 
 3C 268A 268T Dif:000 None 
 4C 040A 040T Dif:000 None 
 5C 030A 030T Dif:000 None 
 6C 107A 107T Dif:000 None 
 7C 081A 081T Dif:000 None 
 8C 057A 057T Dif:000 None 
 9C 089A 089T Dif:000 None 
10C 062A 062T Dif:000 None 
11C 042A 042T Dif:000 None 
12C 082A 082T Dif:000 None 
13C 039A 039T Dif:000 None 
14C 070A 070T Dif:000 None 
15C 131A 131T Dif:000 None 
16C 154A 154T Dif:000 None 
17C 084A 084T Dif:000 None 
18C 090A 090T Dif:000 None 
19C 074A 074T Dif:000 None 
20C 053A 053T Dif:000 None 
21C 075A 075T Dif:000 None 
22C 082A 082T Dif:000 None 
23C 052A 052T Dif:000 None 
24C 048A 048T Dif:000 None 
25C 074A 074T Dif:000 None 
26C 473A 473T Dif:000 None 
27C 164A 164T Dif:000 None 
28C 239A 239T Dif:000 None 
29C 302A 302T Dif:000 None 
30C 081A 081T Dif:000 None 
31C 361A 361T Dif:000 None 
32C 111A 111T Dif:000 None 
33C 055A 055T Dif:000 None 
34C 117A 117T Dif:000 No

### Extrair e salvar dados brutos da extração de remanescentes

In [25]:
for i in dict_list:
    print(i.get('Identificação').get('Nome'))

Alessandra Aparecida Guarneri
Alexandre de Magalhaes Vieira Machado
Ana Lúcia Teles Rabello
Andréa Teixeira de Carvalho
Carlos Eduardo Calzavara Silva
Caroline Furtado Junqueira
Celia Maria Ferreira Gontijo
Cristiana Ferreira Alves de Brito
Cristina Toscano Fonseca
Edelberto Santos Dias
Edward José de Oliveira
Flora Satiko Kano
Glaucia Fernandes Cota
Jaquelline Germano de Oliveira
Jeronimo Conceição Ruiz
José Dilermando Andrade Filho
Lileia Gonçalves Diotaiuti
Lis Ribeiro do Valle Antonelli
Luciano Andrade Moreira
Luzia Helena Carvalho
Marcelo Antonio Pascoal Xavier
Marcelo Gustavo Lorenzo
Márcio Sobreira Silva Araújo
Marco Antônio da Silva Campos
Marina de Moraes Mourão
Nagila Francinete Costa Secundino
Olindo Assis Martins Filho
Paulo Filemon Paolucci Pimenta
Paulo Marcos Zech Coelho
Ricardo Tostes Gazzinelli
Roberta Lima Caldeira
Rodrigo Correa de Oliveira
Rodrigo Pedro Pinto Soares
Rubens Lima do Monte Neto
Silvane Maria Fonseca Murta
Taís Nóbrega de Sousa
Vanessa Peruhype Magalhãe

In [27]:
# lista_restante

In [26]:
# # Iniciar a extração de currículos remanescentes
# t0 = time.time()

# # Extrai e monta a lista de dicionários
# dict_list_1 = scraper.scrape(lista_restante, instituicao, termo1, termo2, termo3,
#                             pasta_dados+"dicts_list1.json", 
#                             pasta_dados+"dicts_list1.hdf5")

# print(f'\n{preparer.tempo(t0,time.time())} para busca de {len(lista_restante)} nomes com extração de dados de {len(dict_list_1)} dicionários')

In [28]:
# Concatenando os dicionários
# dict_concatenado = dom_dict_list.copy()  # Cria uma cópia de dict1 para não modificar o original
# dict_concatenado.update(dom_dict_list_1)   # Atualiza dict_concatenado com os itens de dict2        
# dict_list=dict_concatenado
# pasta_dados = os.path.join(find_repo_root(os.getcwd()),'data','output','dict_list.json')
# save_to_json(dict_list, pasta_dados)

# print(f"Arquivo JSON salvo em: {pasta_dados}")

# <b>F03: Carregar JSON salvo</b>

### Visualizar arquivos gerados previamente

In [1]:
from pathlib import Path
from getpass import getpass
from datetime import datetime
from IPython.display import clear_output
import pandas as pd, os, re, sys, time, pytz, json, subprocess

## Configurar exibição do pandas para melhor visualizar os dados
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 600)

def find_repo_root(path='.', depth=10):
    ''' 
    Busca o arquivo .git e retorna string com a pasta raiz do repositório
    '''
    # Prevent infinite recursion by limiting depth
    if depth < 0:
        return None
    path = Path(path).absolute()
    if (path / '.git').is_dir():
        return path
    return find_repo_root(path.parent, depth-1)

delay = 10

## Definir a pasta de base do repositório local
base_repo_dir = find_repo_root()

## Sempre construir os caminhos usando os.path.join para compatibilidade WxL
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, '_data', 'in_csv')
folder_data_output = os.path.join(base_repo_dir, 'data', 'output')

## Adicionar pastas locais ao sys.path para importar pacotes criados localmente
sys.path.append(folder_utils)
sys.path.append(folder_domain)
from scraper_pasteur import PasteurScraper
from environment_setup import EnvironmentSetup
from scraper_sucupira import SucupiraScraper
from scraper_sucupira_edge import SucupiraScraperEdge
from chromedriver_manager import ChromeDriverManager
from lattes_scrapper import JSONFileManager, LattesScraper, HTMLParser, SoupParser, GetQualis, ArticlesCounter, DictToHDF5, attribute_to_be_non_empty

In [2]:
# Carregar o conteúdo do arquivo 'dict_list.json' para a variável dict_list
jfm = JSONFileManager()
jfm.list_json(folder_data_input)
filename = 'dict_list.json'

dict_list, formatted_creation_date, formatted_modification_date, time_count, unit = jfm.load_from_json(os.path.join(folder_data_input,filename))
print(f"\n{len(dict_list)} currículos carregados na lista de dicionários '{filename}'")
print(f"Arquivo criado inicialmente em {formatted_creation_date} carregado com sucesso")
print(f"Extração realizada em {formatted_modification_date} a {time_count} {unit}")

Arquivos disponíveis na pasta para dados de entrada:
  dict_list.json

55 currículos carregados na lista de dicionários 'dict_list.json'
Arquivo criado inicialmente em 14/04/2024 18:28:18 carregado com sucesso
Extração realizada em 16/04/2024 16:22:04 a 1.4 horas


### Quantidade de artigos e defasagem de atualização do currículo

In [3]:
import pandas as pd

atualizador = ArticlesCounter(dict_list)
dtf_atualizado = atualizador.extrair_data_atualizacao(dict_list)
dtf_atualizado

,id_lattes,curriculos,ultima_atualizacao,dias_defasagem,qte_artigos_periodicos
0,5565463591721568,Alessandra Aparecida Guarneri,14/03/2024,33,57
1,5804297852066640,Alexandre de Magalhaes Vieira Machado,21/09/2022,573,34
2,7027792126979693,Ana Lúcia Teles Rabello,22/06/2023,299,149
3,3798623797837575,Andréa Teixeira de Carvalho,21/02/2024,55,268
4,8454953908544325,Carlos Eduardo Calzavara Silva,21/03/2024,26,40
5,9405337414047265,Caroline Furtado Junqueira,12/04/2024,4,30
6,7004385035447880,Celia Maria Ferreira Gontijo,08/03/2024,39,107
7,0553918889655570,Cristiana Ferreira Alves de Brito,06/03/2024,41,81
8,7920312439713999,Cristina Toscano Fonseca,09/02/2024,67,57
9,0492208637289837,Edelberto Santos Dias,01/04/2024,15,89


#### Contagem de artigos completos publicados em periódicos estratificada pelo Qualis Periódicos

In [4]:
fonte_planilha = 'classificações_publicadas_todas_as_areas_avaliacao1672761192111.xlsx'
dados_qualis = pd.read_excel(os.path.join(LattesScraper.find_repo_root(),'_data','in_xls',fonte_planilha))
dados_qualis

,ISSN,Título,Área de Avaliação,Estrato
0,2541-965X,РУССКАЯ ПОЛИТОЛОГИЯ (RUSSIAN POLITICAL SCIENCE),"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS CONTÁBEIS E TURISMO",B2
1,2236-6695,A BARRIGUDA: REVISTA CIENTÍFICA,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS CONTÁBEIS E TURISMO",B2
2,1413-6090,A ECONOMIA EM REVISTA,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS CONTÁBEIS E TURISMO",B3
3,2236-2029,A ECONOMIA EM REVISTA - AERE,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS CONTÁBEIS E TURISMO",B3
4,1528-7106,AACE INTERNATIONAL TRANSACTIONS,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS CONTÁBEIS E TURISMO",C
...,...,...,...,...
154404,0944-2006,ZOOLOGY (JENA),ZOOTECNIA / RECURSOS PESQUEIROS,A1
154405,0720-213X,ZOOMORPHOLOGY (BERLIN. PRINT),ZOOTECNIA / RECURSOS PESQUEIROS,A3
154406,1175-5326,ZOOTAXA (AUCKLAND. PRINT),ZOOTECNIA / RECURSOS PESQUEIROS,A4
154407,1175-5334,ZOOTAXA (ONLINE),ZOOTECNIA / RECURSOS PESQUEIROS,A4


In [5]:
atualizador.contar_qualis(dict_list)

Contagem de Publicações por Qualis Periódicos no período  a 2024


Qualis,A1,A2,A3,A4,B1,B2,B3,B4,C,Não encontrado
Autor,,,,,,,,,,
Alessandra Aparecida Guarneri,35,12,3,6,1,0,0,0,0,0
Alexandre de Magalhaes Vieira Machado,13,15,3,0,0,0,0,1,0,2
Ana Lúcia Teles Rabello,43,10,32,37,22,0,1,0,0,4
Andréa Teixeira de Carvalho,68,76,45,40,12,15,2,1,3,6
Antonio Mauro Rezende,19,21,11,3,4,1,0,0,2,0
Carina Margonari de Souza,11,5,3,7,4,0,1,0,0,2
Carlos Eduardo Calzavara Silva,15,11,4,6,2,0,0,1,0,1
Caroline Furtado Junqueira,21,2,1,0,1,0,0,0,0,5
Celia Maria Ferreira Gontijo,30,23,9,14,4,4,1,1,2,19


#### Classificação dos artigos no período ponderada pelo Qualis Periódicos

Obs.: 'Não encontrado' significa que o ISSN da revista da publicação não consta na lista de revistas avaliadas no Qualis Periódico

In [6]:
# Definir os anos de interesse
ano_inicio = 2017
ano_final = 2024

In [7]:
atualizador.apurar_qualis_periodo(dict_list, ano_inicio, ano_final).fillna('')

Ano,2017,2018,2019,2020,2021,2022,2023,2024
Autor,,,,,,,,
Alessandra Aparecida Guarneri,"A1, A2, A1, A1","A4, A1, A1","A1, A2, A4","A1, A2, A2, A1, A1","A2, A1, A2, A1","A1, A2, A1","A1, A1, A1, A1",A1
Alexandre de Magalhaes Vieira Machado,"A3, A2, A2, A2, A2","A2, B4","A3, A1",A1,"A1, A2",,,
Ana Lúcia Teles Rabello,"B1, A2, A1, A1, B1, A1, A4, A1, A1","A4, B1, B1, A1, B1, B1","A1, A4, A1, B1, A1, A1, A3, A1, A4","A2, A1, B1, A2, A1","B1, A2","A1, A1",B1,
Andréa Teixeira de Carvalho,"A3, A3, A2, B2, A1, A4, A1, A1, B2, A3, A2, A1, A2, A3, A1, B2, A3, A3","A3, A2, B2, A1, A1, A2, A4, A1, A1, A2, A2, A1, A3, A2, A1","A4, A3, A2, A1, A1, A2, A2, A1, A2, A1, A2, A2, A2, A3, A2, A3, A1, A3","B1, A2, B2, A2, A2, A4, A2, A2, A4, A4, A2, A1, A2, A1, A1, A1, A1, A1, A2, A4, A1, A1","A3, C, A2, B3, A1, A2, A3, A4, A1, A2, A3, A2","A3, A2, A2, A1, A1, A3, A2, A3, A2, A2, A2, A2, A2, A2, A2, A2, A2, A2, A2, A1, A4, Não encontrado","A4, A4, A1, A4, A3, A1, A2, A4, A2, A2, C, A2, Não encontrado, A1, A2, A4","A2, A1, C, A2"
Antonio Mauro Rezende,"A1, A2, A3","A1, A1, A2, A4, A3","A1, A2, A3, A1, A2, A2","A2, A3, A2, B1, A1, A1, B1, A4, A2","A2, A3, A3, A2, A2, A1, C","A1, B1, A2, A1, C","A2, A2, B1, A1, A3, A3",B2
Carina Margonari de Souza,A2,"A1, B3",A3,"A4, A4","A1, A4, A1, A1",A2,A2,
Carlos Eduardo Calzavara Silva,B1,"A2, A2, A4, A1",A3,"A2, A1, A2, A2",A1,"A2, A1, A4, A2, A3, Não encontrado","A1, A1, A3, A1",A1
Caroline Furtado Junqueira,"Não encontrado, A2, A1","A2, A1",Não encontrado,"A1, A1, A1, A1","A1, A1, A3","A1, A1, B1","A1, A1, A1, A1, A1",
Celia Maria Ferreira Gontijo,"A1, A1, A4, A2, A1, A1","B2, A3, A1, A2, A1, A1, A2, A2","A4, A2, A4, A3, A1, A2","A2, A2, A1, A4, A2, A1, A4","A1, A1, A2, C","A1, A2, A2, B1, C","A3, A4, A2, A2",


#### Pontuação participação em artigos por anos do período ponderada pelo Qualis Periódicos

In [8]:
atualizador.apurar_pontos_periodo(dict_list, ano_inicio, ano_final)

Ano,2017,2018,2019,2020,2021,2022,2023,2024,Soma de Pontos
Autor,,,,,,,,,
Olindo Assis Martins Filho,1505,1120,1850,1730,1485,1850,1465,480,11485
Luiz Carlos Júnior Alcantara,430,650,845,1300,1560,2025,1610,435,8855
Andréa Teixeira de Carvalho,1135,1115,1370,1555,730,1610,930,250,8695
Ricardo Tostes Gazzinelli,510,530,430,790,940,790,500,0,4490
Gabriel da Rocha Fernandes,920,370,170,430,150,825,750,470,4085
Glaucia Fernandes Cota,570,545,310,620,610,430,290,380,3755
Lis Ribeiro do Valle Antonelli,310,630,690,470,670,530,270,90,3660
Rodrigo Correa de Oliveira,1290,360,770,360,360,230,270,0,3640
Rodrigo Pedro Pinto Soares,540,560,620,520,750,150,220,170,3530


### Contagem da publicação com discentes

- Busca por nomes de discentes nas publicações dos currículos dos docentes e 
- Calcular o percentual de publicações com participação de discente da lista fornecida pelo programa

In [10]:
fonte_planilha = 'ppgcs_estudantes_2021-2024.xlsx'
dados_discentes = pd.read_excel(os.path.join(LattesScraper.find_repo_root(),'_data','in_xls',fonte_planilha), header=1)
lista_discentes = list(dados_discentes['Discente'].unique())
print(f'{len(lista_discentes)} discentes no período 2021-2024 informados pelos programa')

# Função para normalizar os nomes
def normalizar_nome(nome):
    # Normaliza o nome para comparar de forma mais flexível
    nome_normalizado = nome.lower().replace('  ', ' ').replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('â', 'a').replace('ê', 'e').replace('ô', 'o').replace('ã', 'a').replace('õ', 'o').replace('ç', 'c').replace('ü', 'u')
    return nome_normalizado

lista_normalizada_discentes=[]
for i in lista_discentes:
    lista_normalizada_discentes.append(normalizar_nome(i))

lista_normalizada_discentes

93 discentes no período 2021-2024 informados pelos programa


['agata lopes ribeiro',
 'alana karen de oliveira',
 'alessandra mara de sousa',
 'aline silva soares',
 'amanda cupertino de freitas',
 'amanda domingues de araujo',
 'ana carolina de oliveira costa',
 'ana luiza pessoa de mendonca angelo',
 'ana maria murta santi',
 'ana raquel viana de godoy',
 'andre luiz de menezes',
 'andreza parreiras goncalves',
 'anna carolina cancado figueiredo',
 'arthur ribeiro cheloni soares',
 'barbara andreza soares lima',
 'breno dos anjos costa',
 'bruna estefania diniz frias',
 'camila amormino corsini',
 'camila binder soares de souza',
 'camila medeiros costa',
 'camila sales nascimento',
 'cinthia catharina azevedo oliveira',
 'cristhiane oliveira da fonseca',
 'davi alvarenga lima',
 'debora cristina capucci',
 'eduardo ribeiro de oliveira',
 'eliza flores de souza',
 'fabio magalhaes da gama',
 'felipe miguel nery lunkes',
 'flavio campos ferreira',
 'flavio junior barbosa figueiredo',
 'franklin pereira araujo',
 'frederico goncalves guimaraes',

### Montar lista de coautores para buscar participação discente

In [37]:
import re

def extract_coauthors(publication_list):
    coauthors_list = []

    for publication in publication_list:
        # Match author and year pattern
        match = re.match(r'(?P<year>\d{4})', publication)

        if match:
            # Extract data from the match object
            author_name = match.group('author')
            year = match.group('year')
            coauthor_string = match.group('coauthors')
            print(coauthor_string)

            # Separate author name and year
            author_parts = author_name.split(',')
            primary_author = ', '.join(author_parts[:-1]).strip()
            year = year.strip()

            # Remove initial part of the string (up to first comma after year)
            if len(author_parts) > 1:
                first_comma_index = author_name.find(',', len(year) + len(', '))
                if first_comma_index > 0:
                    publication = publication[first_comma_index + 1:].strip()

            # Extract coauthors
            coauthors = []
            for part in publication.split('; '):
                if part.endswith('.. ') or part.endswith(' . '):
                    break

                coauthor_name = part.strip()
                if coauthor_name != '':
                    coauthors.append(coauthor_name)

            # Create and append coauthor tuple
            coauthor_tuple = (primary_author, year, coauthors)
            coauthors_list.append(coauthor_tuple)

        else:
            # Handle cases where the expected pattern doesn't match
            print(f"Warning: Publication string '{publication}' doesn't follow the expected format.")

    return coauthors_list



In [13]:
lista_coautores = []
for dic in dict_list:
    autor = dic.get('Identificação',{}).get('Nome',{})
    artigos = dic.get('Produções', {}).get('Artigos completos publicados em periódicos', {})
    for i in artigos:
        ano = i.get('ano',{})
        qualis = i.get('Qualis',{})
        autores = i.get('autores')
        lista_coautores.append(autores)
        # lista_pubqualis.append((ano, autor, qualis))

In [38]:
extract_coauthors(lista_coautores)

[]

## Montar Grafo e analisar redes

In [ ]:
import logging
from neo4j import GraphDatabase
from neo4j.exceptions import Neo4jError

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s', filename='logs/persister.log')
logger = logging.getLogger(__name__)

class Neo4jPersister:
    def __init__(self, uri, user, password):
        self._uri = uri
        self._user = user
        self._password = password
        self._driver = GraphDatabase.driver(self._uri, auth=(self._user, self._password))
        self.logger = logger

        # Devem ser persistidos como 
        self.tipos = ['Identificação',
                      'Idiomas',
                      'Formação',
                      'Atuação Profissional',
                      'Linhas de Pesquisa',
                      'Áreas',
                      'Produções',
                      'ProjetosPesquisa',
                      'ProjetosExtensão',
                      'ProjetosDesenvolvimento',
                      'ProjetosOutros',
                      'Bancas',
                      'Orientações',
                      ]

        self.subtipos= ['Acadêmica',
                        'Pos-Doc',
                        'Complementar',
                        'Artigos completos publicados em periódicos',
                        'Resumos publicados em anais de congressos',
                        'Apresentações de Trabalho',
                        'Outras produções bibliográficas',
                        'Entrevistas, mesas redondas, programas e comentários na mídia',
                        'Concurso público',
                        'Outras participações',
                        'Livros publicados/organizados ou edições',
                        'Capítulos de livros publicados',
                        'Resumos expandidos publicados em anais de congressos',
                        'Resumos publicados em anais de congressos (artigos)',
                        'Trabalhos técnicos',
                        'Demais trabalhos',
                        'Mestrado',
                        'Teses de doutorado',
                        'Qualificações de Doutorado',
                        'Qualificações de Mestrado',
                        'Monografias de cursos de aperfeiçoamento/especialização',
                        'Trabalhos de conclusão de curso de graduação',
                        'Orientações e supervisões concluídas',
                        'Citações',
                        'Trabalhos completos publicados em anais de congressos',
                        'Produtos tecnológicos',
                        'Artigos  aceitos para publicação',
                        'Assessoria e consultoria',
                        'Programas de computador sem registro',
                        'Professor titular',
                        'Avaliação de cursos',
                        'Orientações e supervisões em andamento',
                        'Processos ou técnicas',
                        'Outras produções artísticas/culturais',
                        'Textos em jornais de notícias/revistas',
                        'Redes sociais, websites e blogs',
                        'Artes Visuais'            
                        ]

        self.propriedades = ['Nome',
                             'ID Lattes',
                             'Última atualização',
                             ]
    # def run(self, query, **parameters):
    #     with self.driver.session() as session:
    #         return session.run(query, **parameters)
        
    def close(self):
        self._driver.close()
    
    def persistir_revistas_da_planilha(self):
        """
        Persiste dados de revistas a partir da planilha 'classificações_publicadas_todas_as_areas_avaliacao1672761192111.xlsx' no Neo4j.

        Args:
            session: Objeto de sessão do Neo4j.
        """
        # Leitura da planilha
        dados_qualis = pd.read_excel(os.path.join(LattesScraper.find_repo_root(),'data','classificações_publicadas_todas_as_areas_avaliacao1672761192111.xlsx'))

        # Extração e persistência de dados de revista
        with self._driver.session() as session:
            for index, row in dados_qualis.iterrows():
                issn = row['ISSN'].replace('-','')
                nome_revista = row['Título']
                area_avaliacao = row['Área de Avaliação']
                estrato = row['Estrato']

                # Verificação de existência da revista
                revista_node = session.run("""
                    MATCH (j:Revista {issn: $issn})
                    RETURN j
                """, issn=issn).single()

                if not revista_node:
                    # Criação da revista se não existir
                    session.run("""
                        CREATE (j:Revista {issn: $issn, nome_revista: $nome_revista, area_avaliacao: $area_avaliacao, estrato: $estrato})
                    """, nome_revista=nome_revista, issn=issn, area_avaliacao=area_avaliacao,  estrato=estrato)

    # Testes Ok! 
    def persist_pessoa_nodes(self, dict_list):
        query_pessoa = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        ON CREATE SET p.nome = $nome, p.ultima_atualizacao = $ultima_atualizacao
        ON MATCH SET p.nome = $nome, p.ultima_atualizacao = $ultima_atualizacao
        """
        try:
            with self._driver.session() as session:
                for item in dict_list:
                    identificacao = item.get('Identificação')
                    nome = identificacao.get('Nome')
                    id_lattes = identificacao.get('ID Lattes')
                    ultima_atualizacao = identificacao.get('Última atualização')
                    if nome:
                        session.run(query_pessoa, id_lattes=id_lattes, nome=nome, ultima_atualizacao=ultima_atualizacao)
        except Exception as e:
            self.logger.error('Erro ao criar node "Pesquisador": {}'.format(e))

    # Testes Ok!         
    def persist_pesquisador_grande_area_relationships(self, dict_list):
        query_rel_pessoa_grande_area = """
        MATCH (p:Pesquisador {id_lattes: $id_lattes})
        MATCH (ga:GrandeArea {nome: $grande_area_nome})
        MERGE (p)-[:ATUA_EM]->(ga)
        """

        with self._driver.session() as session:
            for item in dict_list:
                identificacao = item.get('Identificação')
                id_lattes = identificacao.get('ID Lattes')
                areas = item.get('Áreas').values()
                for area_string in areas:
                    grande_area_nome, _, _ = self.extract_area_info(area_string)
                    if grande_area_nome:
                        session.run(query_rel_pessoa_grande_area, id_lattes=id_lattes, grande_area_nome=grande_area_nome)

    # Testes Ok! 
    def persist_areas_nodes(self, dict_list):
        query_grande_area = """
        MERGE (ga:GrandeArea {nome: $nome})
        """
        query_area = """
        MATCH (ga:GrandeArea {nome: $grande_area_nome})
        MERGE (a:Area {nome: $nome}) ON CREATE SET a:Area
        MERGE (ga)-[:CONTEM]->(a)
        """
        query_subarea = """
        MATCH (a:Area {nome: $area_nome})
        MERGE (sa:Subarea {nome: $nome}) ON CREATE SET sa:Subarea
        MERGE (a)-[:CONTEM]->(sa)
        """
        query_rel_pessoa_grande_area = """
        MATCH (p:Pesquisador {id_lattes: $id_lattes})
        MATCH (ga:GrandeArea {nome: $grande_area_nome})
        MERGE (p)-[:ATUA_EM]->(ga)    
        """

        with self._driver.session() as session:
            for item in dict_list:
                areas = item.get('Áreas').values()
                for area_string in areas:
                    grande_area_nome, area_nome, subarea_nome = self.extract_area_info(area_string)
                    
                    # Verificar se o nome não está vazio
                    if grande_area_nome:
                        session.run(query_grande_area, nome=grande_area_nome)
                    if area_nome:
                        session.run(query_area, grande_area_nome=grande_area_nome, nome=area_nome)
                    if subarea_nome:
                        session.run(query_subarea, area_nome=area_nome, nome=subarea_nome)
                    
                    # Adicionar relacionamento Pesquisador - GrandeÁrea
                    id_lattes = item['Identificação']['ID Lattes']
                    if grande_area_nome:
                        session.run(query_rel_pessoa_grande_area, id_lattes=id_lattes, grande_area_nome=grande_area_nome)

    # Testes Ok! 
    @staticmethod
    def extract_area_info(area_string):
        # Extraindo os nomes de GrandeÁrea, Área e Subárea da string
        try:
            grande_area_nome = area_string.split('/')[0].strip().split(': ')[1]
        except:
            grande_area_nome = ''
        try:
            area_nome = area_string.split('/')[1].strip().split(': ')[1]
        except:
            area_nome = ''
        try:
            subarea_nome = area_string.split('/')[2].strip().split(': ')[1]
        except:
            subarea_nome = ''
        return grande_area_nome, area_nome, subarea_nome

    ## PRODUÇÕES
    def persist_producoes_pesquisador(self, dict_list):
        with self._driver.session() as session:
            for pesq in dict_list:
                identificacao = pesq.get('Identificação')
                id_lattes = identificacao.get('ID Lattes')
                producoes = pesq.get('Produções')

                if not isinstance(producoes, dict):
                    print(f"Erro!! Dicionário da seção 'Produções' não encontrado para {id_lattes}")
                    continue

                for chave_producao, valores_producao in producoes.items():
                    print(f'{chave_producao} | {valores_producao}')
                    if chave_producao == 'Artigos completos publicados em periódicos':
                        # self.persistir_artigos_completos(session, id_lattes, valores_producao)
                        self.persistir_artigos_revistas(session, id_lattes, valores_producao)

    def _get_or_create_node(self, session, label, properties):
        properties = [x.rstrip('.') for x in properties]
        node = session.run("MATCH (n: {label}) WHERE {properties} RETURN n", {"label": label, "properties": properties}).single()

        if not node:
            node = session.run("CREATE (n: {label} {properties}) RETURN n", {"label": label, "properties": properties}).single()["n"]
            self._node_created_count += 1

        return node

    def persist_tipo_producao(self, session, id_lattes, tipo_producao):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (t:TipoProducao {nome: $tipo_producao})
        MERGE (p)-[:PRODUZ]->(t)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, tipo_producao=tipo_producao)
        summary = result.consume()
        return summary.counters.nodes_created, summary.counters.nodes_deleted, summary.counters.relationships_created, summary.counters.relationships_deleted

    def persist_subtipo_producao(self, session, id_lattes, tipo_producao, subtipo_producao, dados_producao):
        def checar_e_serializar(dados):
            """ Verifica e serializa dicionários recursivamente """
            if isinstance(dados, dict):
                for chave, valor in dados.items():
                    if isinstance(valor, dict):
                        dados[chave] = json.dumps(valor)
                    # Checagem adicional para outros tipos inválidos, se necessário
            return dados
        # Serialização recursiva do dicionário
        dados_producao = checar_e_serializar(dados_producao)

        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (t:TipoProducao {nome: $tipo_producao})
        MERGE (s:SubtipoProducao {nome: $subtipo_producao})
        MERGE (p)-[PRODUZ:]->(t)-[:DO_TIPO]->(s)
        """

        if subtipo_producao in ["ArtigoCompleto", "ResumoCongresso", "ApresentacaoTrabalho", "OutrasProducoesBibliograficas"]:
            query_create_node += """
            MERGE (o:Ocorrencia {tipo: $subtipo_producao, dados: $dados})
            MERGE (s)-[:OCORRENCIA]->(o)
            """

        result = session.run(query_create_node, id_lattes=id_lattes, tipo_producao=tipo_producao, 
                            subtipo_producao=subtipo_producao, dados=dados_producao)

        # Obtendo as informações de contadores
        summary = result.consume()

        return summary.counters.nodes_created, summary.counters.nodes_deleted, summary.counters.relationships_created, summary.counters.relationships_deleted

    def persistir_artigos_completos(self, session, id_lattes, dados):
        created_nodes = 0
        updated_nodes = 0
        created_relations = 0
        updated_relations = 0
        
        for dados_artigo in dados:
            dados_artigo['dados'] = json.dumps(dados_artigo)  # Conversão para JSON da estrutura completa
            query_create_node_artigo = """
                MERGE (p:Pesquisador {id_lattes: $id_lattes})
                CREATE (a:ArtigoPublicado {
                    ano: $ano,
                    fator_impacto_jcr: $fator_impacto_jcr,
                    ISSN: $ISSN,
                    titulo: $titulo,
                    revista: $revista,
                    autores: $autores,
                    Qualis: $Qualis,
                    DOI: $DOI,
                    dados: $dados_artigo
                })
                CREATE (p)-[:PRODUZ]->(a)

                MERGE (j:Revista {nome: $revista, issn: $ISSN})
                CREATE (a)-[:PUBLICADO_EM]->(j)
            """

            result_artigo = session.run(query_create_node_artigo, 
                                id_lattes=id_lattes, 
                                ano=dados_artigo['ano'],
                                fator_impacto_jcr=dados_artigo['fator_impacto_jcr'],
                                ISSN=dados_artigo['ISSN'],
                                titulo=dados_artigo['titulo'],
                                revista=dados_artigo['revista'],
                                autores=dados_artigo['autores'],
                                Qualis=dados_artigo['Qualis'],
                                DOI=dados_artigo['DOI'],
                                dados_artigo=dados_artigo
                                )
            summary_artigo = result_artigo.consume()
            created_nodes += summary_artigo.counters.nodes_created
            updated_nodes += summary_artigo.counters.nodes_deleted
            created_relations += summary_artigo.counters.relationships_created
            updated_relations += summary_artigo.counters.relationships_deleted

        return created_nodes, updated_nodes, created_relations, updated_relations


    def persistir_artigos_completos(self, session, id_lattes, dados):
        created_nodes = 0
        updated_nodes = 0
        created_relations = 0
        updated_relations = 0
                
        for dados_artigo in dados:
            ano = dados_artigo['ano']
            impact_jcr = dados_artigo['fator_impacto_jcr']
            issn = dados_artigo['ISSN']
            titulo = dados_artigo['titulo']
            revista = dados_artigo['revista']
            autores = dados_artigo['autores']
            qualis = dados_artigo['Qualis']
            doi = dados_artigo['DOI']

            query_create_node_artigo = """
                MERGE (p:Pesquisador {id_lattes: $id_lattes})
                CREATE (a:ArtigoPublicado {ano: $ano, impact_jcr: $impact_jcr, issn: $issn, titulo: $titulo, revista: $revista, autores: $autores, qualis: $qualis, doi: $doi})
                CREATE (p)-[:PRODUZ]->(a)
                MERGE (j:Revista {nome: $revista, issn: $issn})
                CREATE (a)-[:PUBLICADO_EM]->(j)
            """
            print(query_create_node_artigo)
            result_artigo = session.run(query_create_node_artigo, 
                                id_lattes=id_lattes, 
                                ano=ano,
                                impact_jcr=impact_jcr,
                                issn=issn,
                                titulo=titulo,
                                revista=revista,
                                autores=autores,
                                qualis=qualis,
                                doi=doi,
                                )
            summary_artigo = result_artigo.consume()
            created_nodes += summary_artigo.counters.nodes_created
            updated_nodes += summary_artigo.counters.nodes_deleted
            created_relations += summary_artigo.counters.relationships_created
            updated_relations += summary_artigo.counters.relationships_deleted

        return created_nodes, updated_nodes, created_relations, updated_relations

    ## Agrupando nós de artigos por revista
    # def persistir_artigos_revistas(self, session, id_lattes, dados):
    #     created_nodes = 0
    #     updated_nodes = 0
    #     created_relations = 0
    #     updated_relations = 0

    #     for dados_artigo in dados:
    #         ano = dados_artigo['ano']
    #         impact_jcr = dados_artigo['fator_impacto_jcr']
    #         issn = dados_artigo['ISSN']
    #         titulo = dados_artigo['titulo']
    #         revista = dados_artigo['revista']
    #         autores = dados_artigo['autores']
    #         qualis = dados_artigo['Qualis']
    #         doi = dados_artigo['DOI']
            
    #         query_create_node_artigo = """
    #             MERGE (p:Pesquisador {id_lattes: $id_lattes})
    #             CREATE (a:ArtigoPublicado {ano: $ano, impact_jcr: $impact_jcr, issn: $issn, titulo: $titulo, revista: $revista, autores: $autores, qualis: $qualis, doi: $doi})
    #             CREATE (p)-[:PRODUZ]->(a)
    #             MERGE (j:Revista {nome: $revista, issn: $issn})
    #             CREATE (a)-[:PUBLICADO_EM]->(j)
    #         """

    #         with session.begin_transaction() as tx:
    #             print("DEBUG: ISSN da revista:", issn)
    #             # Verificação de existência da revista
    #             revista_node = tx.run("""
    #                 MATCH (j:Revista {issn: $issn})
    #                 RETURN j
    #             """, issn=issn).single()

    #             if not revista_node:
    #                 # Revista não encontrada, crie-a
    #                 tx.run("""
    #                     CREATE (j:Revista {nome: $revista, issn: $issn})
    #                 """, revista=revista, issn=issn)

    #                 revista_node = tx.run("""
    #                     MATCH (j:Revista {issn: $issn})
    #                     RETURN j
    #                 """, issn=issn).single()

    #             if revista_node is not None:
    #                 print("DEBUG: Propriedades do nó Revista:", revista_node.items())  
    #                 node_revista = revista_node[0]  # Extrair o objeto Node
    #                 print(f"revista_node[0] {node_revista}")
    #                 issn = node_revista['issn']  # Acessar a propriedade 'issn'
    #                 print(f"node_revista['issn'] {issn}")
    #                 revista_nome = node_revista['nome_revista']
    #                 revista_area_avaliacao = node_revista['area_avaliacao']
    #                 revista_estrato = node_revista['estrato']
    #             else:
    #                 print("Erro: O retorno para a revista com ISSN", issn, "é None.")

    #             # Criação do nó do artigo
    #             tx.run(query_create_node_artigo, 
    #                 id_lattes=id_lattes, 
    #                 ano=ano,
    #                 impact_jcr=impact_jcr,
    #                 issn=issn,
    #                 titulo=titulo,
    #                 revista=revista,
    #                 autores=autores,
    #                 qualis=qualis,
    #                 doi=doi)

    #             # Criação do relacionamento PUBLICADO_EM
    #             tx.run("""
    #                 MATCH (a:ArtigoPublicado {doi: $doi}), (j:Revista {nome_revista: $revista_nome, issn: $issn, area_avaliacao: $revista_area_avaliacao, estrato: $revista_estrato})
    #                 CREATE (a)-[:PUBLICADO_EM]->(j)
    #             """, doi=doi, revista_nome=revista_nome, issn=issn, revista_area_avaliacao=revista_area_avaliacao, revista_estrato=revista_estrato)

    #             tx.commit()

    #         # Atualização dos contadores
    #         created_nodes += tx.run("MATCH (n) WHERE n:ArtigoPublicado RETURN count(n)").single()[0]
    #         updated_nodes += tx.run("MATCH (n) WHERE n:ArtigoPublicado SET n.updated_at = datetime() RETURN count(n)").single()[0]
    #         created_relations += tx.run("MATCH (r) WHERE r:PUBLICADO_EM RETURN count(r)").single()[0]
    #         updated_relations += tx.run("MATCH (r) WHERE r:PUBLICADO_EM SET r.updated_at = datetime() RETURN count(r)").single()[0]

    #     return created_nodes, updated_nodes, created_relations, updated_relations

    def buscar_revista_por_issn(session, issn):
        query = """
            MATCH (revista:Revista {issn: $issn})
            RETURN revista
        """

        try:
            result = session.run(query, issn=issn)
            return result.single()
        except Neo4jError as e:
            print(f"Erro Neo4j ao buscar a revista por ISSN: {e}")
            return None
        
    def persistir_artigos_revistas(self, session, id_lattes, dados):
        """
        Função para persistir os dados de artigos completos publicados em periódicos.

        Args:
            session (neo4j.Session): Sessão Neo4j.
            id_lattes (str): ID do Lattes do pesquisador.
            dados (dict): Dicionário contendo os dados dos artigos.

        Returns:
            None
        """

        for artigo in dados:
            # Extraindo informações do artigo
            ano = artigo['ano']
            impact_jcr = artigo['fator_impacto_jcr']
            issn = artigo['ISSN']
            titulo = artigo['titulo']
            revista = artigo['revista']
            autores = artigo['autores']
            data_issn = artigo['data_issn']
            doi = artigo['DOI']
            qualis = artigo['Qualis']

            query_create_node_artigo = """
                MERGE (p:Pesquisador {id_lattes: $id_lattes})
                CREATE (a:ArtigoPublicado {ano: $ano, impact_jcr: $impact_jcr, issn: $issn, titulo: $titulo, revista: $revista, autores: $autores, qualis: $qualis, doi: $doi})
                CREATE (p)-[:PRODUZ]->(a)
                MERGE (j:Revista {nome: $revista, issn: $issn})
                CREATE (a)-[:PUBLICADO_EM]->(j)
            """
            
            # Buscando o nó da revista
            revista_node = self.buscar_revista_por_issn(session, issn)

            # Criando o nó do artigo
            with session.begin_transaction() as tx:
                tx.run(query_create_node_artigo,
                    id_lattes=id_lattes,
                    ano=ano,
                    impact_jcr=impact_jcr,
                    issn=issn,
                    titulo=titulo,
                    revista=revista,
                    autores=autores,
                    data_issn=data_issn,
                    doi=doi,
                    qualis=qualis
                    )

                # Criando o relacionamento PUBLICADO_EM
                if revista_node is not None:
                    node_revista = revista_node[0][1]
                    if node_revista is not None:
                        revista_nome = node_revista['nome_revista']
                        revista_issn = node_revista['issn']
                        revista_area_avaliacao = node_revista['area_avaliacao']
                        revista_estrato = node_revista['estrato']
                    else:
                        print("Erro: O nó da revista não foi encontrado para o ISSN", issn)
                        # Lógica de tratamento de erro (opcional)
                else:
                    print("Erro: O retorno para a revista com ISSN", issn, "é None.")

                if revista_nome:
                    tx.run("""
                        MATCH (a:ArtigoPublicado {doi: $doi}), (j:Revista {nome_revista: $revista_nome, issn: $revista_issn, area_avaliacao: $revista_area_avaliacao, estrato: $revista_estrato})
                        CREATE (a)-[:PUBLICADO_EM]->(j)
                    """, doi=doi, revista_nome=revista_nome, revista_issn=revista_issn, revista_area_avaliacao=revista_area_avaliacao, revista_estrato=revista_estrato)

                tx.commit()

        # Atualização dos contadores
        with session.begin_transaction() as tx:
            created_nodes += tx.run("MATCH (n) WHERE n:ArtigoPublicado RETURN count(n)").single()[0]
            updated_nodes += tx.run("MATCH (n) WHERE n:ArtigoPublicado SET n.updated_at = datetime() RETURN count(n)").single()[0]
            created_relations += tx.run("MATCH (r) WHERE r:PUBLICADO_EM RETURN count(r)").single()[0]



    def persistir_resumos_congressos(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (r:ResumoCongresso {titulo: $titulo, ano: $ano, evento: $evento, autores: $autores, data_issn: $data_issn, doi: $doi})
        MERGE (p)-[:PRODUZ]->(r)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, **dados)
        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

    def persistir_apresentacoes_trabalho(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (a:ApresentacaoTrabalho {
            titulo: $titulo,
            ano: $ano,
            evento: $evento,
            autores: $autores
        })
        MERGE (p)-[:PRODUZ]->(a)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, **dados)

        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

    def persistir_outras_producoes_bibliograficas(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (o:OutrasProducoesBibliograficas {
            titulo: $titulo,
            ano: $ano,
            autores: $autores,
            doi: $doi
        })
        MERGE (p)-[:PRODUZ]->(o)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, **dados)

        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

    def persistir_orientacoes_concluidas(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (o:OrientacaoConcluida {
            tipo: $tipo,
            titulo: $titulo,
            ano: $ano,
            autor: $autor,
            instituicao: $instituicao
        })
        MERGE (p)-[:ORIENTA]->(o)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, **dados)

        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

    def persistir_participacoes_bancas(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (b:Banca {
            tipo: $tipo,
            titulo: $titulo,
            ano: $ano,
            instituicao: $instituicao
        })
        MERGE (p)-[:PARTICIPA_BANCA]->(b)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, **dados)

        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

    def persistir_projetos_pesquisa(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (pr:ProjetoPesquisa {
            titulo: $titulo,
            ano_inicio: $ano_inicio,
            ano_fim: $ano_fim,
            agencia_financiadora: $agencia_financiadora,
            valor_financiamento: $valor_financiamento
        })
        MERGE (p)-[:COORDENA]->(pr)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, dados=dados)

        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

    def persistir_premios_distincoes(self, session, id_lattes, dados):
        query_create_node = """
        MERGE (p:Pesquisador {id_lattes: $id_lattes})
        MERGE (pd:PremioDistincao {
            titulo: $titulo,
            ano: $ano,
            instituicao: $instituicao,
        })
        MERGE (p)-[:RECEBE]->(pd)
        """
        result = session.run(query_create_node, id_lattes=id_lattes, **dados)

        # Obtendo as informações de contadores
        summary = result.consume()
        created_nodes = summary.counters.nodes_created
        updated_nodes = summary.counters.nodes_deleted  
        created_relations = summary.counters.relationships_created
        updated_relations = summary.counters.relationships_deleted 

        return created_nodes, updated_nodes, created_relations, updated_relations

uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
persister = Neo4jPersister(uri, user, password)

### Persistir dados no Neo4j

In [ ]:
# dict_list
filename = 'dict_list.json'
json_data, formatted_creation_date, formatted_modification_date, time_count, unit = jfm.load_from_json(os.path.join(folder_data_input,filename))
print(f"\n{len(dict_list)} currículos carregados na lista de dicionários '{filename}'")

In [ ]:
persister.persistir_revistas_da_planilha()

In [ ]:
persister.persist_areas_nodes(json_data)

In [ ]:
persister.persist_pessoa_nodes(json_data)

In [ ]:
persister.persist_pesquisador_grande_area_relationships(json_data)

In [ ]:
persister.persist_producoes_pesquisador(json_data)

In [ ]:
import json
import logging

from neo4j import GraphDatabase

class Neo4jDataPersister:
    def __init__(self, uri, user, password):
        self._uri = uri
        self._user = user
        self._password = password
        self._driver = GraphDatabase.driver(self._uri, auth=(self._user, self._password))
        self.logger = logging.getLogger(__name__)

        self._node_created_count = 0
        self._node_updated_count = 0
        self._node_deleted_count = 0

        self._relationship_created_count = 0
        self._relationship_updated_count = 0
        self._relationship_deleted_count = 0

    def close(self):
        self._driver.close()

    def persist_data_from_json(self, json_data):
        with self._driver.session() as session:
            self._persist_data(session, json_data)

        self.logger.info("Total nodes created: %d", self._node_created_count)
        self.logger.info("Total nodes updated: %d", self._node_updated_count)
        self.logger.info("Total nodes deleted: %d", self._node_deleted_count)

        self.logger.info("Total relationships created: %d", self._relationship_created_count)
        self.logger.info("Total relationships updated: %d", self._relationship_updated_count)
        self.logger.info("Total relationships deleted: %d", self._relationship_deleted_count)

    def _persist_data(self, session, data):
        if isinstance(data, dict):  # Check if it's a dictionary
            for key, value in data.items():
                if key == 'Identificação':
                    self._handle_identificacao(session, value)
                elif key == 'Atuação Profissional':
                    self._handle_atuacao_profissional(session, value)
                elif key == 'Atuação Profissional':
                    self._handle_atuacao_profissional(session, value)                    
                elif key == 'Produções':
                    self._handle_producoes(session, value)
                elif isinstance(value, list) and not value:
                    # Ignore empty lists
                    pass
                elif isinstance(value, dict):
                    if 'JCR2' in value:
                        # Ignore the 'JCR2' subdictionary
                        del value['JCR2']

                    self._persist_data(session, value)

        elif isinstance(data, list):  # Check if it's a list
            for item in data:
                self._persist_data(session, item)  # Recurse on list items

        else:
            # Handle other data types (strings, numbers, etc.) or raise an error
            self.logger.warning("Unexpected data type: %s", type(data))


    def _handle_identificacao(self, session, data):
        for item in data:
            if item['campo'] == 'ID Lattes':
                node_id = item['valor']
                node = self._get_or_create_node(session, 'Pesquisador', {'ID Lattes': node_id})
                self._persist_other_properties(session, node, data)
            else:
                node = self._get_or_create_node(session, 'Pesquisador', {data.items()})

    def _handle_atuacao_profissional(self, session, data):
        for item in data:
            node = self._create_node(session, 'AtuaçãoProfissional', item)

    def _handle_producoes(self, session, data):
        for production_type, production_data in data.items():
            if production_type == 'Artigos completos publicados em periódicos':
                for article in production_data:
                    node = self._create_node(session, production_type, article)
            elif production_type.startswith('1.'):
                for index, article in enumerate(production_data):
                    node = self._create_node(session, production_type, article)
                    node['OrdemCronológica'] = index + 1

    def _get_or_create_node(self, session, label, properties):
        node = session.run("MATCH (n: {label}) WHERE {properties} RETURN n", {"label": label, "properties": properties}).single()

        if not node:
            node = session.run("CREATE (n: {label} {properties}) RETURN n", {"label": label, "properties": properties}).single()["n"]
            self._node_created_count += 1

        return node

    def _create_node(self, session, label, properties):
        node = session.run("CREATE (n: {label} {properties}) RETURN n", {"label": label, "properties": properties}).single()["n"]
        self._node_created_count += 1

        return node

    def _persist_other_properties(self, session, node, data):
        for key, value in data.items():
            if key not in ['campo', 'valor']:
                node[key] = value  

    def _create_relationship(self, session, start_node, relationship_type, end_node, properties={}):
        session.run("MATCH (a), (b) WHERE ID(a) = {start_node_id} AND ID(b) = {end_node_id} CREATE (a)-[r:{type} {props}]->(b) RETURN r", 
                    {"start_node_id": id(start_node), "end_node_id": id(end_node), "type": relationship_type, "props": properties})
        self._relationship_created_count += 1


In [ ]:
tipos=[]
subtipos=[]
for dict_pesq in dict_list:
    # Avaliando elementos da lista de dicionários do arquivo JSON de entrada
    if isinstance(dict_pesq, dict):
        for key1,val1 in dict_pesq.items():
            if key1 not in tipos:
                tipos.append(key1)            
            print(f'N01: Elementos armazenados em dicionário:')
            print(f'     {key2:25} | Tipo dos valores: {type(val2)}')
            print(f'       Conteúdo disponível nos valores do dicionário de Nível 01:')
            # Avaliando filhos de primeiro nível na hierarquia (Seções)
            if isinstance(val1, list):
                print(f'N01: Elementos armazenados em lista:')
                print(f'     Conteúdo disponível nos valores:')
                print(f'       {[x for x in val1]}')
            elif isinstance(val1, dict):
                for key2,val2 in val1.items():
                    if key2 not in subtipos:
                        subtipos.append(key2)
                    print(f'       Chave: {key2:25} | Tipo dos valores: {type(val2)}')
                    print(f'         Conteúdo disponível nos valores do dicionário de Nível 02:')
                    # Avaliando filhos de segundo nível na hierarquia (Tipos de Seções)
                    if isinstance(val2, dict):
                        for key3,val3 in val2.items():
                            if key3 not in tipos:
                                print(f'         Chave: {key3:25} | Tipo dos valores: {type(val3)}')
                                print(f'          Conteúdo disponível nos valores do dicionário de Nível 03:')
                                # Avaliando filhos de terceiro nível na hierarquia (Ocorrências de Tipos de Seções)
                                if isinstance(val3, dict):
                                    print(f'               Chave: {key3:25} | Tipo dos valores: {type(val3)}')
                                    print(f'                 Conteúdo disponível nos valores do dicionário de Nível 04:')
                                    # print(val3)
            else:
                print(f'N01: Elementos armazenados em {type(val1)}')
    else:
        print('ERRO NA ESTRUTURA DO JSON!!')

In [ ]:
# # Create a Neo4jDataPersister instance
# data_persister = Neo4jDataPersister('neo4j://localhost:7687', 'neo4j', 'password')
# filename = 'dict_list.json'
# json_data, formatted_creation_date, formatted_modification_date, time_count, unit = jfm.load_from_json(os.path.join(folder_data_input,filename))
# print(f"\n{len(dict_list)} currículos carregados na lista de dicionários '{filename}'")

# # Persist the data to Neo4j
# data_persister.persist_data_from_json(json_data)

In [ ]:
for pesq in json_data:
    print(len(pesq.get('Produções').keys()))

In [ ]:
def avaliar_tipo_instancias_arvore(estrutura, nivel=1, identacao=""):
    """
    Função recursiva que avalia o tipo e a quantidade de instâncias em cada nível da estrutura, exibindo-a em formato de árvore.

    Args:
        estrutura (list|dict): A estrutura a ser avaliada.
        nivel (int): Nível atual da recurssão (inicia em 1).
        identacao (str): String de indentação para cada nível (inicia vazia).

    Returns:
        None: A função imprime os resultados na tela e não retorna nada.
    """

    if isinstance(estrutura, list):
        print(f"{identacao}N{nivel}. Lista: {len(estrutura)} elementos")
        for item in estrutura:
            avaliar_tipo_instancias_arvore(item, nivel + 1, identacao + "    ")

    elif isinstance(estrutura, dict):
        print(f"{identacao}N{nivel}. Mapa: {estrutura.keys()}")
        for chave, valor in estrutura.items():
            print(f"{identacao}  {chave}:")
            avaliar_tipo_instancias_arvore(valor, nivel + 1, identacao + "    ")

    elif isinstance(estrutura, str):
        print(f"{identacao}N{nivel}. String: {estrutura}")

for dict_pesq in dict_list[:3]:
    avaliar_tipo_instancias_arvore(dict_pesq)

In [ ]:
for dict_pesq in dict_list:
    if isinstance(dict_pesq, list):
        print(f'L1. Lista: {dict_pesq}')
    elif isinstance(dict_pesq, dict):
        print(f'L1. Mapa: {dict_pesq.keys()}')
        for tipo in dict_pesq.values():
            if isinstance(tipo, list):
                print(f'    L2. Lista: {tipo}')
            elif isinstance(tipo, dict):
                print(f'    L2. Mapa: {tipo}')
                for subtipo in tipo.values():
                    if isinstance(subtipo, list):
                        print(f'        L3. Lista: {len(subtipo)} elementos')
                        for subsubtipo in subtipo:
                            if isinstance(subsubtipo, list):
                                print(f'            L4. Lista: {subsubtipo}')
                            elif isinstance(subsubtipo, dict):
                                print(f'            L4. Mapa: {subsubtipo.keys()}')
                                for subsubsubtipo in subsubtipo.values():
                                    if isinstance(subsubsubtipo, list):
                                        print(f'                L5. Lista: {subsubsubtipo}')
                                    elif isinstance(subsubsubtipo, dict):
                                        print(f'                L5. Mapa: {subsubsubtipo.keys()}')
                                    elif isinstance(subsubsubtipo, str):
                                        print(f'                L5. String: {subsubsubtipo}')                                  
                            elif isinstance(subsubtipo, str):
                                print(f'            L4. String: {subsubtipo}')     
                    elif isinstance(subtipo, dict):
                        print(f'        L3. Mapa: {subtipo.keys()}')
                        for subsubtipo in subtipo.values():
                            if isinstance(subsubtipo, list):
                                print(f'            L4. Lista: {subsubtipo}')
                            elif isinstance(subsubtipo, dict):
                                print(f'            L4. Mapa: {subsubtipo.keys()}')
                            elif isinstance(subsubtipo, str):
                                print(f'            L4. String: {subsubtipo}')                             
                    elif isinstance(tipo, str):
                        print(f'        L3. String: {subtipo}')                
            elif isinstance(tipo, str):
                print(f'    L2. String: {tipo}')
    elif isinstance(dict_pesq, str):
        print(f'L1. String: {dict_pesq}')


In [ ]:
dict_list[20:21]

In [ ]:
list(dict_list[20].get('Identificação').items())

In [ ]:
list(dict_list[5].get('Idiomas'))

In [ ]:
list(x.get('Descrição') for x in dict_list[5].get('Linhas de Pesquisa'))

In [ ]:
list(dict_list[5].get('Áreas').values())

In [ ]:
tipos=[]
for pesq in dict_list:
    for t, s in pesq.get('Produções').items():
        if t not in tipos:
            tipos.append(t)
            print(t, type(s))

In [ ]:
list(dict_list[1].get('Produções').get('Artigos completos publicados em periódicos')[0].keys())

In [ ]:
list(dict_list[5].get('Atuação Profissional'))

In [ ]:
list(dict_list[5].get('Formação').items())

## Classe para gerar dados para grafos

In [ ]:
# from neo4j import GraphDatabase
# import json

# class Neo4jConnection:
#     def __init__(self, uri, user, password):
#         self._driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self._driver.close()

#     def persist_data(self, data):
#         with self._driver.session() as session:
#             for person_data in data:
#                 person_id = person_data['Identificação']['ID Lattes']
#                 prepared_data, relationships = self._prepare_data(person_data, person_id)
#                 session.write_transaction(self._create_person_node, person_id, prepared_data)
#                 self._create_relationships(session, person_id, relationships)

#     @staticmethod
#     def _create_person_node(tx, person_id, person_data):
#         tx.run(
#             "MERGE (p:Person {id_lattes: $person_id}) "
#             "SET p += $person_data",
#             person_id=person_id,
#             person_data=person_data
#         )

#     def _prepare_data(self, data, person_id):
#         prepared_data = {}
#         relationships = []
#         for key, value in data.items():
#             if isinstance(value, dict):
#                 label = key.capitalize().replace(' ', '_')
#                 prepared_data.update({f'{label}_{k}': v for k, v in value.items()})
#                 sub_prepared_data, sub_relationships = self._prepare_data(value, person_id)
#                 prepared_data.update(sub_prepared_data)
#                 relationships.extend(sub_relationships)
#                 relationships.append((label, {'label': 'Person', 'person_id': person_id}))
#             elif isinstance(value, list):
#                 for idx, item in enumerate(value):
#                     if isinstance(item, dict):
#                         nested_label = f'{key}_{idx}'
#                         nested_prepared_data, nested_relationships = self._prepare_data(item, person_id)
#                         prepared_data.update({f'{nested_label}_{k}': v for k, v in item.items()})
#                         prepared_data.update(nested_prepared_data)
#                         relationships.extend(nested_relationships)
#                         relationships.append((nested_label, {'label': key.capitalize().replace(' ', '_'), 'person_id': person_id}))
#                     elif isinstance(item, str):
#                         # Tratar strings
#                         if ":" in item:
#                             label, value = item.split(":", 1)
#                             prepared_data[f'{key}_{idx}'] = value.strip()
#                             relationships.append((label.strip(), {'label': key.capitalize().replace(' ', '_'), 'person_id': person_id}))
#                         else:
#                             prepared_data[f'{key}_{idx}'] = item.strip()
#         return prepared_data, relationships

#     def _create_relationships(self, session, person_id, relationships):
#         for label, parent_id in relationships:
#             query = (
#                 f"MATCH (p:Person {{id_lattes: '{person_id}'}}) "
#                 f"MATCH (n:{label} {{id_lattes: '{person_id}_{label}'}}) "
#                 f"MERGE (p)-[:HAS_{label}]->(n)"
#             )
#             if parent_id:
#                 query += (
#                     f" MERGE (parent:{parent_id['label']} {{id_lattes: '{parent_id['person_id']}'}}) "
#                     f"MERGE (parent)-[:HAS_{label}]->(n)"
#                 )
#             session.run(query)

# # Dados de conexão com o banco de dados Neo4j
# uri = "bolt://localhost:7687"
# user = "neo4j"
# password = "password"

# # Conecta-se ao banco de dados Neo4j e persiste os dados
# connection = Neo4jConnection(uri, user, password)
# connection.persist_data(dict_list)
# connection.close()

In [ ]:
dict_list

In [ ]:
from datetime import datetime
from neo4j import GraphDatabase
import ast

class GraphPreparer:
    def __init__(self, uri, user, password):
        self._uri = uri
        self._user = user
        self._password = password
        self._driver = None

    def connect(self):
        self._driver = GraphDatabase.driver(self._uri, auth=(self._user, self._password))

    def prepare_data_from_file(self, file_path):
        nodes = []
        relationships = []

        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        for person_data in data:
            print(person_data.keys())
            person_id = person_data['Identificação'].get('ID Lattes')
            nodes.append(('Pesquisador', {'id_lattes': person_id}))
            
            for language in person_data.get('Idiomas', []):
                language_name = language['Idioma']
                proficiency = language['Proficiência']
                nodes.append(('Idioma', {'nome_idioma': language_name, 'proficiencia': proficiency}))
                relationships.append(('Pesquisador', {'id_lattes': person_id}, 'FALA', {'nome_idioma': language_name}))
            
            for academic_info in person_data.get('Formação', {}).get('Acadêmica', []):
                description = academic_info['Descrição']
                nodes.append(('Curso', {'descricao': description}))
                relationships.append(('Pesquisador', {'id_lattes': person_id}, 'REALIZOU', {'descricao': description}))
                
            for professional_info in person_data.get('Atuação Profissional', []):
                institution = professional_info['Instituição']
                description = professional_info['Descrição']
                relationships.append(('Pesquisador', {'id_lattes': person_id}, 'ATUA', {'instituicao': institution, 'descricao': description}))

        return nodes, relationships

    def persist_data(self, data_for_neo4j):
        try:
            with self._driver.session() as session:
                for data in data_for_neo4j:
                    if data[0] == 'Node':
                        label, properties = data[1:]
                        session.write_transaction(self._create_node, label, **properties)
                    elif data[0] == 'Relationship':
                        start_label, rel_type, end_label = data[1:]
                        session.write_transaction(self._create_relationship, start_label, rel_type, end_label)
        except Exception as e:
            print(f"Error persisting data: {e}")

    @staticmethod
    def _create_node(tx, label, **properties):
        query = f"CREATE (n:{label} $props)"
        tx.run(query, props=properties)

    @staticmethod
    def _create_relationship(tx, start_label, rel_type, end_label):
        query = "MATCH (a:{start_label}), (b:{end_label}) " \
                "CREATE (a)-[r:{rel_type}]->(b)"
        tx.run(query, start_label=start_label, rel_type=rel_type, end_label=end_label)

    def close(self):
        if self._driver is not None:
            self._driver.close()

# Exemplo de utilização:
graph_preparer = GraphPreparer(uri="bolt://localhost:7687", user="neo4j", password="password")
graph_preparer.connect()

filepath = os.path.join(folder_data_input,'dict_list.json')

In [ ]:
dict_list[0]['Idioma']

In [ ]:
filepath

In [ ]:
nodes, relationships = graph_preparer.prepare_data_from_file(filepath)

In [ ]:
data_for_neo4j = nodes + relationships
graph_preparer.persist_data(data_for_neo4j)

graph_preparer.close()


In [ ]:
from neo4j import GraphDatabase
from datetime import datetime

uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
graph_preparer = GraphPreparer(uri, user, password) # Instanciar classe GraphPreparer
graph_preparer.connect() # Conectar ao banco de dados Neo4j
graph_preparer.process_data(dict_list) # Processar e persistir dados

# Transformar os dados para o formato adequado do Neo4j
dados_para_neo4j = graph_preparer.transform_data_for_neo4j(dict_list)
graph_preparer.persist_nodes_and_relationships(dados_para_neo4j) # PersistiR nós e relações no Neo4j

In [ ]:
# dict_list

In [ ]:
dados_para_neo4j[:15]

In [ ]:
from neo4j import GraphDatabase
# Agora você pode inserir os dados no Neo4j utilizando sua biblioteca de acesso ao banco de dados Neo4j, por exemplo:
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
with neo4j_driver.session() as session:
    for item in data_for_neo4j:
        label, properties = item
        cypher_query = f"CREATE (:{label} $properties)"
        session.run(cypher_query, properties=properties)

In [ ]:
# # Função para criar nós e relações
# def create_nodes_and_relationships(tx, nodes_and_relationships):
#     for item in nodes_and_relationships:
#         if len(item) == 3:  # Se for uma relação
#             origin_node, rel_type, dest_node = item
#             tx.run(
#                 f"MATCH (a), (b) WHERE a.name = $origin_node AND b.name = $dest_node "
#                 "CREATE (a)-[r:" + rel_type + "]->(b)",
#                 origin_node=origin_node, dest_node=dest_node
#             )

# # Conectando ao banco de dados Neo4j e executando a transação
# with GraphDatabase.driver(uri, auth=(user, password)) as driver:
#     with driver.session() as session:
#         session.write_transaction(create_nodes_and_relationships, relacoes)

In [ ]:
import json

class Neo4jGraphGenerator:
    def __init__(self, dict_list):
        self.dict_list = dict_list
        self.graph_data = {'nodes': [], 'relationships': []}

    def generate_graph_json(self):
        for item in self.dict_list:
            id_lattes = item.get('Identificação', {}).get('ID Lattes')
            pesquisador = item.get('Identificação', {}).get('Nome')
            
            # Adicionar nó de pessoa (Person)
            person_node = {'id': id_lattes, 'label': 'Person', 'name': pesquisador}
            self.graph_data['nodes'].append(person_node)

            areas = item.get('Áreas', {})
            for area in areas.values():
                # Adicionar nó de Grande Área (GrandeArea)
                grande_area_node = {'id': area['Grande Área'], 'label': 'GrandeArea', 'description': area['Grande Área']}
                self.graph_data['nodes'].append(grande_area_node)
                self.graph_data['relationships'].append({'source': id_lattes, 'target': area['Grande Área'], 'type': 'BELONGS_TO'})
                
                for subarea in area.get('Subáreas', []):
                    # Adicionar nó de Área (Area)
                    area_node = {'id': subarea['Área'], 'label': 'Area', 'description': subarea['Área']}
                    self.graph_data['nodes'].append(area_node)
                    self.graph_data['relationships'].append({'source': area['Grande Área'], 'target': subarea['Área'], 'type': 'BELONGS_TO'})
                    
                    # Adicionar nó de Subárea (Subarea)
                    subarea_node = {'id': subarea['Subárea'], 'label': 'Subarea', 'description': subarea['Subárea']}
                    self.graph_data['nodes'].append(subarea_node)
                    self.graph_data['relationships'].append({'source': subarea['Área'], 'target': subarea['Subárea'], 'type': 'BELONGS_TO'})

        return json.dumps(self.graph_data, indent=4)

In [ ]:
generator = Neo4jGraphGenerator(dict_list)
generator.generate_graph_json()

In [ ]:
import json
import re
from neo4j import GraphDatabase

class AreasHandler:

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()
        
    def consult_areas_atuacao(self, name):
        with self._driver.session() as session:
            result = session.run("MATCH (p:Person {name: $name}) RETURN p.`Áreas de atuação` as areas_atuacao", name=name)
            record = result.single()
            if record:
                return record['areas_atuacao']
            return None

    def debug_and_convert(self, areas_str):
        try:
            return json.loads(areas_str)
        except json.JSONDecodeError:
            print(f"Failed to deserialize JSON string: {areas_str}")
            return None

    def extract_subarea(self, area_detail):
        # Extract the 'Subárea' content from the area detail
        match = re.search(r'Subárea: ([^/]+)', area_detail)
        if match:
            return match.group(1).strip()
        return None

    def extract_areas(self, area_detail):
        # Extract the 'Grande Área', 'Área', and 'Subárea' contents from the area detail
        grande_area_match = re.search(r'Grande área: ([^/]+)', area_detail)
        area_match = re.search(r'Área: ([^/]+)', area_detail)
        subarea_match = re.search(r'Subárea: ([^/]+)', area_detail)
        
        grande_area = grande_area_match.group(1).strip() if grande_area_match else None
        area = area_match.group(1).strip() if area_match else None
        subarea = subarea_match.group(1).strip() if subarea_match else None
        
        return grande_area, area, subarea

    def create_areas_relations(self, name):
        # Get 'Áreas de atuação' properties
        areas_properties = self.consult_areas_atuacao(name)

        # Convert the serialized JSON strings back into dictionaries
        try:
            deserialized_areas_properties = self.debug_and_convert(areas_properties)
        except Exception as e:
            print(f"Error deserializing 'Áreas de atuação' properties: {e}")
            return

        successful_areas_creations = 0

        with self._driver.session() as session:
            for _, area_detail in deserialized_areas_properties.items():
                try:
                    # Extracting Grande Área, Área, and Subárea from the details
                    match = re.match(r'Grande área: (.*?) / Área: (.*?) / Subárea: (.*?)(?:/Especialidade: (.*?))?\.?$', area_detail)
                    if not match:
                        print(f"Unexpected format for 'Áreas de atuação' detail: {area_detail}")
                        continue
                    grande_area, area, subarea = match.groups()[:3]

                    # Creating or merging nodes for Subárea, Área, and Grande Área
                    session.run("MERGE (s:Subárea {name: $subarea})", subarea=subarea)
                    session.run("MERGE (a:Área {name: $area})", area=area)
                    session.run("MERGE (ga:GrandeÁrea {name: $grande_area})", grande_area=grande_area)

                    # Creating or merging relationships. Using MERGE ensures no duplicate relationships are created.
                    session.run("MATCH (p:Person {name: $name}), (s:Subárea {name: $subarea}) MERGE (p)-[r:ATUA_EM]->(s)", name=name, subarea=subarea)
                    session.run("MATCH (ga:GrandeÁrea {name: $grande_area}), (a:Área {name: $area}) MERGE (ga)-[r:CONTÉM_ÁREA]->(a)", grande_area=grande_area, area=area)
                    session.run("MATCH (a:Área {name: $area}), (s:Subárea {name: $subarea}) MERGE (a)-[r:CONTEM_SUBÁREA]->(s)", area=area, subarea=subarea)

                    successful_areas_creations += 1
                except Exception as e:
                    print(f"Error processing 'Áreas de atuação' detail '{area_detail}': {e}")

            # Inform the user about areas
            print(f"{successful_areas_creations} 'Áreas de atuação' relations successfully created.")

In [ ]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

handleareas = AreasHandler(uri, user, password)
name = 'Antonio Marcos Aires Barbosa'
handleareas.consult_areas_atuacao(name)

In [ ]:
class ProjectsHandler:

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def consult_data_by_property(self, name, property_name):
        with self._driver.session() as session:
            result = session.run(f"MATCH (p:Person {{name: $name}}) RETURN p.`{property_name}` as data", name=name)
            record = result.single()
            return record['data'] if record else None

    def create_projects_relations(self, name):
        successful_creations = 0

        with self._driver.session() as session:
            # Process 'Atuação Profissional' data
            professional_data = self.consult_data_by_property(name, 'Atuação Profissional')
            if professional_data:
                for institution_name, _ in json.loads(professional_data).items():
                    session.run("MERGE (i:Instituição {name: $institution_name})", institution_name=institution_name)
                    print(f"Institution node created/merged for: {institution_name}")

                    session.run("MATCH (p:Person {name: $name}), (i:Instituição {name: $institution_name}) MERGE (p)-[:TEM]->(i)", name=name, institution_name=institution_name)
                    print(f"Relationship established between {name} and {institution_name}.")

            # Process other dynamic nodes
            key_labels_to_check = ['Linhas de pesquisa', 'Projetos de pesquisa', 'Projetos de extensão', 'Projetos de desenvolvimento']
            for key in key_labels_to_check:
                formatted_key = f"`{key}`"  # Wrap the key with backticks
                project_data = self.consult_data_by_property(name, key)
                if project_data:
                    for project_time, project_name in json.loads(project_data).items():
                        if project_name:  # to avoid empty names
                            session.run(f"MERGE (p:{formatted_key} {{name: $project_name}})", project_name=project_name)
                            print(f"{key} node created/merged for: {project_name}")

                            session.run(f"MATCH (a:Person {{name: $name}}), (p:{formatted_key} {{name: $project_name}}) MERGE (a)-[:TEM]->(p)", name=name, project_name=project_name)
                            print(f"Relationship established between {name} and {project_name} ({key}).")
                            successful_creations += 1
                else:
                    print(f"'{key}' data not found for {name}")

        print(f"{successful_creations} projetos atualizados com sucesso.")


In [ ]:
import json
import re
from neo4j import GraphDatabase

class ArticleHandler:

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def fetch_person_productions(self, name):
        with self._driver.session() as session:
            result = session.run("MATCH (p:Person {name: $name}) RETURN p.Produções as produções", name=name)
            record = result.single()
            return record['produções'] if record else None

    def extract_article_info(self, input_str):
        # Encontre todas as abreviaturas de iniciais em maiúsculas e seus índices
        abbreviations = [(match.group(), match.start()) for match in re.finditer(r'\b[A-Z]\.', input_str)]

        # Encontre a posição da maior ocorrência de abreviaturas de iniciais, se houver
        if abbreviations:
            max_abbr_position = max(abbreviations, key=lambda x: x[1])

            # Encontre a primeira ocorrência de '. ' ou ' . ' após a maior ocorrência de abreviaturas de iniciais
            first_separator_candidates = [
                input_str.find('. ', max_abbr_position[1] + 3),
                input_str.find(' . ', max_abbr_position[1] + 3),
                input_str.find('.. ')
            ]
            first_separator_candidates = [pos for pos in first_separator_candidates if pos != -1]

            if first_separator_candidates:
                first_separator = min(first_separator_candidates)

                # Encontre a primeira ocorrência de '. ' após o primeiro separador
                second_separator = input_str.find('. ', first_separator + 2)

                # Encontre a primeira ocorrência de ', ' após o segundo separador
                third_separator = input_str.find(', ', second_separator + 2)
            else:
                first_separator = second_separator = third_separator = -1
        else:
            first_separator = second_separator = third_separator = -1

        # Defina o padrão para encontrar "p." e o conteúdo até a próxima vírgula
        pages_match = re.search(r' p\.\s*(.*?),', input_str)
        pages = pages_match.group(1) if pages_match else ""

        # Defina o padrão para encontrar "v." e o conteúdo até a próxima vírgula
        volume_match = re.search(r' v\.\s*(.*?),', input_str)
        volume = volume_match.group(1) if volume_match else ""

        # Encontre a primeira ocorrência de um ano de quatro dígitos seguido de ponto final após o terceiro separador
        year_match = re.search(r' \d{4}\.', input_str[third_separator + 2:])
        year = year_match.group().strip('.').strip() if year_match else ""

        # Extraia os dados com base nas posições dos separadores
        authors = input_str[:first_separator].strip()
        title = input_str[first_separator + 2:second_separator].strip()
        journal = input_str[second_separator + 2:third_separator].strip()

        # Verifique se a lista de autores e o título não estão vazios
        if not authors or not title:
            return None  # Retorna None para indicar falha

        # Crie um dicionário com os dados extraídos
        article_info = {
            "authors": authors,
            "title": title,
            "original_title": journal,
            "pages": pages,
            "volume": volume,
            "year": year
        }

        return article_info
    
    def deserialize_and_create_nodes(self, name):
        print(f"Fetching 'Produções' data for {name}...")
        productions_data = self.fetch_person_productions(name)
        
        if not productions_data:
            print(f"'Produções' data not found or empty for {name}.")
            return

        print(f"Attempting to deserialize 'Produções' data for {name}...")
        try:
            productions_data = json.loads(productions_data)
        except json.JSONDecodeError as e:
            print(f"Failed to deserialize 'Produções' data for {name}: {e}")
            return

        successful_articles = 0
        unsuccessful_articles = []

        with self._driver.session() as session:
            print(f"Processing 'Produção bibliográfica' for {name}...")
            bibliographic_production = productions_data.get("Produção bibliográfica", {})
            
            if isinstance(bibliographic_production, str):
                print(f"Attempting to deserialize 'Produção bibliográfica' for {name}...")
                try:
                    bibliographic_production = json.loads(bibliographic_production)
                except json.JSONDecodeError as e:
                    print(f"Failed to deserialize 'Produção bibliográfica' for {name}: {e}")
                    return

            articles = json.loads(bibliographic_production.get("Artigos completos publicados em periódicos", "{}"))

            for _, article_str in articles.items():
                article_details = self.extract_article_info(article_str)

                # Vamos imprimir os detalhes de cada artigo e verificar se os autores estão presentes.
                print(f"Original Article: {article_str}")
                print(f"Extracted Details: {article_details}")

                if article_details:
                    article_details["title"] = article_details["title"].strip()
                    article_details["original_title"] = article_details["original_title"].strip()

                    session.run(f"MERGE (a:Artigo {{title: $title}}) SET a += $details", title=article_details["title"], details=article_details)
                    session.run(f"MATCH (p:Person {{name: $name}}), (a:Artigo {{title: $title}}) MERGE (p)-[:PUBLICOU]->(a)", name=name, title=article_details["title"])
                    successful_articles += 1
                else:
                    unsuccessful_articles.append(article_str)

        print(f"Processed {successful_articles} articles successfully for {name}.")

        if unsuccessful_articles:
            print("Failed to process the following articles:")
            for article in unsuccessful_articles:
                print(article)

    def process_articles(self, name):
        self.deserialize_and_create_nodes(name)

In [ ]:
# class JcrHandler:
#     def __init__(self, uri, user, password):
#         self._driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self._driver.close()

#     def _consultar_propriedades_jcr(self, tx, name):
#         query = (
#             "MATCH (p:Person {name: $name})"
#             "RETURN p.JCR AS jcr"
#         )
#         result = tx.run(query, name=name)
#         return [record["jcr"] for record in result]
   
#     ## Versão para usar com criação de nós secundários retorna JSON
#     def consultar_propriedades_jcr(self, name):
#         with self._driver.session() as session:
#             query = (
#                 "MATCH (p:Person {name: $name})"
#                 "RETURN p.JCR AS jcr"
#             )
#             result = session.run(query, name=name)
#             jcr_data = result.single()["jcr"]
#             jcr_properties_list = json.loads(jcr_data)
#             return jcr_properties_list

#     @staticmethod
#     def _convert_list_to_dict(lst):
#         """
#         Converts a list into a dictionary with indices as keys.
        
#         Parameters:
#         - lst: list, input list to be transformed.
        
#         Returns:
#         - dict: Transformed dictionary.
#         """
#         return {str(i): item for i, item in enumerate(lst)}
    
#     def create_person_with_jcr(self, name, jcr_properties):
#         with self._driver.session() as session:
#             session.write_transaction(self._create_person_with_jcr, name, jcr_properties)

#     @staticmethod
#     def _create_person_with_jcr(tx, name, jcr_properties):
#         # Cria o nó Person
#         person_query = (
#             "CREATE (p:Person {name: $name}) "
#             "RETURN p"
#         )
#         person_result = tx.run(person_query, name=name)
#         person_node = person_result.single()[0]

#         # Cria os nós secundários para cada valor único de data-issn
#         data_issn_values = set(prop.get("data-issn") for prop in jcr_properties)
#         for data_issn in data_issn_values:
#             if data_issn:
#                 secondary_node_query = (
#                     "CREATE (s:SecondaryNode {data_issn: $data_issn}) "
#                     "RETURN s"
#                 )
#                 tx.run(secondary_node_query, data_issn=data_issn)

#                 # Cria a relação entre o nó Person e o nó secundário
#                 relation_query = (
#                     "MATCH (p:Person {name: $name}), (s:SecondaryNode {data_issn: $data_issn}) "
#                     "CREATE (p)-[:HAS_JCR]->(s)"
#                 )
#                 tx.run(relation_query, name=name, data_issn=data_issn)

#     def createJournalsNodes(self, name):
#         # Get JCR properties
#         jcr_properties = self.consultar_propriedades_jcr(name)

#         # Convert the serialized JSON strings back into dictionaries
#         deserialized_jcr_properties = [json.loads(prop) for prop in jcr_properties.values()]

#         # Inform the user about the total number of JCR property entries
#         total_entries = len(deserialized_jcr_properties)
#         print(f"Read {total_entries} entries from JCR properties of Person '{name}'.")

#         # Extract relevant journal properties and their count
#         journal_counts = Counter(prop.get("data-issn") for prop in deserialized_jcr_properties)
        
#         # Number of unique ISSNs
#         unique_issns = len(journal_counts)
#         print(f"Identified {unique_issns} unique ISSN values.")

#         null_count = journal_counts.pop(None, 0)  # Remove None (null) ISSN and get its count
#         null_count += journal_counts.pop("NULL", 0)  # Also account for "NULL" as a string

#         # Counters for journals
#         successful_journal_creations = 0

#         with self._driver.session() as session:
#             for data_issn, count in journal_counts.items():
#                 if data_issn and data_issn != "NULL":
#                     representative_entry = next(prop for prop in deserialized_jcr_properties if prop.get("data-issn") == data_issn)
#                     journal_name = representative_entry.get("original_title")
#                     fator_impacto = representative_entry.get("impact-factor")
#                     jcr_year = representative_entry.get("jcr-year")

#                     # Create or merge the Journal node
#                     journal_node_query = (
#                         "MERGE (j:Revistas {ISSN: $data_issn}) "
#                         "ON CREATE SET j.name = $journal_name, j.FatorImpacto = $impact_factor, j.JCRYear = $jcr_year "  # Corrected this line
#                         "RETURN j"
#                     )
#                     session.run(journal_node_query, data_issn=data_issn, journal_name=journal_name, impact_factor=fator_impacto, jcr_year=jcr_year)  # And this line

#                     # Create or update the "PUBLICOU_EM" relationship
#                     relation_query = (
#                         "MATCH (p:Person {name: $name}), (j:Revistas {ISSN: $data_issn}) "  # corrected this line
#                         "MERGE (p)-[r:PUBLICOU_EM]->(j) "
#                         "ON CREATE SET r.QuantidadePublicações = $count "
#                         "ON MATCH SET r.QuantidadePublicações = r.QuantidadePublicações + $count"
#                     )
#                     session.run(relation_query, name=name, data_issn=data_issn, count=count)
                    
#                     successful_journal_creations += 1
                
#                 if null_count:
#                     # For example, to print the count:
#                     pass
        
#         # Inform the user about journals
#         print(f"{successful_journal_creations} Revistas adicionadas com sucesso.")
#         print(f"{null_count} Revistas não foram criadas por terem valor NULL de ISSN.")

In [ ]:
from neo4j import GraphDatabase
import urllib.parse
import json
import re

class AdvisorHandler:

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    @staticmethod
    def convert_to_primitives(input_data):
        if input_data is None:
            return None
        if isinstance(input_data, dict):
            for key, value in input_data.items():
                if isinstance(value, dict):  
                    input_data[key] = json.dumps(AdvisorHandler.convert_to_primitives(value), ensure_ascii=False)
                else:
                    input_data[key] = AdvisorHandler.convert_to_primitives(value)
            return input_data
        elif isinstance(input_data, list):
            return [AdvisorHandler.convert_to_primitives(item) for item in input_data]
        elif isinstance(input_data, str):
            if 'http://' in input_data or 'https://' in input_data:
                parts = input_data.split(" ")
                new_parts = [urllib.parse.quote(part) if part.startswith(('http://', 'https://')) else part for part in parts]
                return " ".join(new_parts)
            return input_data
        elif isinstance(input_data, (int, float, bool)):
            return input_data
        else:
            return str(input_data)

    @staticmethod
    def debug_and_convert(input_data):
        try:
            return AdvisorHandler.convert_to_primitives(input_data)
        except:
            print("Conversion failed for:", input_data)
            raise

    def consult_orientacoes(self, name):
        with self._driver.session() as session:
            query = (
                "MATCH (p:Person {name: $name})"
                "RETURN p.Orientações AS orientacoes"
            )
            result = session.run(query, name=name)
            orient_data = result.single()["orientacoes"]
            if orient_data is None:
                raise ValueError(f"No data found for 'Orientações' attribute for Person '{name}'")
            orient_properties_list = json.loads(orient_data)
            return orient_properties_list

    def create_advisor_relations(self, name):
        # Get Orientações properties
        orient_properties = self.consult_orientacoes(name)

        # Convert the serialized JSON strings back into dictionaries
        try:
            deserialized_orient_properties = self.debug_and_convert(orient_properties)
        except Exception as e:
            print(f"Error deserializing Orientações properties: {e}")
            return

        # Advisory relationship mapping
        advisory_types = {
            "Dissertação de mestrado": "ORIENTOU_MESTRADO",
            "Tese de doutorado": "ORIENTOU_DOUTORADO",
            "Trabalho de conclusão de curso de graduação": "ORIENTOU_GRADUAÇÃO"
        }

        successful_advisory_creations = 0

        with self._driver.session() as session:
            for orientacao_category, advisories in deserialized_orient_properties.items():
                if isinstance(advisories, str):
                    try:
                        advisories = json.loads(advisories)
                    except json.JSONDecodeError:
                        print(f"Failed to deserialize JSON string in 'Orientações' for category '{orientacao_category}': {advisories}")
                        continue

                if not isinstance(advisories, dict):
                    print(f"Unexpected data type in 'Orientações' for category '{orientacao_category}': {advisories}")
                    continue

                for advisory_type, relationships in advisories.items():
                    relation_label = advisory_types.get(advisory_type)
                    if not relation_label:
                        continue  # skip if the advisory type is not one of the specified ones

                    for _, detail in json.loads(relationships).items():
                        try:
                            student_name = detail.split(".")[0]
                            title = detail.split(".")[1]
                            
                            # Extract the year from the detail string
                            year_match = re.search(r'(\d{4})', detail)
                            year = year_match.group(1) if year_match else None

                            # Create or merge the Orientações node
                            node_query = (
                                "MERGE (a:Orientações {Title: $title}) "
                                "ON CREATE SET a.StudentName = $student_name, a.Tipo = $advisory_type, a.Year = $year "
                                "ON MATCH SET a.Tipo = $advisory_type, a.Year = $year "  # Ensure the 'Tipo' and 'Year' are always updated
                                "RETURN a"
                            )
                            session.run(node_query, title=title, student_name=student_name, advisory_type=advisory_type, year=year)

                            # Create or update the advisory relationship
                            relation_query = (
                                f"MATCH (p:Person {{name: $name}}), (a:Orientações {{Title: $title}}) "
                                f"MERGE (p)-[r:{relation_label}]->(a) "
                            )
                            session.run(relation_query, name=name, title=title)

                            successful_advisory_creations += 1
                        except Exception as e:
                            print(f"Error processing advisory '{detail}': {e}")

        # Inform the user about advisories
        print(f"{successful_advisory_creations} orientações atualizadas com sucesso.")

In [ ]:
from neo4j import GraphDatabase

class DataRemovalHandler:

    def __init__(self, uri, user, password):
        """
        Inicializa a classe DataRemovalHandler com informações de conexão ao banco de dados Neo4j.

        Parâmetros:
        - uri (str): URI de conexão ao Neo4j.
        - user (str): Nome de usuário para autenticação.
        - password (str): Senha para autenticação.
        """
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        """
        Fecha a conexão com o banco de dados Neo4j.
        """
        self._driver.close()

    def delete_nodes_by_label(self, label):
        """
        Deleta todos os nós associados a um label específico no Neo4j.

        Parâmetro:
        - label (str): O label dos nós a serem deletados.

        Retorna:
        - int: O número de nós deletados.
        """
        with self._driver.session() as session:
            # Esta consulta combina com todos os nós do label especificado e os deleta
            result = session.run(f"MATCH (n:{label}) DETACH DELETE n RETURN count(n) as deleted_count")
            deleted_count = result.single()["deleted_count"]
            return deleted_count

# Interação com dados do e-Lattes

- Fazer análise com lista dos nomes no e-lattes e baixar os arquivos gerados
- Salvar na pasta _data/in_zip

## Grupos de Funções antigas para refatorar nas classes

In [ ]:
# !pip3 install levenshtein
# !pip3 install editdistance
# !pip3 install seaborn
# !pip3 install pyjarowinkler

import os
import re
import csv
import json
import time
import json
import zipfile
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from string import Formatter
from datetime import date, timedelta
from datetime import datetime as dt
from unidecode import unidecode
from plotly.subplots import make_subplots
from pyjarowinkler.distance import get_jaro_distance
from IPython.display import clear_output, display, HTML

pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')

class PreparadorDePublicacoes:
    def __init__(self):
        self.data = []
        self.colunas = ['idLattes', 'nome', 'tipo', 'titulo_do_capitulo', 'idioma', 'titulo_do_livro', 'ano', 'doi', 'pais_de_publicacao', 'isbn', 
        'nome_da_editora', 'numero_da_edicao_revisao', 'organizadores', 'paginas', 'autores', 'autores-endogeno', 'autores-endogeno-nome', 'tags', 
        'Hash', 'tipo_producao', 'natureza', 'titulo', 'nome_do_evento', 'ano_do_trabalho', 'pais_do_evento', 'cidade_do_evento', 'classificacao', 
        'periodico', 'volume', 'issn', 'estrato_qualis', 'editora', 'numero_de_paginas', 'numero_de_volumes']

    def extrair_dados(self, registro, tipo_producao):
        linha = {coluna: None for coluna in self.colunas}
        linha['tipo_producao'] = tipo_producao  # Define o tipo de produção com base na chave do dicionário
        
        # Mapear diretamente os campos do registro para a linha, assegurando que todos os campos desejados sejam extraídos
        for campo in ['titulo', 'idioma', 'periodico', 'ano', 'volume', 'issn', 'estrato_qualis', 'pais_de_publicacao', 'paginas', 'doi']:
            linha[campo] = registro.get(campo, '')

        # Tratar os autores como uma string concatenada se eles existirem
        linha['autores'] = '; '.join(registro.get('autores', []))

        # Tratar os campos 'autores-endogeno' e 'autores-endogeno-nome'
        if 'autores-endogeno' in registro and registro['autores-endogeno']:
            id_endogeno = registro['autores-endogeno'][0]
            linha['idLattes'] = id_endogeno
            linha['nome'] = registro['autores-endogeno-nome'][0].get(id_endogeno, None)
        
        # Adicionar os outros campos conforme necessário aqui
        # Por exemplo, tratamento de 'tags', 'Hash', etc., quando necessário

        return linha

    def processar_publicacoes(self):
        linhas = []
        # Itera diretamente sobre cada registro em self.data
        for registro in self.data:
            linha = self.extrair_dados(registro, registro.get('tipo_producao', 'Desconhecido'))
            linhas.append(linha)
        return linhas

    def extract_zips(self, pathzip):
        destination = os.path.join(os.getcwd(), '_data', 'in_json')
        if not os.path.exists(destination):
            os.makedirs(destination)
            print(f"Criada pasta para armazenar dados descompactados: {destination}")
        else:
            print(f"Descompactando arquivos para: {destination}")

        with zipfile.ZipFile(pathzip, 'r') as zip_ref:
            zip_ref.extractall(destination)
        
        print("Descompactação concluída...")
        return destination  # Retorna o caminho onde os arquivos foram descompactados

    def find_and_merge_publication_json_files(self, pathjson):
        all_data = []
        for filename in os.listdir(pathjson):
            if 'publication.json' in filename:
                print(f"Extraindo dados do arquivo '{filename}'...")
                with open(os.path.join(pathjson, filename), 'r', encoding='utf-8') as file:
                    file_data = json.load(file)
                    for tipo_producao in file_data:  # Para cada tipo de produção no arquivo
                        for ano in file_data[tipo_producao]:  # Para cada ano dentro de um tipo de produção
                            for registro in file_data[tipo_producao][ano]:  # Itera sobre cada registro
                                # Adicionar ou atualizar o campo 'tipo_producao' em cada registro
                                registro_atualizado = registro.copy()  # Fazer cópia para evitar modificar o original
                                registro_atualizado['tipo_producao'] = tipo_producao  # Atualizar ou adicionar o campo 'tipo_producao'
                                all_data.append(registro_atualizado)  # Adicionar o registro atualizado à lista

        # Salva a lista unificada em um novo arquivo JSON
        unified_json_path = os.path.join(pathjson, 'unified_pub.json')
        with open(unified_json_path, 'w', encoding='utf-8') as unified_file:
            json.dump(all_data, unified_file, ensure_ascii=False, indent=4)
        
        print(f"Arquivo unificado criado em: {unified_json_path}")

        # Atualiza self.data com os dados unidos
        self.data = all_data

    def merge_publication_json_files(self, pathjson):
        """
        This function receives a path to a JSON folder, accesses the folder's contents, searches for files
        containing 'publication.json' in their filename, merges their contents and saves the resulting
        data as a CSV file in destination folder.
        """
        data = []
        # Looping through the files in the given path
        for filename in os.listdir(pathjson):
            if 'publication.json' in filename:
                print(f"Extraindo dados do arquivo {filename}...")
                # Opening the file and appending its data to the list
                with open(os.path.join(pathjson, filename), 'r', encoding='utf-8') as file:
                    data.append(json.load(file))

        # Creating the output directory if it doesn't exist
        destination = os.path.join(os.getcwd(), '_data','powerbi')
        if not os.path.exists(destination):
            os.mkdir(destination)

        # Writing the merged data to a CSV file
        print(f"Criando arquivo CSV...")
        with open(os.path.join(destination, 'publication.csv'), 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            # Writing the header based on the keys of the first item in the list
            writer.writerow(data[0].keys())
            # Looping through the items in the list and writing them as rows in the CSV file
            for item in data:
                writer.writerow(item.values())

    def exportar_para_csv(self, nome_arquivo='publicacoes.csv'):
        linhas = self.processar_publicacoes()
        df = pd.DataFrame(linhas, columns=self.colunas)
        filepathcsv = os.path.join("./","_data","powerbi",nome_arquivo)
        df.to_csv(filepathcsv, index=False)
        print(f'Arquivo criado com sucesso em {filepathcsv}')

    def ler_csv_dados(self, pathdata,filename):
        filepath = os.path.join(pathdata,filename)
        df_pub=pd.read_csv(filepath)
        tipos = df_pub['tipo_producao'].value_counts()
        qualis=df_pub['estrato_qualis'].value_counts()  
        quantidade_nan = df_pub['estrato_qualis'].isna().sum()
        tipos_qualis = qualis.count()
        percentual_nan=np.round(quantidade_nan/tipos.sum()*100,1)
        print(f"\n{tipos.sum()} linhas no total, distribuídas nos seguintes tipos de produção:")
        print(f"{quantidade_nan} linhas ({percentual_nan}% do total de linhas) com NaN no campo estrato_qualis")
        print(tipos)
        ano_min=df_pub['ano'].min()
        ano_max=df_pub['ano'].max()
        print(f"\n{tipos.get('PERIODICO', 0)} publicações em periódicos, no período de {int(ano_min)} a {int(ano_max)}")
        percentual=np.round(qualis.sum()/tipos.get('PERIODICO', 0)*100,1)
        print(f"{qualis.sum()} publicações ({percentual}%) classificadas no qualis periodicos conforme estratificação:")
        quantidade_nan_periodico = df_pub[df_pub['tipo_producao'] == 'PERIODICO']['estrato_qualis'].isna().sum()
        percentual_nan_periodico = np.round(quantidade_nan_periodico/tipos.get('PERIODICO', 0)*100,1)
        print(f"{quantidade_nan_periodico} publicações em periódicos ({percentual_nan_periodico}%) com valor 'NaN' no estrato_qualis\n")
        print(qualis)
        return df_pub

    def ler_xls_dados(self, pathdata,filename):
        filepath = os.path.join(pathdata,filename)
        df_pub=pd.read_excel(filepath)
        tipos = df_pub['tipo_producao'].value_counts()
        qualis=df_pub['estrato_qualis'].value_counts()
        tipos_qualis = qualis.count()
        print(f"\n{tipos.sum()} linhas no total, distribuídas nos seguintes tipos de produção:")
        print(tipos)
        ano_min=df_pub['ano'].min()
        ano_max=df_pub['ano'].max()
        print(f"\n{tipos.get('PERIODICO', 0)} publicações em periódicos, no período de {int(ano_min)} a {int(ano_max)}")
        percentual=np.round(qualis.sum()/tipos.get('PERIODICO', 0)*100,1)
        print(f"{qualis.sum()} publicações ({percentual}%) classificadas no qualis periodicos conforme estratificação:")
        print(qualis)
        return df_pub

    def sigla_producao(self, tipo_producao, separador=""):
        """
        Função para converter os valores da coluna 'tipo_producao' em siglas, desconsiderando preposições,
        fazendo split de 'tipo_producao' em palavras também pelo caractere "_".
        
        Parâmetros:
        - tipo_producao: O valor da coluna 'tipo_producao'.
        - separador: O caractere para separar as iniciais. Padrão é "_".
        
        Retorna:
        - Uma string que é a sigla formada pelas iniciais das palavras em 'tipo_producao', desconsiderando preposições.
        """
        # Lista de preposições para serem ignoradas
        preposicoes = ['e', 'de', 'do', 'da', 'dos', 'das', 'em', 'na', 'no', 'nas', 'nos', 'por', 'para', 'com']

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "DTPB": 1, #DEMAIS_TIPOS_DE_PRODUCAO_BIBLIOGRAFICA 
            "TJ": 2, #TEXTO_EM_JORNAIS
            "E": 3, #EVENTO
            "CL": 4, #CAPITULO_DE_LIVRO
            "L": 5, #LIVRO
            "AA": 6, #ARTIGO_ACEITO
            "P": 7, #PERIODICO
        }

        # Fazendo split por espaço e por underscore
        palavras = tipo_producao.replace('_', ' ').replace('-', ' ').split()

        # Inicializar a lista de iniciais
        iniciais = []

        for palavra in palavras:
            # Verificar se a palavra não é uma preposição e não está vazia
            if palavra.lower() not in preposicoes and palavra:
                # Adicionar a inicial em maiúsculo à lista de iniciais
                iniciais.append(palavra[0].upper())

        # Juntar as iniciais para formar a sigla
        sigla = "".join(iniciais)

        # Aplicar o mapeamento para encontrar o número correspondente à sigla
        numero = mapeamento_siglas.get(sigla, "")

        # Retornar a string formatada com o número e a sigla
        return f"{numero}-{sigla}"  

    def sigla_qualis(self, estrato_qualis, separador=""):
        """
        Função para converter os valores da coluna 'estrato_qualis' em siglas com números de ordenação,
        
        Parâmetros:
        - estrato_qualis: O valor da coluna 'estrato_qualis'.
        - separador: O caractere para separar as iniciais. Padrão é "_".
        
        Retorna:
        - Uma string que é a sigla formada pelas iniciais das palavras em 'estrato_qualis' antecedida por seu número de ordenação.
        """

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "C": 1,
            "B4": 2,
            "B3": 3,
            "B2": 4,
            "B1": 5,
            "B2": 6,
            "B1": 7,
            "A4": 8,
            "A3": 9,
            "A2": 10,
            "A1": 11,
        }

        # Aplicar o mapeamento para encontrar o número correspondente à sigla
        numero = mapeamento_siglas.get(estrato_qualis, "")

        # Retornar a string formatada com o número e a sigla
        return f"{numero}-{estrato_qualis}"

    def agrupar_publicacoes(self, filename):
        # Caminho para o arquivo CSV
        pathfilename = os.path.join(os.getcwd(), '_data', 'powerbi', filename)
        df = pd.read_csv(pathfilename)

        # Use .loc para modificar o DataFrame diretamente e evitar o aviso
        df.loc[:, 'sigla_producao'] = df['tipo_producao'].apply(lambda x: self.sigla_producao(x))
        df_publicacoes = df[['idLattes','ano','sigla_producao']]

        # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
        contagem_publicacoes = df_publicacoes.groupby(['ano','sigla_producao','idLattes']).size().reset_index(name='contagem')

        # Retornar a contagem de orientações por tipo e por ano
        return contagem_publicacoes

    def agrupar_qualis(self, filename):
        # Caminho para o arquivo CSV
        pathfilename = os.path.join(os.getcwd(), '_data', 'powerbi', filename)
        df = pd.read_csv(pathfilename)

        # # Use .loc para modificar o DataFrame diretamente e evitar o aviso
        df.loc[:, 'estrato_qualis'] = df['estrato_qualis'].apply(lambda x: self.sigla_qualis(x))
        df_publicacoes = df[['idLattes','ano','estrato_qualis']]

        # Remover todas as linhas que contêm pelo menos um valor NaN
        df_publicacoes_limpo = df_publicacoes.dropna()

        # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
        contagem_qualis = df_publicacoes_limpo.groupby(['ano','estrato_qualis','idLattes']).size().reset_index(name='contagem')

        # Retornar a contagem de orientações por tipo e por ano
        return contagem_qualis

    def plotar_producoes_barras_empilhadas(self, filename):
        contagem_publicacoes = self.agrupar_publicacoes(filename)
        fig = make_subplots(rows=1, cols=1)
        anos = sorted(contagem_publicacoes['ano'].unique())

        mapeamento_siglas_para_numeros = {
            "1-DTPB": 1, # Demais tipos de produção bibliográfica
            "2-TJ": 2, # Texto em jornais
            "3-E": 3, # Evento
            "4-CL": 4, # Capítulo de Livro
            "5-L": 5, # Livro
            "6-AA": 6, # Artigo Aceito
            "7-P": 7, # Periódico
        }

        naturezas_ordenadas = sorted(contagem_publicacoes['sigla_producao'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

        for natureza in naturezas_ordenadas:
            contagem_por_ano = []
            labels_por_ano = [] # Para armazenar os rótulos de dados
            for ano in anos:
                contagem = contagem_publicacoes[(contagem_publicacoes['ano'] == ano) & (contagem_publicacoes['sigla_producao'] == natureza)]['contagem'].sum()
                if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4]:
                    contagem = -contagem
                contagem_por_ano.append(contagem)
                labels_por_ano.append(str(contagem)) # Convertendo a contagem em string para usar como rótulo
            
            # Adicionar a barra ao gráfico com rótulos de dados
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza, text=labels_por_ano, textposition='auto'))

        # Atualizar o layout para permitir barras empilhadas, ajustar o eixo Y para mostrar valores negativos, e garantir que todos os anos sejam mostrados no eixo X
        fig.update_layout(
            barmode='relative',
            title_text='Contagem de Produções por Tipo e Ano (Produção de divulgação plotadas abaixo do eixo X)',
            xaxis_title="Ano",
            yaxis_title="Quantidade de Produções Biliográficas",
            yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),
        )        
        # Garantir que todos os anos apareçam na barra de rótulos do eixo X
        fig.update_xaxes(tickmode='array', tickvals=anos)
        fig.show()


    def plotar_qualis_barras_empilhadas(self, filename):
        contagem_publicacoes = self.agrupar_qualis(filename)
        contagem_publicacoes.replace("-nan", np.nan, inplace=True)
        contagem_publicacoes.dropna(inplace=True)
        fig = make_subplots(rows=1, cols=1)
        anos = sorted(contagem_publicacoes['ano'].unique())

        mapeamento_siglas_para_numeros = {
            "1-C": 1, "2-B4": 2, "3-B3": 3, "4-B2": 4, "5-B1": 5,
            "6-B2": 6, "7-B1": 7, "8-A4": 8, "9-A3": 9, "10-A2": 10, "11-A1": 11,
        }
        naturezas_ordenadas = sorted(contagem_publicacoes['estrato_qualis'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))
        
        # Obter a paleta de cores 'Greens'
        cores = px.colors.sequential.Greens
        # Certificar que temos cores suficientes para todas as barras, repetindo a paleta se necessário
        num_barras = len(naturezas_ordenadas)
        cores_repetidas = cores * (num_barras // len(cores) + 1)
        
        for index, natureza in enumerate(naturezas_ordenadas):
            contagem_por_ano = []
            for ano in anos:
                contagem = contagem_publicacoes[(contagem_publicacoes['ano'] == ano) & (contagem_publicacoes['estrato_qualis'] == natureza)]['contagem'].sum()
                if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4, 5, 6, 7]:
                    contagem = -contagem
                contagem_por_ano.append(contagem)
            
            # Usar uma cor da paleta para cada barra
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza, marker_color=cores_repetidas[index]))

        fig.update_layout(
            barmode='relative',
            title_text='Contagem de Produções por Estrato Qualis e Ano (Qualis abaixo de B plotado do eixo X)',
            xaxis_title="Ano",
            yaxis_title="Quantidade de Publicações",
            yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),
        )
        # Garantir que todos os anos apareçam na barra de rótulos do eixo X
        fig.update_xaxes(tickmode='array', tickvals=anos)        
        fig.show()

class PreparadorDeOrientacoes():
    def __init__(self):
        self.data = []
        self.colunas = ['id_lattes_orientadores', 'tipo_orientacao', 'natureza', 'titulo', 'idioma', 'ano', 'id_lattes_aluno', 'nome_aluno', 'instituicao', 'pais', 'curso', 'codigo_do_curso', 'bolsa', 'agencia_financiadora', 'codigo_agencia_financiadora', 'nome_orientadores', 'tags', 'Hash']
        self.pathjson = os.path.join('_data','in_json')
        # self.filename = '863.advise.json'

        # Abrir o arquivo e carregando o JSON
    def open_file(self, filename):
        with open(os.path.join(self.pathjson, filename), 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        linhas_achatadas = self.extrair_orientacoes(data)
        df = pd.DataFrame(linhas_achatadas)
        df.to_csv('orientacoes_achatadas.csv', index=False)
        return df
    
    def agrupar_orientacoes(self, filename):
        # Caminho para o arquivo JSON
        pathfilename = os.path.join('_data', 'in_json', filename)
        # dict_orientacoes = pd.read_json(pathfilename)

        # Carregar o JSON e aplicar a função
        with open(pathfilename, 'r', encoding='utf-8') as file:
            data = json.load(file)

        linhas_achatadas = self.extrair_orientacoes(data)
        df = pd.DataFrame(linhas_achatadas)

        # Salvar os dados achatados em um CSV
        df.to_csv('orientacoes_achatadas.csv', index=False)

        # Use .loc para modificar o DataFrame diretamente e evitar o aviso
        df.loc[:, 'sigla_natureza'] = df['natureza'].apply(lambda x: self.sigla_natureza(x))
        df_orientacoes = df[['id_lattes_orientadores','ano','sigla_natureza']]

        # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
        contagem_orientacoes = df_orientacoes.groupby(['ano','sigla_natureza','id_lattes_orientadores']).size().reset_index(name='contagem')

        # Retornar a contagem de orientações por tipo e por ano
        return contagem_orientacoes

    # Prosseguir com a definição e uso da função para extrair as orientações
    def extrair_orientacoes(self, json_data):
        colunas = self.colunas
        linhas = []

        # Iterar sobre cada tipo de orientação e ano
        for tipo_orientacao, anos in json_data.items():
            for ano, orientacoes in anos.items():
                for orientacao in orientacoes:
                    # Preparar um dicionário para cada linha de acordo com as colunas definidas
                    linha = {}
                    # Adicionar dados específicos da orientação
                    for campo in orientacao:
                        if campo in colunas:
                            linha[campo] = orientacao[campo]
                    # Tratar 'id_lattes_orientadores' como uma lista, juntar os IDs com vírgula se houver mais de um
                    linha['id_lattes_orientadores'] = ', '.join(orientacao.get('id_lattes_orientadores', []))
                    # Adicionar o tipo de orientação como uma coluna
                    linha['tipo_orientacao'] = tipo_orientacao
                    # Adicionar o ano, garantindo que sobreponha qualquer valor de 'ano' nos dados individuais
                    linha['ano'] = ano
                    linhas.append(linha)

        return linhas

    def sigla_natureza(self, natureza, separador=""):
        """
        Função para converter os valores da coluna 'natureza' em siglas, desconsiderando preposições,
        fazendo split de 'natureza' em palavras também pelo caractere "_".
        
        Parâmetros:
        - natureza: O valor da coluna 'natureza'.
        - separador: O caractere para separar as iniciais. Padrão é "_".
        
        Retorna:
        - Uma string que é a sigla formada pelas iniciais das palavras em 'natureza', desconsiderando preposições.
        """
        # Lista de preposições para serem ignoradas
        preposicoes = ['e', 'de', 'do', 'da', 'dos', 'das', 'em', 'na', 'no', 'nas', 'nos', 'por', 'para', 'com']

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "OON": 1, #Orientações de Outra Natureza
            "IC": 2, #Iniciação Científica
            "MCCAE": 3, #Monografia de Conclusão Curso de Atualização ou Especialização
            "SPD": 4, #
            "TCCG": 5, #Trabalho de Conclusão de Curso de Graduação
            "DM": 6, #Dissertação de Mestrado
            "TD": 7, #Tese de Doutorado
        }

        # Fazendo split por espaço e por underscore
        palavras = natureza.replace('_', ' ').replace('-', ' ').split()

        # Inicializar a lista de iniciais
        iniciais = []

        for palavra in palavras:
            # Verificar se a palavra não é uma preposição e não está vazia
            if palavra.lower() not in preposicoes and palavra:
                # Adicionar a inicial em maiúsculo à lista de iniciais
                iniciais.append(palavra[0].upper())

        # Juntar as iniciais para formar a sigla
        sigla = "".join(iniciais)

        # Aplicar o mapeamento para encontrar o número correspondente à sigla
        numero = mapeamento_siglas.get(sigla, "Desconhecido")

        # Retornar a string formatada com o número e a sigla
        return f"{numero}-{sigla}"     

    def plotar_orientacoes_barras_agrupadas(self, filename):
        # montar dataframe com agrupamento de orientações por tipo por ano no programa
        contagem_orientacoes = self.agrupar_orientacoes(filename)

        # Criar uma figura com subplots
        fig = make_subplots(rows=1, cols=1)

        # Encontrar anos e naturezas únicos para as orientações
        anos = sorted(contagem_orientacoes['ano'].unique())

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "OON": 1,
            "IC": 2,
            "MCCAE": 3,
            "SPD": 4,
            "TCCG": 5,
            "DM": 6,
            "TD": 7,
        }

        # Ordenar as naturezas com base nos números associados em mapeamento_siglas
        mapeamento_siglas_para_numeros = {sigla: int(sigla.split('-')[0]) for sigla in contagem_orientacoes['sigla_natureza'].unique()}
        naturezas_ordenadas = sorted(contagem_orientacoes['sigla_natureza'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

        # Criar uma barra para cada tipo de orientação em cada ano
        for natureza in naturezas_ordenadas:
            contagem_por_ano = []
            for ano in anos:
                # Somar as contagens para cada ano e natureza
                contagem = contagem_orientacoes[(contagem_orientacoes['ano'] == ano) & (contagem_orientacoes['sigla_natureza'] == natureza)]['contagem'].sum()
                contagem_por_ano.append(contagem)
            
            # Adicionar a barra ao gráfico
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza))

        # Atualizar o layout para permitir barras agrupadas
        fig.update_layout(barmode='group', title_text='Contagem de Orientações no Programa por Tipo e Ano', xaxis_title="Ano", yaxis_title="Quantidade de Orientações")

        # Mostrar o gráfico
        fig.show()


    def plotar_orientacoes_barras_empilhadas(self, filename):
        # montar dataframe com agrupamento de orientações por tipo por ano no programa
        contagem_orientacoes = self.agrupar_orientacoes(filename)
        
        # Criar uma figura com subplots
        fig = make_subplots(rows=1, cols=1)

        # Encontrar anos únicos para as orientações
        anos = sorted(contagem_orientacoes['ano'].unique())

        # Mapeamento para siglas de tipos de orientações
        mapeamento_siglas = {
            "OON": 1,
            "IC": 2,
            "MCCAE": 3,
            "SPD": 4,
            "TCCG": 5,
            "DM": 6,
            "TD": 7,
        }

        # Mapeamento para ordenar siglas por duração/complexidade
        mapeamento_siglas_para_numeros = {
            "1-OON": 1,
            "2-IC": 2,
            "3-MCCAE": 3,
            "4-SPD": 4,
            "5-TCCG": 5,
            "6-DM": 6,
            "7-TD": 7,
        }

        # Ordenar as naturezas baseando-se no número extraído da sigla
        naturezas_ordenadas = sorted(contagem_orientacoes['sigla_natureza'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

        # Criar uma barra para cada tipo de orientação em cada ano, seguindo a ordem definida
        for natureza in naturezas_ordenadas:
            contagem_por_ano = []
            for ano in anos:
                # Somar as contagens para cada ano e natureza
                contagem = contagem_orientacoes[(contagem_orientacoes['ano'] == ano) & (contagem_orientacoes['sigla_natureza'] == natureza)]['contagem'].sum()
                # Se a sigla estiver associada ao número 1 ou 2, tornar a contagem negativa
                if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4]:
                    contagem = -contagem
                contagem_por_ano.append(contagem)
            
            # Adicionar a barra ao gráfico
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza))

        # Atualizar o layout para permitir barras empilhadas e ajustar o eixo Y para mostrar valores negativos
        fig.update_layout(
            barmode='relative',  # Usar 'relative' para empilhar incluindo valores negativos
            title_text='Contagem de Orientações por Tipo e Ano (Orientações de curta duração plotadas abaixo do eixo X)',
            xaxis_title="Ano",
            yaxis_title="Quantidade de Orientações",
            yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),  # Destacar a linha zero
        )

        # Mostrar o gráfico
        fig.show()


In [ ]:
path = "./"
print(f"     Pasta corrente: {path}")
pathjson = os.path.join(path,'_data','in_json')
print(f"Pasta arquivos JSON: {pathjson}")
try:
    pathdata = os.path.join(path,'_data','powerbi')
    print(f" Pasta de dados CSV: {pathdata}")
except:
    print('Pasta de dados ainda não existe.')

print("\nConteúdo da pasta JSON:")
list(os.listdir(pathjson))

In [ ]:
## Descompactar arquivo zipado na pasta JSON
prep_pub = PreparadorDePublicacoes()
pathfilezip = '_data/in_zip/890.files.zip'
destination = prep_pub.extract_zips(pathfilezip)

## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Ler e montar análise exploratória
filename='publicacoes.csv'
df_pub = prep_pub.ler_csv_dados(pathdata,filename)

In [ ]:
filename='890.advise.json'
pathjson = os.path.join('_data','in_json')
pathfilename = os.path.join(pathjson,filename)
print(pathfilename)
dict_orientacoes = pd.read_json(pathfilename)
print(f'{len(dict_orientacoes):02} dicionários com dados de orientações')

In [ ]:
dict_orientacoes.keys()

In [ ]:
prep_ort = PreparadorDeOrientacoes()
prep_ort.plotar_orientacoes_barras_agrupadas(filename)
prep_ort.plotar_orientacoes_barras_empilhadas(filename)

In [ ]:
pathcsv = os.path.join('_data','powerbi')
os.listdir(pathcsv)

# Demais passos

Calcular índice de publicação em conjunto com alunos do programa

- Levantar nome dos discentes do programa
- Levantar os nomes de autores nas publicações de docentes
- Identificar por similadidade as publicações onde constam nome de alunos do programa

    Padronizar nomes de autores

In [ ]:
def limpar_nomes(linha_texto):
    '''
    Retira erros e sujeira da string formada pela lista de nomes de autor dos artigos retirada do Lattes
     Recebe: Uma string com os nomes de autor
    Retorna: Uma string com os nomes de autor, removidas preposições, acentos, e demais erros pontuais
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    string = linha_texto.replace('Network for Genomic Surveillance in South Africa;Network for Genomic Surveillance in South Africa (NGS-SA);10.1002/jmv.27190;','')
    string = string.replace('Autores: ','').replace('(Org)','').replace('(Org.)','').replace('et. al.','').replace('et al','').replace('(Org).','').replace('.','').replace('\'','')
    string = string.replace(',,,',',').replace(',,',',').replace(';',', ').replace('-',' ').replace('S?', 'SA').replace('S?', 'SA').replace('ARA?JO', 'ARAUJO').replace('FL?VIO','FLAVIO').replace('F?BIO','FABIO').replace('VIT?RIO','VITORIO')
    string = re.sub(r'[0-9]+', '', string)
    partes_string = string.split(' ')

    ## Retirar partes de nomes caso sejam preposições
    preposicoes = ['da', 'de', 'do', 'das', 'dos', ' e ']
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)

    ## Retirar símbolos não unicode, como acentuação gráfica e cedilha
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', string) if not unicodedata.combining(ch))
    
    ## Retirar iniciais juntas, separando-as com espaço em branco
    letras_duasconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2}$')            # Duas Letras consoantes maiúsculas juntas do início ao final da string
    letras_tresconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3}$')            # Três Letras consoantes maiúsculas juntas do início ao final da string
    letras_duasconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2},$')       # Duas Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    letras_tresconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3},$')       # Três Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    
    partes_nome=[]
    for j in string.split(' '):
        div_ltrduasconsnts = letras_duasconsnts.findall(j)
        div_ltrtriplicadas = letras_tresconsnts.findall(j)
        div_ltrduasconsntsvirg = letras_duasconsntsvirg.findall(j)
        div_ltrtresconsntsvirg = letras_tresconsntsvirg.findall(j)
        if div_ltrduasconsnts or div_ltrtriplicadas:
            iniciais_separadas = ' '.join(x for x in j)
            partes_nome.append(iniciais_separadas)
        elif div_ltrduasconsntsvirg or div_ltrtresconsntsvirg:
            iniciais_separadas = ' '.join(x for x in j[:-1])
            partes_nome.append(iniciais_separadas+',')
        else:
            partes_nome.append(j)
    string = ' '.join(x for x in partes_nome).strip()
    return string



def padronizar_nome(linha_texto):
    '''
    Procura sobrenomes e abreviaturas e monta nome completo
     Recebe: String com todos os sobrenomes e nomes, abreviados ou não
    Retorna: Nome completo no formato padronizado em SOBRENOME AGNOME, Partes de nomes
      Autor: Marcos Aires (Mar.2022)
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    partes_string = linha_texto.split(' ')

    ## Retirar partes de nomes caso sejam preposições
    preposicoes = ['da', 'de', 'do', 'das', 'dos', ' e ']
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)

    ## Retirar símbolos não unicode, como acentuação gráfica e cedilha
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', string) if not unicodedata.combining(ch))
    
    ## Expressões regulares para encontrar padrões de divisão de nomes de autores
    sobrenome_inicio   = re.compile(r'^[A-ZÀ-ú-a-z]+,')                  # Sequência de letras maiúsculas no início da string
    sobrenome_composto = re.compile(r'^[A-ZÀ-ú-a-z]+[ ][A-ZÀ-ú-a-z]+,')  # Duas sequências de letras no início da string, separadas por espaço, seguidas por vírgula
    letra_abrevponto   = re.compile(r'^[A-Z][.]')                        # Uma letra maiúscula no início da string, seguida por ponto
    letra_abrevespaco  = re.compile(r'^[A-Z][ ]')                        # Uma letra maiúscula no início da string, seguida por espaço
    letras_dobradas    = re.compile(r'[A-Z]{2}')                         # Duas letras maiúsculas juntas
    letras_dobradasini = re.compile(r'[A-Z]{2}[ ]')                      # Duas letras maiúsculas juntas, seguidas por espaço
    letras_dobradasfim = re.compile(r'[ ][A-Z]{2}')                      # Duas letras maiúsculas juntas, precedidas por espaço

    ## Expressões regulares para encontrar iniciais juntas, separando-as com espaço em branco
    letras_duasconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2}$')            # Duas Letras consoantes maiúsculas juntas do início ao final da string
    letras_tresconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3}$')            # Três Letras consoantes maiúsculas juntas do início ao final da string
    letras_duasconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2},$')       # Duas Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    letras_tresconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3},$')       # Três Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula

    ## Agnomes e preprosições a tratar, agnomes vão em maiúsculas para sobrenome e preposições vão em minúsculas para restante das partes de nomes
    nomes=[]
    agnomes       = ['NETO','JUNIOR','FILHO','SEGUNDO','TERCEIRO','SOBRINHO']
    preposicoes   = ['de','da','do','das','dos', ' e ']
    nome_completo = ''
    
    ## Ajustar lista de termos, identificar sobrenomes compostos e ajustar sobrenome com ou sem presença de vírgula
    div_sobrenome   = sobrenome_inicio.findall(string)
    div_sbrcomposto = sobrenome_composto.findall(string)
    
    # print('-'*100)
    # print('                 Recebido:',string)
    
    ## Caso haja vírgulas na string, tratar sobrenomes e sobrenomes compostos
    if div_sobrenome != [] or div_sbrcomposto != []:
        # print('CASO_01: Há víruglas na string')
        div = string.split(', ')
        sobrenome     = div[0].strip().upper()
        try:
            div_espaco    = div[1].split(' ')
        except:
            div_espaco    = ['']
        primeiro      = div_espaco[0].strip('.').strip()
        
        # print('     Dividir por vírgulas:',div)
        # print('      Primeira DivVirgula:',sobrenome)
        # print('Segunda DivVrg/DivEspaços:',div_espaco)
        # print('      Primeira DivEspaços:',primeiro)
               
        # Caso primeiro nome seja somente duas letras maiúsculas juntas, trata-se de duas iniciais
        if len(primeiro)==2 or letras_tresconsnts.findall(primeiro) or letras_duasconsnts.findall(primeiro):
            # print('CASO_01.a: Há duas letras ou três letras consoantes juntas, são iniciais')
            primeiro_nome=primeiro[0].strip()
            # print('          C01.a1_PrimNome:',primeiro_nome)
            nomes.append(primeiro[1].strip().upper())
            try:
                nomes.append(primeiro[2].strip().upper())
            except:
                pass
            try:
                nomes.append(primeiro[3].strip().upper())
            except:
                pass            
        else:
            # print('CASO_01.b: Primeiro nome maior que 2 caracteres')
            primeiro_nome = div_espaco[0].strip().title()
            # print('          C01.a2_PrimNome:',primeiro_nome)
        
        ## Montagem da lista de partes de nomes do meio
        for nome in div_espaco:
            # print('CASO_01.c: Para cada parte de nome da divisão por espaços após divisão por vírgula')
            if nome not in nomes and nome.lower()!=primeiro_nome.lower() and nome.lower() not in primeiro_nome.lower() and nome!=sobrenome:   
                # print('CASO_01.c1: Se o nome não está nem como primeiro nome, nem sobrenomes')
                # print(nome, len(nome))
                
                ## Avaliar se é abreviatura seguida de ponto e remover o ponto
                if len(nome)<=2 and nome.lower() not in preposicoes:
                    # print('    C01.c1.1_Nome<=02:',nome)
                    for inicial in nome:
                        # print(inicial)
                        if inicial not in nomes and inicial not in primeiro_nome:
                            nomes.append(inicial.replace('.','').strip().title())
                elif len(nome)==3 and nome.lower() not in preposicoes:
                        # print('    C01.c1.2_Nome==03:',nome)
                        for inicial in nome:
                            if inicial not in nomes and inicial not in primeiro_nome:
                                nomes.append(inicial.replace('.','').strip().title())
                else:
                    if nome not in nomes and nome!=primeiro_nome and nome!=sobrenome and nome!='':
                        if nome.lower() in preposicoes:
                            nomes.append(nome.replace('.','').strip().lower())
                        else:
                            nomes.append(nome.replace('.','').strip().title())
                        # print(nome,'|',primeiro_nome)
                        
        ## Caso haja sobrenome composto que não esteja nos agnomes considerar somente primeira parte como sobrenome
        if div_sbrcomposto !=[] and sobrenome.split(' ')[1] not in agnomes and sobrenome.split(' ')[0].lower() not in preposicoes:
            # print('CASO_01.d: Sobrenome composto sem agnomes')
            # print(div_sbrcomposto)
            # print('Sobrenome composto:',sobrenome)
            
            nomes.append(sobrenome.split(' ')[1].title())
            sobrenome = sobrenome.split(' ')[0].upper().strip()
            # print('Sobrenome:',sobrenome)
            
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('    Nomes:',nomes)
        
        ## Caso haja preposição como agnome desconsiderar e passar para final dos nomes
        if div_sbrcomposto !=[] and sobrenome.split(' ')[0].lower() in preposicoes:
            # print('CASO_01.e: Preposição no Sobrenome passar para o final dos nomes')
            # print('   div_sbrcomposto:', div_sbrcomposto)
            # print('Sobrenome composto:',div_sbrcomposto)
            
            nomes.append(div_sbrcomposto[0].split(' ')[0].lower())
            # print('    Nomes:',nomes)
            sobrenome = div_sbrcomposto[0].split(' ')[1].upper().strip(',').strip()
            # print('Sobrenome:',sobrenome)
            
            for i in nomes:
                # print('CASO_01.e1: Para cada nome avaliar se o sobrenome está na lista')
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('  Nomes:',nomes)
        
        # print('Ao final do Caso 01')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('           Lista de nomes:',nomes, len(nomes),'nomes')
        
    ## Caso não haja vírgulas na string considera sobrenome o último nome da string dividida com espaço vazio
    else:
        # print('CASO_02: Não há víruglas na string')
        try:
            div = string.split(' ')
            # print('      Divisões por espaço:',div)
            
            if div[-1] in agnomes: # nome final é um agnome
                sobrenome     = div[-2].upper().strip()+' '+div[-1].upper().strip()
                for i in div[1:-2]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.title().strip())
                    if i in preposicoes:
                        nomes.append(i.lower().strip())
            else:
                if len(div[-1]) > 2:
                    sobrenome     = div[-1].upper().strip()
                    primeiro_nome = div[1].title().strip()
                    for i in div[1:-1]:
                        if i != sobrenome and i not in preposicoes:
                            nomes.append(i.title().strip())
                        if i in preposicoes:
                            nomes.append(i.lower().strip())
                else:
                    sobrenome     = div[-2].upper().strip()
                    for i in div[-1]:
                        nomes.append(i.title())
                    primeiro_nome = nomes[0].title().strip()
                    for i in div[1:-1]:
                        if i != sobrenome and i not in preposicoes:
                            nomes.append(i.title().strip())
                        if i in preposicoes:
                            nomes.append(i.lower().strip())
        except:
            sobrenome = div[-1].upper().strip()
            for i in div[1:-1]:
                    if i != sobrenome and i not in preposicoes:
                        nomes.append(i.title().strip())
                    if i in preposicoes:
                        nomes.append(i.lower().strip())
            
        if sobrenome.lower() != div[0].lower().strip():
            primeiro_nome=div[0].title().strip()
        else:
            primeiro_nome=''
        
        # print('Ao final do Caso 02')
        # print('    Sobrenome sem vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome sem vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio sem vírgula:',nomes, len(nomes),'nomes')
    
    ## Encontrar e tratar como abreviaturas termos com apenas uma ou duas letras iniciais juntas, com ou sem ponto
    for j in nomes:
        # print('CASO_03: Avaliar cada nome armazenado na variável nomes')
        # Procura padrões com expressões regulares na string
        div_sobrenome      = sobrenome_inicio.findall(j)
        div_sbrcomposto    = sobrenome_composto.findall(j)
        div_abrevponto     = letra_abrevponto.findall(j)
        div_abrevespaco    = letra_abrevespaco.findall(j)
        div_ltrdobradasini = letras_dobradasini.findall(j)
        div_ltrdobradasfim = letras_dobradasfim.findall(j)
        div_ltrdobradas    = letras_dobradas.findall(j)
        div_ltrduasconsnts = letras_duasconsnts.findall(j)
        div_ltrtriplicadas = letras_tresconsnts.findall(j)
        tamanho=len(j)
        # print('\n', div_ltrdobradasini, div_ltrdobradasfim, tamanho, 'em:',j,len(j))
        
        ## Caso houver abreviatura com uma letra em maiúscula nos nomes
        if div_abrevponto !=[] or tamanho==1:
            # print('CASO_03.1: Há abreviaturas uma letra maiúscula nos nomes')
            nome = j.replace('.','').strip()
            if nome not in nomes and nome != sobrenome and nome != primeiro_nome:
                # print('CASO_03.1a: Há abreviaturas uma letra maiúscula nos nomes')
                nomes.append(nome.upper())
        
        ## Caso houver duas inicias juntas em maiúsculas
        elif div_ltrdobradasini !=[] or div_ltrdobradasfim !=[] or div_ltrdobradas !=[] :
            # print('CASO_03.2: Há abreviaturas uma letra maiúscula nos nomes')
            for letra in j:
                # print('CASO_03.2a: Avaliar cada inicial do nome')
                if letra not in nomes and letra != sobrenome and letra != primeiro_nome:
                    # print('CASO_03.2a.1: Se não estiver adicionar inicial aos nomes')
                    nomes.append(letra.upper())
        
        # Caso haja agnomes ao sobrenome
        elif sobrenome in agnomes:
            # print('CASO_03.3: Há agnomes nos sobrenomes')
            sobrenome = nomes[-1].upper()+' '+sobrenome
            # print(sobrenome.split(' '))
            # print('Sobrenome composto:',sobrenome)
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
            
        else:
            # print('CASO_03.4: Não há agnomes nos sobrenomes')
            if j not in nomes and j not in sobrenome and j != primeiro_nome:
                if len(nomes) == 1:
                    nomes.append(j.upper())
                elif 1 < len(nomes) <= 3:
                    nomes.append(j.lower())
                else:
                    nomes.append(j.title())
         
        # print('Ao final do Caso 03')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
        
    nomes_meio=' '.join([str for str in nomes]).strip()
    # print('        Qte nomes do meio:',nomes,len(nomes))
    
    if primeiro_nome.lower() == sobrenome.lower():
        # print('CASO_04: Primeiro nome é igual ao sobrenome')
        try:
            primeiro_nome=nomes_meio.split(' ')[0]
        except:
            pass
        try:
            nomes_meio.remove(sobrenome)
        except:
            pass
    
        # print('Ao final do caso 04')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
    
    ## Caso sobrenome seja só de 1 letra passá-lo para nomes e considerar o próximo nome como sobrenome
    for i in range(len(div)):
        if len(sobrenome)==1 or sobrenome.lower() in preposicoes:
            # print('CASO_05: Mudar sobrenomes até o adequado')
            div    = string.split(', ')
            # print('Divisão por vírgulas:',div)
            avaliar0       = div[0].split(' ')[0].strip()
            if 1< len(avaliar0) < 3:
                # print('CASO_05.1: 1 < Sobrenome < 3 fica em minúsculas')
                sbrn0          = avaliar0.lower()
            else:
                # print('CASO_05.2: Sobrenome de tamanho 1 ou maior que 3 fica em maiúsculas')
                sbrn0          = avaliar0.title()
            # print('sbrn0:',sbrn0, len(sbrn0))
            
            try:
                avaliar1=div[0].split(' ')[1].strip()
                # print('avaliar0',avaliar0)
                # print('avaliar1',avaliar1)
                if 1 < len(avaliar1) <=3:
                    sbrn1     = avaliar1.lower()
                else:
                    sbrn1     = avaliar1.title()
                # print('sbrn1:',sbrn1, len(sbrn1))

            except:
                pass

            if div != []:
                # print('CASO_05.3: Caso haja divisão por vírgulas na string')
                try:
                    div_espaco     = div[1].split(' ')
                except:
                    div_espaco     = div[0].split(' ')
                sobrenome      = div_espaco[0].strip().upper()
                try:
                    primeiro_nome  = div_espaco[1].title().strip()
                except:
                    primeiro_nome  = div_espaco[0].title().strip()
                if len(sbrn0) == 1:
                    # print('CASO_05.3a: Avalia primeiro sobrenome de tamanho 1')
                    # print('Vai pros nomes:',str(sbrn0).title())
                    nomes_meio = nomes_meio+str(' '+sbrn0.title())
                    # print('   NomesMeio:',nomes_meio)

                elif 1 < len(sbrn0) <= 3:
                    # print('CASO_05.3b: Avalia primeiro sobrenome 1< tamanho <=3')
                    # print('Vão pros nomes sbrn0:',sbrn0, 'e sbrn1:',sbrn1)

                    div_tresconsoantes = letras_tresconsnts.findall(sobrenome)
                    if div_tresconsoantes != []:
                        # print('CASO_05.4: Três consoantes como sobrenome')
                        for letra in sobrenome:
                            nomes.append(letra)

                        if len(sobrenome) >2:
                            sobrenome=nomes[0]
                        else:
                            sobrenome=nomes[1]
                        nomes.remove(sobrenome)
                        primeiro_nome=nomes[0]
                        nomes_meio=' '.join([str for str in nomes[1:]]).strip()
                        nome_completo=sobrenome.upper()+', '+nomes_meio                
                    
                    try:                       
                        # print(' 05.3b    Lista de Nomes:',nomes_meio)
                        nomes_meio=nomes_meio.replace(sbrn0,'')
                        # print(' 05.3b ReplaceSobrenome0:',nomes_meio)
                        nomes_meio=nomes_meio.replace(sbrn1,'')
                        # print(' 05.3b ReplaceSobrenome1:',nomes_meio)
                    except Exception as e:
                        # print('   Erro ReplaceSobrenome:',e)
                        pass
                    try:
                        nomes_meio.replace(primeiro_nome.title(),'')
                        nomes_meio.replace(primeiro_nome.lower(),'')
                        nomes_meio.replace(primeiro_nome,'')
                        # print(' 05.3b Replace PrimNome:',nomes_meio)
                    except Exception as e:
                        print('Erro no try PrimeiroNome:',e)
                        pass
                    nomes_meio = nomes_meio.replace(sobrenome,'')
                    try:
                        for n,i in enumerate(avaliar1):
                            nomes.append(i.upper())
                            sbrn1     = avaliar1[0]
                        else:
                            sbrn1     = avaliar1.title()
                        # print('sbrn1:',sbrn1, len(sbrn1))
                        nomes_meio = nomes_meio+str(' '+sbrn0)+str(' '+sbrn1)
                    except:
                        nomes_meio = nomes_meio+str(' '+sbrn0)
                    nomes      = nomes_meio.strip().strip(',').split(' ')
                    # print(' 05.3b NomesMeio:',nomes_meio)
                    # print(' 05.3b     Nomes:',nome)

                else:
                    # print('CASO_05.3c: Avalia primeiro sobrenome >3')
                    nomes_meio = nomes_meio+str(' '+div[0].strip().title())
                    nomes      = nomes_meio.strip().split(' ')
                    # print(' 05.3c NomesMeio:',nomes_meio)
                    # print(' 05.3c     Nomes:',nomes)

                nomes_meio=nomes_meio.replace(sobrenome,'').replace(',','').strip()
                nomes_meio=nomes_meio.replace(primeiro_nome,'').strip()

            # print('Ao final do caso 05')
            # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
            # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
            # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
    
    if sobrenome != '' and primeiro_nome !='':
        nome_completo=sobrenome.upper().replace(',','')+', '+primeiro_nome.replace(',','')+' '+nomes_meio.replace(sobrenome,'').replace(',','')
    elif sobrenome != '':
        nome_completo=sobrenome.upper().replace(',','')+', '+nomes_meio.replace(sobrenome,'').replace(',','')
    else:
        nome_completo=sobrenome.upper()
    
#     print('Após ajustes finais')
#     print('     Sobrenome:',sobrenome)
#     print(' Primeiro Nome:',primeiro_nome)
#     print('         Nomes:',nomes)
#     print('     NomesMeio:',nomes_meio)        
        
#     print('                Resultado:',nome_completo)
    
    return nome_completo.strip()



def padronizar_titulo(titulo_bruto):
    '''
    Retira acentos, expressão (Org.) e espaços vazios do título da publicação
    Autor: Marcos Aires (Fev.2022)
    '''
    import re
    import unicodedata
    
    ## Retirar caracteres não unicode
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', titulo_bruto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.',' ').replace('-',' ').replace('\'',' ').lower()
    
    substitui_iniciais=[('rl,', 'r l,'), ('hs,', 'h s,'), ('gc,', 'g c,'), ('sf,', 's f,'), ('fo,', 'f o,'), (' oa,', ' o a,'), ('mss,', 'm s s,'), 
                        ('lagares, ma', 'lagares, m a'), ('cota, gf', 'cota, g f'), ('cota gf,', 'cota, g f,'), ('diotaiut,', 'diotaiuti,'), ('grenfell, r f q','queiroz, rafaella fortini grenfell')]
    for i in substitui_iniciais:
        string = string.replace(i[0], i[1])
    
    ## Retirar preposições
    preposicoes = ['da', 'de', 'do', 'das', 'dos']
    partes_string = string.split(' ')
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)
    
    titulo_padronizado = string.strip().strip('"')
    
    return titulo_padronizado



def iniciais_nome(linha_texto):
    '''
    Função para retornar sobrenome+iniciais das partes de nome, na forma: SOBRENOME, X Y Z
     Recebe: String com nome
    Retorna: Tupla com nome e sua versão padronizada em sobrenome+agnome em maiúsculas, seguida de vírgula e das iniciais das demais partes de nome
      Autor: Marcos Aires (Mar.2022)
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    
    ## Retirar caracteres não unicode
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', linha_texto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.','')
    
    ## Retirar preposições
    preposicoes   = ['da','de','do','das','dos']
    partes_string = string.split(' ')
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)
        
    ## Expressões regulares para encontrar padrões de divisão de nomes de autores
    sobrenome_inicio   = re.compile(r'^[A-ZÀ-ú-a-z]+,')                 # Sequência de letras maiúsculas no início da string
    sobrenome_composto = re.compile(r'^[A-ZÀ-ú-a-z]+[ ][A-ZÀ-ú-a-z]+,') # Duas sequências de letras no início da string, separadas por espaço, seguidas por vírgula
    letra_abrevponto   = re.compile(r'^[A-Z][.]')                       # Uma letra maiúscula no início da string, seguida por ponto
    letra_abrevespaco  = re.compile(r'^[A-Z][ ]')                       # Uma letra maiúscula no início da string, seguida por espaço
    letras_dobradas    = re.compile(r'[A-Z]{2}')                        # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasini = re.compile(r'[A-Z]{2}[ ]')                     # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasfim = re.compile(r'[ ][A-Z]{2}')                     # Duas letras maiúsculas juntas no final da string, precedida por espaço
        
    nomes=[]
    agnomes       = ['NETO','JUNIOR','FILHO','SEGUNDO','TERCEIRO', 'SOBRINHO']
    nome_completo = ''
    
    ## Ajustar lista de termos, identificar sobrenomes compostos e ajustar sobrenome com ou sem presença de vírgula
    div_sobrenome      = sobrenome_inicio.findall(string)
    div_sbrcomposto    = sobrenome_composto.findall(string)
    
    ## Caso haja vírgulas na string, tratar sobrenomes e sobrenomes compostos
    if div_sobrenome != [] or div_sbrcomposto != []:
        div   = string.split(', ')
        sobrenome     = div[0].strip().upper()
        try:
            div_espaco    = div[1].split(' ')
        except:
            div_espaco  = ['']
        primeiro      = div_espaco[0].strip('.')
        
        ## Caso primeiro nome sejam somente duas letras maiúsculas juntas, trata-se de duas iniciais
        if len(primeiro)==2:
            primeiro_nome=primeiro[0].strip()
            nomes.append(primeiro[1].strip())
        else:
            primeiro_nome = div_espaco[0].strip().title()
        
        ## Montagem da lista de nomes do meio
        for nome in div_espaco:
            if nome not in nomes and nome.lower()!=primeiro_nome.lower() and nome.lower() not in primeiro_nome.lower() and nome!=sobrenome:   
                # print(nome, len(nome))
                
                ## Avaliar se é abreviatura seguida de ponto e remover o ponto
                if len(nome)<=2 and nome.lower() not in preposicoes:
                    for inicial in nome:
                        # print(inicial)
                        if inicial not in nomes and inicial not in primeiro_nome:
                            nomes.append(inicial.replace('.','').strip().title())
                else:
                    if nome not in nomes and nome!=primeiro_nome and nome!=sobrenome and nome!='':
                        if nome.lower() in preposicoes:
                            nomes.append(nome.replace('.','').strip().lower())
                        else:
                            nomes.append(nome.replace('.','').strip().title())
                        # print(nome,'|',primeiro_nome)
                        
        ## Caso haja sobrenome composto que não esteja nos agnomes considerar somente primeira parte como sobrenome
        if div_sbrcomposto !=[] and sobrenome.split(' ')[1] not in agnomes:
            # print(div_sbrcomposto)
            # print('Sobrenome composto:',sobrenome)
            nomes.append(sobrenome.split(' ')[1].title())
            sobrenome = sobrenome.split(' ')[0].upper()
            # print('Sobrenome:',sobrenome.split(' '))
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
        
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
        
    ## Caso não haja vírgulas na string considera sobrenome o último nome da string dividida com espaço vazio
    else:
        try:
            div       = string.split(' ')
            if div[-2] in agnomes:
                sobrenome = div[-2].upper()+' '+div[-1].strip().upper()
                for i in nomes[1:-2]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
            else:
                sobrenome = div[-1].strip().upper()
                for i in div[1:-1]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
        except:
            sobrenome = div[-1].strip().upper()
            for i in div[1:-1]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
            
        if sobrenome.lower() != div[0].strip().lower():
            primeiro_nome=div[0].strip().title()
        else:
            primeiro_nome=''
        
        # print('    Sobrenome sem vírgula:',sobrenome)
        # print('Primeiro nome sem vírgula:',primeiro_nome)
        # print('Nomes do meio sem vírgula:',nomes)
    
    # Encontrar e tratar como abreviaturas termos com apenas uma ou duas letras iniciais juntas, com ou sem ponto
    for j in nomes:
        # Procura padrões com expressões regulares na string
        div_sobrenome      = sobrenome_inicio.findall(j)
        div_sbrcomposto    = sobrenome_composto.findall(j)
        div_abrevponto     = letra_abrevponto.findall(j)
        div_abrevespaco    = letra_abrevespaco.findall(j)
        div_ltrdobradasini = letras_dobradasini.findall(j)
        div_ltrdobradasfim = letras_dobradasfim.findall(j)
        div_ltrdobradas    = letras_dobradas.findall(j)
        tamanho=len(j)
        # print('\n', div_ltrdobradasini, div_ltrdobradasfim, tamanho, 'em:',j,len(j))
        
        #caso houver abreviatura com uma letra em maiúscula nos nomes
        if div_abrevponto !=[] or tamanho==1:
            cada_nome = j.replace('.','').strip()
            if cada_nome not in nomes and cada_nome != sobrenome and nome != primeiro_nome:
                nomes.append(cada_nome)
        
        #caso houver duas inicias juntas em maiúsculas
        elif div_ltrdobradasini !=[] or div_ltrdobradasfim !=[] or div_ltrdobradas !=[] :
            for letra in j:
                if letra not in nomes and letra != sobrenome and letra != primeiro_nome:
                    nomes.append(letra)
        
        #caso haja agnomes ao sobrenome
        elif sobrenome in agnomes:
            sobrenome = nomes[-1].upper()+' '+sobrenome
            # print(sobrenome.split(' '))
            # print('Sobrenome composto:',sobrenome)
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
            
        else:
            if j not in nomes and j not in sobrenome and j != primeiro_nome:
                nomes.append(j)
    
    nomes_meio=' '.join([str[0] for str in nomes]).strip()
    # print('Qte nomes do meio',len(nomes),nomes)
    if sobrenome != '' and primeiro_nome !='':
        sobrenome_iniciais = sobrenome+', '+primeiro_nome[0]+' '+nomes_meio
    elif sobrenome != '':
        sobrenome_iniciais = sobrenome
    
    return sobrenome_iniciais.strip()


## Agregar aprendizado supervisionado humano à medida que forem sendo identificados erros na situação atual
lista_extra = [
                # ('ALBUQUERQUE, Adriano B', 'ALBUQUERQUE, Adriano Bessa'),
                # ('ALBUQUERQUE, Adriano', 'ALBUQUERQUE, Adriano Bessa'),
                # ('COELHO, Andre L V', 'COELHO, Andre Luis Vasconcelos'),
                # ('DUARTE, Joao B F', 'DUARTE, Joao Batista Furlan'),
                # ('FILHO, Raimir H','HOLANDA FILHO, Raimir'),
                # ('FILHO, Raimir','HOLANDA FILHO, Raimir'),
                # ('FORMIGO, A','FORMICO, Maria Andreia Rodrigues'),
                # ('FORMICO, A','FORMICO, Maria Andreia Rodrigues'),
                # ('FURLAN, J B D', 'FURLAN, Joao Batista Duarte'),
                # ('FURTADO, Elizabeth', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Elizabeth S', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Elizabeth Sucupira','FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, M E S', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Vasco', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, J P', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, J V P', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, Vasco', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, Elizabeth','FURTADO, Maria Elizabeth Sucupira'),
                # ('HOLANDA, Raimir', 'HOLANDA FILHO, Raimir'),
                # ('LEITE, G S', 'LEITE, Gleidson Sobreira'),
                # ('PEQUENO, T H C', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio','PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio Cavalcante', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PINHEIRO, Placido R', 'PINHEIRO, Placido Rogerio'),
                # ('PINHEIRO, Vladia', 'PINHEIRO, Vladia Celia Monteiro'),
                # ('RODRIGUES, M A F', 'RODRIGUES, Maria Andreia Formico'),
                # ('RODRIGUES, Andreia', 'RODRIGUES, Maria Andreia Formico'),
                # ('JOAO, Batista F Duarte,', 'FURLAN, Joao Batista Duarte'),
                # ('MACEDO, Antonio Roberto M de', 'MACEDO, Antonio Roberto Menescal de'),
                # ('MACEDO, D V', 'MACEDO, Daniel Valente'),
                # ('MENDONCA, Nabor C', 'MENDONCA, Nabor das Chagas'),
                # ('PEQUENO, Tarcisio', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio H', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PINHEIRO, Mirian C D', 'PINHEIRO, Miriam Caliope Dantas'),
                # ('PINHEIRO, Mirian Caliope Dantas', 'PINHEIRO, Miriam Caliope Dantas'),
                # ('PINHEIRO, P G C D', 'PINHEIRO, Pedro Gabriel Caliope Dantas'),
                # ('PINHEIRO, Pedro G C', 'PINHEIRO, Pedro Gabriel Caliope Dantas'),
                # ('PINHEIRO, Placido R', 'PINHEIRO, Placido Rogerio'),
                # ('PINHEIRO, Vladia', 'PINHEIRO, Vladia Celia Monteiro'),
                # ('ROGERIO, Placido Pinheiro', 'PINHEIRO, Placido Rogerio'),
                # ('REBOUCRAS FILHO, Pedro', 'REBOUCAS FILHO, Pedro Pedrosa'),
                # ('SAMPAIO, A', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SAMPAIO, Americo', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SAMPAIO, Americo Falcone', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SUCUPIRA, Elizabeth Furtado','FURTADO, Maria Elizabeth Sucupira'),
                ]


def converter_lista_set(lista):
    set1 = set(lista)
    return set1


def jaccard_similarity(set1, set2):
    '''
    Recebe dois conjuntos como entradas e retorna a similaridade Jaccard entre eles. 
    1. calcula a interseção dos dois conjuntos usando a função de interseção e, 
    2. calcula a união dos dois conjuntos usando a função de união. 
    3. retorna a razão entre o comprimento da interseção e o comprimento da união, que é a similaridade de Jaccard.
    '''
    intersection = set1.intersection(set2)
    union        = set1.union(set2)
    return len(intersection) / len(union)


def similares(lista_autores, lista_grupo, limite_jarowinkler, distancia_levenshtein, lista_extra):
    """Função para aplicar padronização no nome de autor da lista de pesquisadores e buscar similaridade na lista de coautores
     Recebe: Lista de pesquisadores do grupo em análise gerada pela lista de nomes dos coautores das publicações em análise
    Utiliza: get_jaro_distance(), editdistance()
    Retorna: Lista de autores com fusão de nomes cuja similaridade esteja dentro dos limites definidos nesta função
      Autor: Marcos Aires (Fev.2022)
      
    Refazer: Inserir crítica de, mantendo sequência ordem alfabética, retornar no final nome mais extenso em caso de similaridade;
    """
    from pyjarowinkler.distance import get_jaro_distance
    from IPython.display import clear_output
    import editdistance
    import numpy as np
    import time
    
    t0=time.time()
    
    # limite_jarowinkler=0.85
    # distancia_levenshtein=6
    similares_jwl=[]
    similares_regras=[]
    similares=[]
    tempos=[]
    
    count=0
    t1=time.time()
    for i in lista_autores:
        count+=1
        if count > 0:
            tp=time.time()-t1
            tmed=tp/count*2
            tempos.append(tp)
        # print("Analisar similaridades com: ", nome_padronizado)
        
        count1=0
        for nome in lista_autores:
            if count1 > 0:
                resta=len(lista_autores)-count
                print(f'Analisando {count1:3}/{len(lista_autores)} resta analisar {resta:3} nomes. Previsão de término em {np.round(tmed*resta/60,1)} minutos')
            else:
                print(f'Analisando {count1:3}/{len(lista_autores)} resta analisar {len(lista_autores)-count1} nomes.')
            
            t2=time.time()
            count1+=1            

            try:
                similaridade_jarowinkler = get_jaro_distance(i, nome)
                print(f'{i:40} | {nome:40} | Jaro-Winkler: {np.round(similaridade_jarowinkler,2):4} Levenshtein: {editdistance.eval(i, nome)}')
                similaridade_levenshtein = editdistance.eval(i, nome)

                # inferir similaridade para nomes que estejam acima do limite ponderado definido, mas não idênticos e não muito distantes em edição
                if  similaridade_jarowinkler > limite_jarowinkler and similaridade_jarowinkler!=1 and similaridade_levenshtein < distancia_levenshtein:
                    # Crítica no nome mais extenso como destino no par (origem, destino)
                    
                    similares_jwl.append((i,nome))

            except:
                pass

            clear_output(wait=True)
    
    # Conjunto de regras de validação de similaridade
    # Monta uma lista de nomes a serem retirados antes de montar a lista de troca
    trocar=[]
    retirar=[]
    for i in similares_jwl:
        sobrenome_i = i[0].split(',')[0]
        sobrenome_j = i[1].split(',')[0]

        try:
            iniciais_i  = iniciais_nome(i[0]).split(',')[1].strip()
        except:
            iniciais_i  = ''

        try:
            iniciais_j  = iniciais_nome(i[1]).split(',')[1].strip()
        except:
            iniciais_j  = ''

        try:
            primnome_i = i[0].split(',')[1].strip().split(' ')[0].strip()
        except:
            primnome_i = ''

        try:
            primnome_j = i[1].split(',')[1].strip().split(' ')[0].strip()
        except:
            primnome_j = ''    

        try:
            inicial_i = i[0].split(',')[1].strip()[0]
        except:
            inicial_i = ''

        try:
            resto_i   = i[0].split(',')[1].strip().split(' ')[0][1:]
        except:
            resto_i   = ''

        try:
            inicial_j = i[1].split(',')[1].strip()[0]
        except:
            inicial_j = ''

        try:
            resto_j   = i[1].split(',')[1].strip().split(' ')[0][1:]
        except:
            resto_j = ''

        # Se a distância de edição entre os sobrenomes
        if editdistance.eval(sobrenome_i, sobrenome_j) > 2 or inicial_i!=inicial_j:
            retirar.append(i)
        else:
            if primnome_i!=primnome_j and len(primnome_i)>1:
                retirar.append(i)
            if primnome_i!=primnome_j and len(primnome_i)>1 and len(primnome_j)>1:
                retirar.append(i)
            if resto_i!=resto_j and resto_i!='':
                retirar.append(i)
            if len(i[1]) < len(i[0]):
                retirar.append(i)
            if len(iniciais_i) != len(iniciais_j):
                retirar.append(i)

    for i in similares_jwl:
        if i not in retirar:
            trocar.append(i)

        if iniciais_nome(i[0]) in iniciais_nome(i[1]) and len(i[0]) < len(i[1]):
            trocar.append(i)

        if iniciais_nome(i[0]) == iniciais_nome(i[1]) and len(i[0]) < len(i[1]):
             trocar.append(i)
    
    trocar=trocar+lista_extra
    trocar.sort()
    
    return trocar



def extrair_variantes(df_dadosgrupo):
    ''' Utiliza campo de Nome em Citações do currículo como filtro para obter variantes do nome de cada membro
     Recebe: Dataframe com os dados brutos do grupo de pesquisa agrupados; lista de nomes de pesquisadores de interesse
    Retorna: Lista de tuplas com pares a serem trocados da variante pelo nome padronizado na forma (origem, destino)
    '''
    filtro1   = 'Nome'
    lista_nomes = df_dadosgrupo[(df_dadosgrupo.ROTULOS == filtro1)]['CONTEUDOS'].values

    variantes=[]
    filtro='Nome em citações bibliográficas'
    variantes=df_dadosgrupo[(df_dadosgrupo.ROTULOS == filtro)]['CONTEUDOS'].to_list()

    trocar=[]
    for j in range(len(variantes)):
        padrao_destino = padronizar_nome(lista_nomes[j])
        trocar.append((lista_nomes[j], padrao_destino))
        for k in variantes[j]:
            padrao_origem = padronizar_nome(k)
            trocar.append((k, padrao_destino))
            trocar.append((padrao_origem, padrao_destino))
    
    return trocar

    Apurar colaborações docente/discente

In [ ]:
def converter_lista_set(lista):
    set1 = set(lista)
    return set1

def montardf_producao(lista_csv):
    print(f'{len(lista_csv):02} nomes a extrair')
    df_public=pd.DataFrame()
    for nome_csv in lista_csv:
        if 'colaboradores' in nome_csv.lower():
            tipo='colaboradores'
        else:
            tipo='permanentes'
        
        df_pub = pd.read_csv(pathcsv+nome_csv)
        print(len(df_pub.index))
        print(df_pub.keys())

        pat='\t\t\t\t\t\t\t\t\t\t\t\t \t\t\t\t\t\t\t\t\t\t\t\t\t'
        df_temp1 = df_pub.Data.str.split(pat=pat,expand=True)
        df_temp1.columns = (['TITULO','RevAut'])

        pat1='\t\t\t\t\t\t\t\t\t\t\t\t \t\t\t\t\t\t\t\t\t\t\t\t'
        df_temp2 = df_temp1.RevAut.str.split(pat=pat1,expand=True)
        df_temp2.columns = (['REVISTA','AUTORES'])

        df_temp0 = df_pub.drop(['Data'], axis=1)
        df_pub=df_temp0.merge(df_temp1['TITULO'],left_index=True,right_index=True)
        df_pub=df_pub.merge(df_temp2,left_index=True,right_index=True)
        try:
            df_pub.drop(['Issn','Natureza'], axis=1, inplace=True)
        except:
            pass
        try:
            df_pub.drop(['Tipo','Idioma'], axis=1, inplace=True)
        except:
            pass

        df_public = pd.concat([df_public, df_pub], ignore_index=True)
        print(len(df_public.index))
        print(df_public.keys())
        
    ## Extrai o período com base nos dados
    inicio = min(df_public['Ano'])
    final  = max(df_public['Ano'])
    
    total_artigos=len(df_public.index)
    # print(f'{total_artigos:4} publicações de artigos de docentes do programa no período de {inicio} a {final} carregadas...') 
    
    return df_public

def ler_nomesdocentes():    
    print(pathcsv)
    import os, sys

    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if 'nomes_docentes' in file:
            lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv


## ler arquivo com as orientações para gerar a lista de discentes
def ler_lista_orientacoes():
    try:
        l1='lista_orientadores-discentes.csv'
        df_orientacoes = pd.read_csv(pathcsv+l1, delimiter=';', header=None)
        
        lista_orientadores = df_orientacoes.iloc[:,0].unique()
        lista_discentes    = df_orientacoes.iloc[:,1].unique()
        print(f'{len(lista_orientadores):4} orientadores, com {len(lista_discentes)} discentes encontrados')
    except Exception as e:
        print('Erro ao gerar lista de orientações:')
        print(e)
        return df_orientacoes
        
    return lista_orientadores, lista_discentes 


## montar um dataframe com nome dos discentes de cada orientador
def montardf_orientacoes():
    try:
        l1='lista_orientadores-discentes.csv'
        df_orientacoes = pd.read_csv(pathcsv+l1, delimiter=';', header=None)
        df_orientacoes.columns=['ORIENTADOR','DISCENTE']
        
    except Exception as e:
        print('Erro ao dividir dataframe de orientações:')
        print(e)
        return
        
    return df_orientacoes



def montardf_docentes_permanentes_colaboradores():
    try:
        l1='lista_docentes_colaboradores.csv'
        l2='lista_docentes_permanentes.csv'
        df_docclbr = pd.read_csv(os.path.join(pathcsv,l1), header=None)
        df_docperm = pd.read_csv(os.path.join(pathcsv,l2), header=None)
        df_docentes = pd.concat([df_docperm, df_docclbr], ignore_index=True)
        print(f'{len(df_docentes.index):4} docentes permanentes e colaboradores encontrados')
    except Exception as e:
        print(e)
        
    return df_docentes


def jaccard_similarity(set1, set2):
    '''
    Recebe dois conjuntos (sets) como entradas e retorna a similaridade Jaccard entre eles e avalia: 
    1. calcula a interseção dos dois conjuntos usando a função de interseção 
    2. calcula a união dos dois conjuntos usando a função de união 
    3. retorna a razão entre o comprimento da interseção e o comprimento da união, que é a similaridade de Jaccard.
    '''
    intersection = set1.intersection(set2)
    union        = set1.union(set2)
    return len(intersection) / len(union)
    
def montardf_docentes(lista_nomes1=False, lista_nomes2=False):
    # print(lista_nomes1)
    # print(lista_nomes2)

    ## Montagem do dataframe de participação docente
    if (lista_nomes1 and lista_nomes2) != False:
        ## Criar dataframe com os nomes do grupo de docentes permanentes
        file_path = os.path.join(pathcsv,'lista_docentes_permanentes.csv')
        df_docentes_permanentes   = pd.read_csv(file_path, header=None)
        # df_docentes_permanentes['GRUPO']='Permanente'
        # try:
        #     df_docentes_permanentes.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        # df_docentes_permanentes.columns = ['DOCENTE','GRUPO']
        df_docentes_permanentes.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']

        ## Criar dataframe com os nomes do grupo de docentes colaboradores
        file_path = os.path.join(pathcsv,'lista_docentes_colaboradores.csv')
        df_docentes_colaboradores = pd.read_csv(file_path, header=None)
        # df_docentes_colaboradores['GRUPO']='Colaborador'
        # try:
        #     df_docentes_colaboradores.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        df_docentes_colaboradores.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']

        ## Criar um dataframe único com todos grupos de docentes juntos
        df_docentes = pd.concat([df_docentes_permanentes, df_docentes_colaboradores]).reset_index(drop=True)
        return df_docentes

    elif 'permanentes' in lista_nomes1.lower():
        ## Criar dataframe com os nomes do grupo de docentes permanentes
        file_path = os.path.join(pathcsv,lista_nomes1)       
        df_docentes_permanentes   = pd.read_csv(file_path, header=None)
        # df_docentes_permanentes['GRUPO']='Permanente'
        # try:
        #     df_docentes_permanentes.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        df_docentes_permanentes.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']
        return df_docentes_permanentes

    elif 'colaboradores' in lista_nomes1.lower():
        ## Criar dataframe com os nomes do grupo de docentes colaboradores
        file_path = os.path.join(pathcsv,lista_nomes1)
        df_docentes_colaboradores = pd.read_csv(file_path, header=None)
        # df_docentes_colaboradores['GRUPO']='Colaborador'
        # try:
        #     df_docentes_colaboradores.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        df_docentes_colaboradores.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']
        return df_docentes_colaboradores
    else:
        print('Erro ao montar dataframe de docentes, verifique os nomes de arquivo.')
        return
        
def montar_listas(lista_csv, csv_permanentes=None, csv_colaboradores=None):
    if csv_permanentes == None and csv_colaboradores == None:
        csv_permanentes   = 'lista_docentes_permanentes.csv'
        csv_colaboradores = 'lista_docentes_colaboradores.csv'
        print(f'\nNomes de docentes não informados, utilizando caminho e nomes padrão:')
        print(f'     Docentes   permanentes de {pathcsv}{csv_permanentes}')
        print(f'     Docentes colaboradores de {pathcsv}{csv_colaboradores}')
        try:
            file_path = os.path.join(pathcsv,csv_permanentes)
            df_docperm = pd.read_csv(file_path, header=None)
            file_path = os.path.join(pathcsv,csv_colaboradores)
            df_docclbr = pd.read_csv(file_path, header=None)
            lista_docentes = pd.concat([df_docperm, df_docclbr], ignore_index=True)[0].values
            print(f'{len(lista_docentes):4} docentes permanentes e colaboradores encontrados')
        except Exception as e:
            print(f'Erro ao ler listas de docentes, verificar se os arquivos CSV estão na pasta {pathcsv}')
            print(e)
    elif 'permanentes' in csv_permanentes.lower():
        print(f'\nArquivo docentes   permanentes informado: {csv_permanentes}')
        try:
            file_path = os.path.join(pathcsv,csv_permanentes)
            lista_docentes = pd.read_csv(file_path, header=None)[0].values
            print(f'{len(lista_docentes)} docentes permanentes encontrados')
        except Exception as e:
            print(e)
    elif 'colaboradores' in csv_permanentes.lower():
        print(f'\nArquivo docentes colaboradores informado: {csv_colaboradores}')
        try:
            file_path = os.path.join(pathcsv,csv_permanentes)
            lista_docentes = pd.read_csv(file_path, header=None)[0].values
            print(f'{len(lista_docentes)} docentes colaboradores encontrados')
        except Exception as e:
            print(e)    
    else:
        print('Erro ao ler listas de docentes, verificar listas')

    df_prod   = montardf_producao(lista_csv)
    
    ## Montar a lista de autores com limpar_nomes remove caracteres, preposições e separa iniciais com espaço
    lista_listas = df_prod['AUTORES'].tolist()
    lista_autores_artigos = []
    for i in lista_listas:
        lista_autores_artigos.append(limpar_nomes(i))
    
    ## Ler nomes de discentes e orientadores
    lista_orientadores, lista_discentes = ler_lista_orientacoes()
    
    return lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes

def montardf_participacao_docente_discente(df_prod, dic_nomes_docentes, dic_nomes_discentes):
    ## Montar dataframe de participação docente
    df_participacao_docente = pd.DataFrame(dic_nomes_docentes).T
    df_participacao_docente.columns = ['DOCENTE','INDICES_ARTIGOS']
    df_participacao_docente['AUTORIAS'] = [len(x) for x in df_participacao_docente['INDICES_ARTIGOS']]

    ## Montar dataframe de participação discente
    df_participacao_discente = pd.DataFrame(dic_nomes_discentes).T
    df_participacao_discente.columns = ['DISCENTE','INDICES_ARTIGOS']
    df_participacao_discente['AUTORIAS'] = [len(x) for x in df_participacao_discente['INDICES_ARTIGOS']]

    ## Criar lista com os artigos onde foi encontado nome de algum docente
    artigos_com_docentes=[]
    for m in df_participacao_docente['INDICES_ARTIGOS']:
        for n in m:
            if n not in artigos_com_docentes:
                artigos_com_docentes.append(n)
    artigos_com_docentes.sort()
                
    ## Criar lista com os artigos onde foi encontado nome de algum discente
    artigos_com_discentes=[]
    for m in df_participacao_discente['INDICES_ARTIGOS']:
        for n in m:
            if n not in artigos_com_discentes:
                artigos_com_discentes.append(n)
    artigos_com_discentes.sort()
                
        
    ## Criar lista com os artigos onde NÃO foi encontado nome de discente
    lista_semparticipacaodiscente=[]
    for i in range(len(df_prod.index)):
        if i not in artigos_com_discentes and i not in lista_semparticipacaodiscente:
            lista_semparticipacaodiscente.append(i)
            
    ## Criar lista com os artigos onde NÃO foi encontado nome de docente
    lista_semparticipacaodocente=[]
    for i in range(len(df_prod.index)):
        if i not in artigos_com_docentes and i not in lista_semparticipacaodocente:
            lista_semparticipacaodocente.append(i)

    ## Apresentar resultados das buscas por nomes de autores docentes e discentes
    lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
    print(len(lista_titulos),'títulos únicos de artigo encontrados')
    pdoc = np.round(100*len(artigos_com_docentes)/len(df_prod.index),2)
    spdoc = np.round(100*len(lista_semparticipacaodocente)/len(df_prod.index),2)
    print(f'{len(artigos_com_docentes):4} ({pdoc:6}%) artigos com nome de  docente encontrado, faltando {len(lista_semparticipacaodocente):3} ({spdoc:5}%)')

    pdis = np.round(100*len(artigos_com_discentes)/len(df_prod.index),2)
    spdis = np.round(100*len(lista_semparticipacaodiscente)/len(df_prod.index),2)
    print(f'{len(artigos_com_discentes):4} ({pdis:6}%) artigos com nome de discente encontrado, faltando {len(lista_semparticipacaodiscente):3} ({spdis:5}%)')

    return df_participacao_docente, df_participacao_discente, lista_semparticipacaodocente, lista_semparticipacaodiscente

    Ler arquivos de dados do disco local

In [ ]:
## ler arquivos de publicação de artigos na pasta de arquivos CSV
def ler_artigostodosperiodos():    
    print(pathcsv)
    import os, sys

    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if 'Artigos' in file:
            lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv



## ler arquivos de publicação de de período determinado
def ler_csvptg(inicio=False, final=False, tipo=False, grupo=False):    
    import os, sys

    print(pathcsv)
    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if (str(inicio) or str(final)) == False:
            if tipo.lower() == False:
                if grupo.lower() == False:
                    lista_csv.append(file)

        elif (str(inicio) and str(final)) in file.lower():
            if unidecode(tipo).lower() in unidecode(file).lower():
                if unidecode(grupo).lower() in unidecode(file).lower():
                    lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv

### Funções para avaliar a PCD e Pontuação de Impacto

In [ ]:
def montardf_impacto_docente_discente(df_prod, dic_nomes_docentes, dic_nomes_discentes, inicio, final):
    artigos_com_discentes=[]
    artigos_com_docentes=[]
    lista_semparticipacaodiscente=[]
    lista_semparticipacaodocente=[]

    try:
        ## Montar dataframe de participação docente
        df_impacto_docente = pd.DataFrame(dic_nomes_docentes).T
        df_impacto_docente.columns = ['DOCENTE','INDICES_ARTIGOS','EXTRATOS_QUALIS']
        df_impacto_docente['AUTORIAS'] = [len(x) for x in df_impacto_docente['INDICES_ARTIGOS']]

        ## Criar lista com os artigos onde foi encontado nome de algum docente
        for m in df_impacto_docente['INDICES_ARTIGOS']:
            for n in m:
                if n not in artigos_com_docentes:
                    artigos_com_docentes.append(n)
        artigos_com_docentes.sort()

        ## Criar lista com os artigos onde NÃO foi encontado nome de docente
        for i in range(len(df_prod.index)):
            if i not in artigos_com_docentes and i not in lista_semparticipacaodocente:
                lista_semparticipacaodocente.append(i)
    except:
        df_impacto_docente = pd.DataFrame()
        print('Não foi possível encontrar nenhuma ocorrência dos nomes dos docentes com este conjunto de dados')
        pass

    try:
        ## Montar dataframe de participação discente
        df_impacto_discente = pd.DataFrame(dic_nomes_discentes).T
        df_impacto_discente.columns = ['DISCENTE','INDICES_ARTIGOS','EXTRATOS_QUALIS']
        df_impacto_discente['AUTORIAS'] = [len(x) for x in df_impacto_discente['INDICES_ARTIGOS']]

        ## Criar lista com os artigos onde foi encontado nome de algum discente
        for m in df_impacto_discente['INDICES_ARTIGOS']:
            for n in m:
                if n not in artigos_com_discentes:
                    artigos_com_discentes.append(n)
        artigos_com_discentes.sort()
    except:
        df_impacto_discente = pd.DataFrame()
        print('Não foi possível encontrar nenhuma ocorrência dos nomes dos discentes com este conjunto de dados')
        pass
                
    ## Criar lista com os artigos onde NÃO foi encontado nome de discente
    for i in range(len(df_prod.index)):
        if i not in artigos_com_discentes and i not in lista_semparticipacaodiscente:
            lista_semparticipacaodiscente.append(i)

    ## Apresentar resultados das buscas por nomes de autores docentes e discentes
    lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
    print(len(lista_titulos),'títulos únicos de artigo encontrados')
    pdoc  = np.round(100*len(artigos_com_docentes)/len(df_prod.index),2)
    spdoc = np.round(100*len(lista_semparticipacaodocente)/len(df_prod.index),2)
    print(f'{len(artigos_com_docentes):4} ({pdoc:6}%) artigos com nome de  docente encontrado, faltando {len(lista_semparticipacaodocente):3} ({spdoc:5}%)')

    pdis = np.round(100*len(artigos_com_discentes)/len(df_prod.index),2)
    spdis = np.round(100*len(lista_semparticipacaodiscente)/len(df_prod.index),2)
    print(f'{len(artigos_com_discentes):4} ({pdis:6}%) artigos com nome de discente encontrado, faltando {len(lista_semparticipacaodiscente):3} ({spdis:5}%)')

    ## A1=100, A2=80, B1=60, B2=40, B3=20, B4=10, B5=2
    soma_impacto=[]
    for linha in df_impacto_docente['EXTRATOS_QUALIS']:
        impacto=0
        for extrato in linha:
            if extrato == 'A1':
                impacto+=100
            elif extrato == 'A2':
                impacto+=80
            elif extrato == 'B1':
                impacto+=60
            elif extrato == 'B2':
                impacto+=40
            elif extrato == 'B3':
                impacto+=20
            elif extrato == 'B4':
                impacto+=10
            elif extrato == 'B5':
                impacto+=2
            elif extrato == 'C':
                impacto+=0
            elif extrato == 'nan':
                impacto+=0
        soma_impacto.append(impacto)

    df_impacto_docente['SOMA_IMPACTO'] = soma_impacto
    qte_anos = (final-inicio+1)
    df_impacto_docente['ANOS'] = qte_anos
    df_impacto_docente['IMPACTO_MEDIO_ANUAL'] = np.round(df_impacto_docente['SOMA_IMPACTO']/df_impacto_docente['ANOS'],1)

    return df_impacto_docente, df_impacto_discente



def apurar_pcd_impacto(df_docentes, df_impacto_docente, df_impacto_discente, meta_pcd=50.0, meta_impacto=150.0):
    print(f'Total de nomes de docentes em análise: {len(df_docentes.index)}')
    print(f'Total de nomes de docentes  encontrados nos artigos: {len(df_impacto_docente.index)}')
    print(f'Total de nomes de discentes encontrados nos artigos: {len(df_impacto_discente.index)}')

    df_docentes_pcd_impacto = df_docentes
    ## Montar lista com os índices do dataframe de artigos onde foram achados nomes de discentes na lista de autores
    lista_participacao_discente = []
    for artigos_discentes in df_impacto_discente['INDICES_ARTIGOS']:
        for indice in artigos_discentes:
            lista_participacao_discente.append(indice)

    ## Contar a quantidade de participações de discentes que ocorrem no dataframe de produção docente
    qte_colab_discente=[]
    for docente,artigos_docente in zip(df_impacto_docente['DOCENTE'], df_impacto_docente['INDICES_ARTIGOS']):
        qte=0
        for indice in artigos_docente:
            if indice in lista_participacao_discente:
                qte+=1
            
        qte_colab_discente.append(qte)

    df_docentes_pcd_impacto['PUBLICAÇÕES']  = df_impacto_docente['AUTORIAS']
    df_docentes_pcd_impacto['COM_DISCENTE'] = qte_colab_discente
    df_docentes_pcd_impacto['PCD'] = np.round(100*(df_docentes_pcd_impacto['COM_DISCENTE']/df_impacto_docente['AUTORIAS']),1)
    df_docentes_pcd_impacto['IMPACTO'] = df_impacto_docente['SOMA_IMPACTO']
    df_docentes_pcd_impacto['IMPACTO_MEDIO_ANUAL'] = df_impacto_docente['IMPACTO_MEDIO_ANUAL']

    ## Definir a meta de produção conjunta com discente e apurar o resultado
    # meta_pcd=50.0
    apuracao_pcd     = df_docentes_pcd_impacto.groupby([df_docentes_pcd_impacto.index,'PCD'])
    df_abaixo_pcd    = apuracao_pcd.filter(lambda x: x['PCD'] < meta_pcd)
    df_atingiram_pcd = apuracao_pcd.filter(lambda x: x['PCD'] >= meta_pcd)

    total_docentes         = len(df_docentes_pcd_impacto.index)

    contagem_abaixometa    = len(df_abaixo_pcd.index)
    contagem_atingindometa = len(df_atingiram_pcd.index)
    indicador_pcd = np.round(100*contagem_atingindometa/total_docentes,1)
    print(f'{indicador_pcd}% dos docentes {contagem_atingindometa}/{total_docentes} atingem a meta de {meta_pcd}% publicação com discente')

    ## Definir a meta de impacto por pesquisador e para o grupo
    # meta_impacto=150
    apuracao_impacto     = df_docentes_pcd_impacto.groupby([df_docentes_pcd_impacto.index,'IMPACTO'])
    df_abaixo_impacto    = apuracao_impacto.filter(lambda x: x['IMPACTO_MEDIO_ANUAL'] < meta_impacto)
    df_atingiram_impacto = apuracao_impacto.filter(lambda x: x['IMPACTO_MEDIO_ANUAL'] >= meta_impacto)

    contagem_abaixometa_impacto    = len(df_abaixo_impacto.index)
    contagem_atingindometa_impacto = len(df_atingiram_impacto.index)
    indicador_impacto = np.round(100*contagem_atingindometa_impacto/total_docentes,1)
    print(f'{indicador_impacto}% dos docentes {contagem_atingindometa_impacto}/{total_docentes} atingem a meta de {meta_impacto} pontos de impacto médio por ano das publicações')

    return df_docentes_pcd_impacto, apuracao_pcd, df_abaixo_pcd, df_atingiram_pcd, indicador_pcd, indicador_impacto

### Funções para plotagem: 
    
    Plotar gráficos de percentual de participação discente

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.patches import Circle
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
from matplotlib.cbook import get_sample_data
plt.rcParams['font.size']      = 12
# plt.rcParams["figure.figsize"] = (15,9)



def plotar_pcd(df, grupo=False, inicio=False, final=False):
    N    = len(df.index)
    percentual = (df['PCD'].values.round(1))

    ind   = np.arange(N) # the x locations for the groups
    width = 0.75         # the width of the bars: can also be len(x) sequence

    # criar figura
    fig, ax = plt.subplots(figsize=(25,10))

    # plotar barras verticais com cores condicionais se abaixo do valor da variável para aceitação
    par = 50 
    cor = ['yellow' if (x < par) else 'green' for x in percentual]
    p1  = ax.bar(ind, percentual, width, 
                #  yerr=dsvpad, 
                 error_kw=dict(lw=0.3, capsize=2, capthick=1),
                 label='Percentual de publicação com discente', color=cor)

    # plotar os rótulos e título
    ax.axhline(par, color='red', linewidth=3, linestyle='dotted')
    ax.axhline(70, color='gray', linewidth=3, linestyle='dotted')
    if (grupo and inicio and final) != False:
        ax.set_title(f'Apuração do percentual de publicação de docentes {grupo.upper()} com discente no período de {inicio} a {final}')
    elif grupo == False:
        ax.set_title(f'Apuração do percentual de publicação de docentes com discente no período de {inicio} a {final}')
    else:
        ax.set_title(f'Apuração do percentual de publicação com discente')

    ax.set_ylabel('Percentual de artigos publicados com discente')
    ax.set_xticks(ind)
    
    labels_pos=np.arange(1,N+1)
    ax.set_xticklabels(labels_pos)

    # Label with label_type 'center' instead of the default 'edge'
    ax.bar_label(p1, label_type='center')
    # ax.bar_label(p1, dsvpad)
    # ax.set_yticks(range(0,100))
    
    # respectivo domínio de cada questão no AGREE II
    grupos = ['01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01',
              '02','02','02','02','02','02','02','02','02','02','02','02','02','02','02','02',]
    
    # calcular limites de retângulos dos domínios
    lista_dominios   = pd.Series(grupos)
    largura_dominios = lista_dominios.value_counts().sort_index().values   
    altura = 100
    rets=[]   
    
    for i in range(1,len(largura_dominios)+1):
        ret = patches.Rectangle((-0.5,0),
                                np.sum(largura_dominios[:i]), altura,
#                                 linestyle='dashdot',
                                linewidth=2,
                                edgecolor='b',
                                fill = False)
        rets.append(ret)
    
    # plotar os retângulos das dimensões na área do gráfico
    for i in rets:
        ax.add_patch(i)
        
    # plotar legenda, comentar para excluir
    # ax.legend(bbox_to_anchor=(0.75,-0.05), ncol=2)
    
    campo='04'
    # savefig_respostas(campo)
    
    plt.show()
    return 



def plotar_medias_impacto(df, grupo=False, inicio=False, final=False):
    N      = len(df.index)
    pontos = (df['IMPACTO_MEDIO_ANUAL'].values.round(1))

    ind   = np.arange(N) # the x locations for the groups
    width = 0.75         # the width of the bars: can also be len(x) sequence

    # criar figura
    fig, ax = plt.subplots(figsize=(25,10))

    # plotar barras verticais com cores condicionais se abaixo do valor da variável para aceitação
    par = 150 
    cor = ['yellow' if (x < par) else 'green' for x in pontos]
    p1  = ax.bar(ind, pontos, width, 
                #  yerr=dsvpad, 
                 error_kw=dict(lw=0.3, capsize=2, capthick=1),
                 label='Pontuação em impacto das publicações de docentes', color=cor)

    # plotar os rótulos e título
    ax.axhline(par, color='red', linewidth=3, linestyle='dotted')
    if (grupo and inicio and final) != False:
        ax.set_title(f'Apuração da pontuação de impacto das publicações de docentes {grupo.upper()} no período de {inicio} a {final}')
    elif (inicio and final) != False:
        ax.set_title(f'Apuração da pontuação de impacto das publicações de docentes no período de {inicio} a {final}')
    else:
        ax.set_title(f'Apuração da pontuação de impacto médio (total do impacto acumulado / quantidade de anos do período) das publicações de docentes')

    ax.set_ylabel('Pontuação ponderada pelo Qualis dos artigos publicados')
    ax.set_xticks(ind)
    
    labels_pos=np.arange(1,N+1)
    ax.set_xticklabels(labels_pos)

    # Label with label_type 'center' instead of the default 'edge'
    ax.bar_label(p1, label_type='center')
    # ax.bar_label(p1, dsvpad)
    vr_maximo = int(max(pontos)+50)
    # ax.set_yticks(range(0,vr_maximo))
    
    # respectivo domínio de cada questão no AGREE II
    grupos = ['01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01',
              '02','02','02','02','02','02','02','02','02','02','02','02','02','02','02','02',]
    
    # calcular limites de retângulos dos domínios
    lista_dominios   = pd.Series(grupos)
    largura_dominios = lista_dominios.value_counts().sort_index().values   
    altura = max(pontos)+50
    rets=[]   
    
    for i in range(1,len(largura_dominios)+1):
        ret = patches.Rectangle((-0.5,0),
                                np.sum(largura_dominios[:i]), altura,
#                                 linestyle='dashdot',
                                linewidth=2,
                                edgecolor='b',
                                fill = False)
        rets.append(ret)
    
    # plotar os retângulos das dimensões na área do gráfico
    for i in rets:
        ax.add_patch(i)
        
    # plotar legenda, comentar para excluir
    # ax.legend(bbox_to_anchor=(0.75,-0.05), ncol=2)
    
    campo='04'
    # savefig_respostas(campo)
    
    plt.show()
    return 

## Apurar participação discente e impacto artigos dos docentes
### Todos os períodos e grupos

In [ ]:
os.listdir(pathcsv)

In [ ]:
lista_nomes_docentes_permanentes   = 'lista_docentes_permanentes.csv'
lista_nomes_docentes_colaboradores = 'lista_docentes_colaboradores.csv'
df_docentes = montardf_docentes(lista_nomes_docentes_permanentes, lista_nomes_docentes_colaboradores)

## Escolha dos arquivos que alimentarão a análise da produção
lista_csv = ler_artigostodosperiodos()
lista_csv

In [ ]:
df_docentes

In [ ]:
pathfilename = os.path.join(pathcsv,'lista_docentes.csv')
lista_csv = pd.read_csv(pathfilename,header=None)[0].values

In [ ]:
df_prod   = montardf_producao(lista_csv)

## Mostrar quantitativos lidos
print(f'\nCarregado dataframe com {len(df_prod.index)} linhas:')
lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
print(f'{len(lista_titulos):4} artigos distintos publicados no período')
lista_revistas = pd.Series(df_prod['REVISTA'].values).unique().tolist()
print(f'{len(lista_revistas):4} revistas distintas utilizadas no período')

## Ler nomes de docentes, discentes e papel de orientador
lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes = montar_listas(lista_csv)
qte_docentes=len(lista_docentes)
qte_discentes=len(lista_discentes)
total_iteracoes=(qte_docentes+qte_discentes)

In [ ]:
df_docentes

    Organizar lista de autores e iniciais de nomes
    PROBLEMA: não está quebrando no número de artigos 1115, mas sim em apenas 2 e última vazia

In [ ]:
## Montar a lista de autores a partir da lista de strings limpa com nomes de autores
def organizar_nomes(str_autores_artigo):
    erros_organizar  = []
    lista_organizada = []
    partes_nomes = str_autores_artigo.split(', ')
    n       = len(partes_nomes)
    pares   = range(0,n+1,2)
    impares = range(1,n+1,2)
    try:
        for i,j in zip(pares,impares):
            nome=[]
            try:
                nomes_ordenado = str(partes_nomes[j].lower()+' '+partes_nomes[i].lower()).replace('  ',' ').strip()
            except:
                if len(pares) > len(impares):
                    nomes_ordenado = str(partes_nomes[j].lower()).replace('  ',' ').strip()
                else:
                    nomes_ordenado = str(partes_nomes[i].lower()).replace('  ',' ').strip()
            lista_organizada.append(nomes_ordenado)
    
    except Exception as e1:
        print('Erro ao montar listas de nomes de autor:',e1)
        erros_organizar.append((i,str_autores_artigo))

    return lista_organizada, erros_organizar

## Quebrar um nome em seu sobrenome seguido das partes de nome
def quebrar_partesnomes(nome):
    padrao     = padronizar_nome(nome).lower()
    sobrenome  = padrao.split(',')[0].strip()
    restonomes = padrao.split(',')[1].strip().split(' ')
    try:
        partenome1 = restonomes[0].strip()
    except:
        partenome1 = np.NaN
    try:
        partenome2 = restonomes[1].strip()
    except:
        partenome2 = np.NaN
    try:
        partenome3 = restonomes[2].strip()
    except:
        partenome3 = np.NaN        
    # print(f'{sobrenome:15} | {partenome1:1} | {partenome2:1} | {partenome3}')
    
    return sobrenome, partenome1, partenome2, partenome3


## Quebrar um nome em seu sobrenome seguido as iniciais das partes de nome
def quebrar_iniciais(nome):
    padrao = iniciais_nome(nome).lower()
    sobrenome  = padrao.split(',')[0].strip()
    restonomes = padrao.split(',')[1].strip().split(' ')

    try:
        partenome1 = restonomes[0].strip()
    except:
        partenome1 = np.NaN
    try:
        partenome2 = restonomes[1].strip()
    except:
        partenome2 = np.NaN
    try:
        partenome3 = restonomes[2].strip()
    except:
        partenome3 = np.NaN        
    # print(f'{sobrenome:15} | {partenome1:1} | {partenome2:1} | {partenome3}')
    
    return sobrenome, partenome1, partenome2, partenome3


## compilar padrão regular expression para buscar ocorência de duas das quatro partes de nome dentro de janela de no máximo 2 palavras de distância
def compilar_partes(sobrenome, partenome1, partenome2, partenome3):
    return re.compile(r'\b({0}|{1}|{2}|{3})(?:\W+\w+){{0,2}}?\W+({1}&{2}|{2}&{3}|{0}&{1}|{0}&{3})\b'.format(sobrenome, partenome1, partenome2, partenome3), flags=re.IGNORECASE)

def compilar_iniciais(sobrenome, inicial1, inicial2, inicial3):
    return re.compile(r'\b({0})(?:\W+\w+){{0,1}}?\W+({1}|{2}|{3})\b'.format(sobrenome, inicial1, inicial2, inicial3), flags=re.IGNORECASE)

In [ ]:
def gerar_dicionarios(df_prod, lista_docentes, lista_discentes, inicio = min(df_prod['Ano']), final  = max(df_prod['Ano'])):
    qte_docentes     = len(lista_docentes)
    qte_discentes    = len(lista_discentes)
    total_iteracoes  = (qte_docentes+qte_discentes)
    
    ## Monta uma lista de strings com nomes dos autores de cada artigo na forma extraída pelo e-lattes
    lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
    qte_artigos         = len(lista_nomes_autores)
        
    ## Define os limites para considerar duas strings com nomes similares entre si
    limite_jaro_nome        = 0.88
    limite_jaro_iniciais    = 0.75
    limite_jaccard_iniciais = 0.32

    ## Função para calcular similaridade dos nomes de docentes com cada nome de autor da lista de autores de cada artigo
    t1 = time.time()
    dic_nomes_docentes  = {}
    dic_nomes_discentes = {}    
    erros=[]
    rot1='Comparando nome do docente'
    rot2='Sobren/Iniciais docen'
    rot3='Sobrenome/Iniciais autor'
    rot4='I.Doc'
    rot5='I.Aut'
    rot6='Jaro-Winkler'
    rot7='Jaccard'
    for m,docente in enumerate(lista_docentes):
        achados_docentes     = []
        lista_indice_docente = []
        lista_qualis_docente = []
        docentes_naoencontrados = []          
        contagem=m+1
        # clear_output(wait=True)       
        try:
            nome_docente_padronizado = padronizar_nome(docente).lower()
            iniciais_nome_docente    = iniciais_nome(docente).lower()
            string_iniciais_docente  = ' '.join(x.strip() for x in iniciais_nome_docente.split(',')[1:])
            set_iniciais_docente     = converter_lista_set([x.strip() for x in iniciais_nome_docente.split(',')[1:]])                
            print(f'\nCálculo das similaridades autores-docentes (nome/iniciais/Jaccard):')
            print(f'{rot1:^40} |{rot2:^20}| {rot3:^25}|{rot4:^5}|{rot5:^5}|{rot6:^15}|{rot7:^10}')
            ## Monta a lista de autores com a divisão da string organizada padronizada no formato: {sobrenome, iniciais de nomes}
            lst_autores_artigo = []
            lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
            for n,nomes_autor in enumerate(lista_nomes_autores):
                autores,erros = organizar_nomes(nomes_autor)
                for o,autor in enumerate(autores):
                    qte_autores_artigo = len(autores)
                    # print(f'{o+1:2}/{qte_autores_artigo:2} nomes de autor do artigo em análise, restando {qte_autores_artigo-o-1:2}')
                    nome_autor_padronizado = padronizar_nome(autor).lower()
                    try:    
                        iniciais_nome_autor    = iniciais_nome(autor).lower()
                        string_iniciais_autor  = ' '.join(x.strip() for x in iniciais_nome_autor.split(',')[1:])
                        set_iniciais_autor     = converter_lista_set([x.strip() for x in iniciais_nome_autor.split(',')[1:]])
                        # time.sleep(1)
                        ## Cálculos de similaridades entre os nomes do autor e do docente
                        if iniciais_nome_docente and iniciais_nome_autor != '':
                            similaridade_nome     = get_jaro_distance(iniciais_nome_docente, iniciais_nome_autor)
                            similaridade_iniciais = get_jaro_distance(string_iniciais_docente, string_iniciais_autor)
                            similaridade_jaccard  = np.round(jaccard_similarity(set_iniciais_docente, set_iniciais_autor),2)
                            print(f'\nCálculo das similaridades autores-discentes (nome/iniciais/Jaccard):')
                            print(f'{padronizar_nome(docente):^40} | {iniciais_nome_docente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_docente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')
                            clear_output(wait=True)
                            if similaridade_nome > limite_jaro_nome and similaridade_iniciais > limite_jaro_iniciais and similaridade_jaccard > limite_jaccard_iniciais:
                                if n not in lista_indice_docente:
                                    print(f'Similaridade encontrada no artigo {n+1}/{qte_artigos}, docente {m+1}/{qte_docentes}')
                                    print(f'{rot1:^40} |{rot2:^20}| {rot3:^25}|{rot4:^5}|{rot5:^5}|{rot6:^15}|{rot7:^10}')
                                    print(f'{padronizar_nome(docente):^40} | {iniciais_nome_docente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_docente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')
                                    achados_docentes.append(docente)
                                    lista_indice_docente.append(n)
                                    # print(len(lista_indice_docente))
                                    extrato = df_prod['Qualis'][n]
                                    lista_qualis_docente.append(extrato)
                                    clear_output(wait=True)                                    
                    except Exception as e1:
                        # print(f'Erro na etapa 1 de gerar_dicionarios, ao tratar iniciais do nome de autor/docente da linha  {n}/{o}/{qte_docentes}/{qte_artigos}')
                        # print(e1)
                        similaridade_iniciais = np.NaN
                        similaridade_jaccard  = np.NaN
                        erros.append(('e1_similaridadesdocente',m,n,o,e1))                    
                lst_autores_artigo.append(iniciais_nome_autor)
            # print(f'Lista organizada de nomes de autores: {len(lst_autores_artigo)} listas de autores de artigos publicados no período')

            ## Ao final da leitura todos artigos para cada docente, criar o dicionário de docentes quando docente tenha aparecido na linha de autores
            if lista_indice_docente != []:
                dic_nomes_docentes[m] = (docente, lista_indice_docente, lista_qualis_docente)
            else:
                docentes_naoencontrados.append(docente)

            tdec=time.time()-t1
            # tres=tdec/(m+1)*((qte_docentes-m)+qte_discentes)
            # print(f'Analisadas{m+1:4}/{total_iteracoes} iterações em {horas(tdec)}, restando {total_iteracoes-m}. Busca{m+1:4}/{qte_docentes:<4}docente: {docente.title():50}')
            # print(f'Nome do docente foi encontrado em {len(lista_indice_docente):2} artigos')
        except Exception as e2:
            # print(f'Erro na etapa 2 de gerar_dicionarios, ao calcular similaridades de autor/docente  da linha {n}/{o}/{qte_docentes}/{qte_artigos}')
            # print(e2)
            erros.append(('e2_padronizardocentes',m,n,o,e2)) 

    ## Função para calcular similaridade dos nomes de discentes com cada nome de autor da lista de autores de cada artigo
    for p,discente in enumerate(lista_discentes):
        achados_discentes     = []
        lista_indice_discente = []
        lista_qualis_discente = []
        discentes_naoencontrados = []
        contagem=m+p+1
        # clear_output(wait=True)       
        try:
            nome_discente_padronizado = padronizar_nome(discente).lower()
            iniciais_nome_discente    = iniciais_nome(discente).lower()
            string_iniciais_discente  = ' '.join(x.strip() for x in iniciais_nome_discente.split(',')[1:])
            set_iniciais_discente     = converter_lista_set([x.strip() for x in iniciais_nome_discente.split(',')[1:]])
            rot2='Sobren/Iniciais disce'
            rot4='I.Dis'            
            ## Monta a lista de autores com a divisão da string organizada padronizada no formato: {sobrenome, iniciais de nomes}
            lst_autores_artigo = []
            lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
            for n,nomes_autor in enumerate(lista_nomes_autores):
                # clear_output(wait=True)
                # print(f'Procurando {nome_discente_padronizado.title()} em {n+1:2}/{qte_artigos:<2} artigos, restando {qte_artigos-n-1:<2}')
                autores,erros = organizar_nomes(nomes_autor)
                for q,autor in enumerate(autores):
                    qte_autores_artigo = len(autores)
                    nome_autor_padronizado = padronizar_nome(autor).lower()
                    # print(f'{q+1:2}/{qte_autores_artigo:2} nomes de autor do artigo em análise, restando {qte_autores_artigo-q-1:2}')
                    nome_autor_padronizado = padronizar_nome(autor).lower()
                    try:    
                        iniciais_nome_autor    = iniciais_nome(autor).lower()
                        string_iniciais_autor  = ' '.join(x.strip() for x in iniciais_nome_autor.split(',')[1:])
                        set_iniciais_autor     = converter_lista_set([x.strip() for x in iniciais_nome_autor.split(',')[1:]])   
                        ## Cálculos de similaridades entre os nomes do autor e do discente
                        if iniciais_nome_discente and iniciais_nome_autor != '':
                            similaridade_nome     = get_jaro_distance(iniciais_nome_discente, iniciais_nome_autor)
                            similaridade_iniciais = get_jaro_distance(string_iniciais_discente, string_iniciais_autor)
                            similaridade_jaccard  = np.round(jaccard_similarity(set_iniciais_discente, set_iniciais_autor),2)
                            print(f'{padronizar_nome(discente):^40} | {iniciais_nome_discente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_docente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')                            
                            if similaridade_nome > limite_jaro_nome and similaridade_iniciais > limite_jaro_iniciais and similaridade_jaccard > limite_jaccard_iniciais:
                                if n not in lista_indice_discente:
                                    print(f'Similaridade encontrada no artigo {n+1}/{qte_artigos}, discente {p+1}/{qte_discentes}')
                                    print(f'{rot1:^40} |{rot2:^20}| {rot3:^25}|{rot4:^5}|{rot5:^5}|{rot6:^15}|{rot7:^10}')
                                    print(f'{padronizar_nome(discente):^40} | {iniciais_nome_discente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_discente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')
                                    achados_discentes.append(discente)
                                    lista_indice_discente.append(n)
                                    # print(len(lista_indice_discente))
                                    extrato = df_prod['Qualis'][n]
                                    lista_qualis_discente.append(extrato)
                                    clear_output(wait=True)
                    except Exception as e3:
                        similaridade_iniciais = np.NaN
                        similaridade_jaccard  = np.NaN
                        # print(f'Erro na etapa 3 de gerar_dicionarios, ao calcular similaridades de nomes de autor/discente da linha {n}/{o}/{qte_discentes}/{qte_artigos}')
                        # print(e3)
                        erros.append(('e3_buscadiscentes',p,n,q,e3))
        
            ## Ao final da leitura todos artigos para cada discente, criar o dicionário de discente quando docente tenha aparecido na linha de autores
            if lista_indice_discente != []:
                dic_nomes_discentes[o] = (discente, lista_indice_discente, lista_qualis_discente)
            else:
                discentes_naoencontrados.append(discente)            
            
            tdec=time.time()-t1
            # tres=tdec/(o+p+1)*((qte_discentes-p)+qte_discentes)
            # print(f'Analisadas{contagem+1:4}/{total_iteracoes} iterações em {horas(tdec)}, restando {horas(tres)} para iterar {total_iteracoes-contagem-1}. Busca{p+1:4}/{qte_discentes:<4}discente: {discente.title():50}')
            # print(f'Nome do discente foi encontrado em {len(lista_indice_discente):2} artigos')   
        except Exception as e4:
            # print('Erro na etapa 4 de gerar_dicionarios, ao finalizar montagem dos dicionários:',e4)
            erros.append(('e4_padronizardiscente',m,n,e4))

    return dic_nomes_docentes, dic_nomes_discentes, docentes_naoencontrados, discentes_naoencontrados, erros

In [ ]:
dic_nomes_docentes, dic_nomes_discentes, docentes_naoencontrados, discentes_naoencontrados, erros = gerar_dicionarios(df_prod, lista_docentes, lista_discentes)

In [ ]:
# dic_nomes_docentes

In [ ]:
print(len(docentes_naoencontrados),'total de nomes de  docentes não encontrados nos artigos')
print(len(discentes_naoencontrados),'total de nomes de discentes não encontrados nos artigos')

In [ ]:
discentes_naoencontrados

In [ ]:
len(erros)

In [ ]:
erros[:3]

In [ ]:
# lista_artigos_problemas = []
# lista_autores_problemas = []
# for etapa,docente,i,discente,artigo in erros:
#     if i not in lista_artigos_problemas:
#         lista_artigos_problemas.append(i)
#         lista_autores_problemas.append(df_prod['AUTORES'][i])

# df_artigos_problemas=pd.DataFrame(lista_artigos_problemas).reset_index(drop=True)
# df_artigos_problemas['LISTA_AUTORES'] = lista_autores_problemas
# df_artigos_problemas

In [ ]:
# ## Lista ordenada alfabeticamente pelos sobrenomes de docentes
# lista_docentes_sobrenome=[]
# for i in lista_docentes:
#     lista_docentes_sobrenome.append(iniciais_nome(i))
    
# lista_docentes_sobrenome.sort()
# for j in lista_docentes_sobrenome:
#     print(f'{j.lower()}')

In [ ]:
inicio = 2017
final  = 2022
df_impacto_docente, df_impacto_discente = montardf_impacto_docente_discente(df_prod, dic_nomes_docentes, dic_nomes_discentes, inicio, final)
df_impacto_docente

In [ ]:
df_docentes = montardf_docentes(lista_nomes_docentes_permanentes, lista_nomes_docentes_colaboradores)
df_docentes_pcd_impacto, apuracao_pcd, df_abaixo_pcd, df_atingiram_pcd, indicador_pcd, indicador_impacto = apurar_pcd_impacto(df_docentes, df_impacto_docente, df_impacto_discente, meta_pcd=50.0, meta_impacto=150.0)

In [ ]:
plotar_pcd(df_docentes_pcd_impacto)
plotar_medias_impacto(df_docentes_pcd_impacto, grupo=False, inicio=False, final=False)

### Funções avaliação PCD e Impacto Médio Anual: 
    Avaliar indicadores PCD e Somatório do Fator de Impacto

In [ ]:
def avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes):
    ## Ler arquivos de dados
    lista_csv   = ler_csvptg(inicio, final, tipo, grupo)
    df_public   = montardf_producao(lista_csv)
    df_docentes = montardf_docentes(lista_nomes_docentes)
    lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes = montar_listas(lista_csv, lista_nomes_docentes)

    ## Avaliação da Publicação Conjunta com Discentes (PCD) e do Impacto Médio Anual (IMA)
    dic_nomes_docentes, dic_nomes_discentes, docentes_naoencontrados, discentes_naoencontrados, erros = gerar_dicionarios(df_public, lista_docentes, lista_discentes, inicio, final)
    df_impacto_docente, df_impacto_discente = montardf_impacto_docente_discente(df_public, dic_nomes_docentes, dic_nomes_discentes, inicio, final)

    ## Gerar gráfico de apuração de impacto médio anual por docente
    df_docentes_pcd_impacto, apuracao_pcd, df_abaixo_pcd, df_atingiram_pcd, indicador_pcd, indicador_impacto = apurar_pcd_impacto(df_docentes, df_impacto_docente, df_impacto_discente)
    plotar_pcd(df_docentes_pcd_impacto, grupo, inicio, final)
    plotar_medias_impacto(df_docentes_pcd_impacto, grupo, inicio, final)

    return df_public, df_impacto_docente, df_impacto_discente, df_docentes_pcd_impacto, indicador_pcd, indicador_impacto

# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .

### Testes

In [ ]:
# a,b='',''
# get_jaro_distance(a, b)

In [ ]:
a,b='1','0'
get_jaro_distance(a, b)

In [ ]:
lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
print(len(lista_nomes_autores))
print(lista_nomes_autores[0])

## Verifica se a divisão em nomes de autor é par (sobrenome separado de nomes por vírgula)
for i in lista_nomes_autores[:14]:
    qte_nomes_autor = len(i.split(','))
    if qte_nomes_autor/2 != qte_nomes_autor//2:
        print(qte_nomes_autor,'\n')

In [ ]:
for m,docente in enumerate(lista_docentes):
    nome_docente_padronizado = padronizar_nome(docente).lower()
    iniciais_nome_docente    = iniciais_nome(docente).lower()
    print(iniciais_nome_docente)

In [ ]:
    # ## Monta a lista de autores com a divisão da string de nomes acima por: sobrenome, nomes
    # lst_autores_artigo = []
    # lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
    # for n,nomes_autor in enumerate(lista_nomes_autores):
    #     autores,erros = organizar_nomes(nomes_autor)
    #     # print(autores,'\n')
    #     for o,autor in enumerate(autores):
    #         # print(autor)
    #         # print(f'{o+1:2}/{qte_autores_artigo:2} nomes de autor do artigo em análise, restando {qte_autores_artigo-o-1:2}')
    #         sobrenome_iniciais_autor = iniciais_nome(autor).lower()
    #         string_iniciais_autor    = ' '.join(x.strip() for x in sobrenome_iniciais_autor.split(',')[1:])
    #         set_iniciais_autor       = converter_lista_set([x.strip() for x in sobrenome_iniciais_autor.split(',')[1:]])
    #         print(f'{sobrenome_iniciais_autor:20}|{string_iniciais_autor:^9}|{set_iniciais_autor}')
    #     lst_autores_artigo.append(sobrenome_iniciais_autor)
    # print(f'Lista organizada de nomes de autores: {len(lst_autores_artigo)} listas de autores de artigos publicados no período')

In [ ]:
qte_docentes     = len(lista_docentes)
qte_discentes    = len(lista_discentes)
total_iteracoes  = (qte_docentes+qte_discentes)

## Monta uma lista de strings com nomes dos autores de cada artigo
lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
qte_artigos         = len(lista_nomes_autores)

## Monta a lista de autores com a divisão da string de nomes acima por: sobrenome, nomes
lst_autores_artigo = []
for cada_lista_autores in lista_nomes_autores:
    lista_organizada, erros_organizar = organizar_nomes(cada_lista_autores)
    lst_autores_artigo.append(lista_organizada)
print(f'{len(lst_autores_artigo)} listas de autores de {qte_artigos} artigos publicados no período')

In [ ]:
lista_nomes_autores[:3]

In [ ]:
lst_autores_artigo[:3]

In [ ]:
a='a b c de oliveira'
padronizar_nome(a)
iniciais_nome(a)

# Apuração segmentada por períodos e grupos

In [ ]:
evolucao_pcd=[]
evolucao_impacto=[]
tipo_analise=[]
grupo_analise=[]
periodo_analise=[]

## Quadriênio 2017-2020 de Docentes Permanentes

In [ ]:
lista_nomes_docentes_permanentes   = 'lista_docentes_permanentes.csv'
lista_nomes_docentes_colaboradores = 'lista_docentes_colaboradores.csv'
df_docentes = montardf_docentes(lista_nomes_docentes_permanentes, lista_nomes_docentes_colaboradores)

In [ ]:
inicio = 2017
final  = 2020
tipo   = 'artigos'
grupo  = 'permanentes'
lista_nomes_docentes = 'lista_docentes_permanentes.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

## .
## Quadriênio 2017-2020 de Docentes Colaboradores

In [ ]:
inicio = 2017
final  = 2020
tipo   = 'artigos'
grupo  = 'colaboradores'
lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

# .
# .
# .
# Avaliação de meio termo biênio [2021-2022]

## Biênio 2021-2022 de Docentes Permanentes

In [ ]:
inicio = 2021
final  = 2022
tipo   = 'artigos'
grupo  = 'permanentes'
lista_nomes_docentes = 'lista_docentes_permanentes.csv'
# lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

## .
## .
## Biênio 2021-2022 de Docentes Colaboradores

In [ ]:
inicio = 2021
final  = 2022
tipo   = 'artigos'
grupo  = 'colaboradores'
# lista_nomes_docentes = 'lista_docentes_permanentes.csv'
lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

# Evolução de indicadores de gestão do programa

In [ ]:
df_indicadores = pd.DataFrame({
    'TIPO': pd.Series(tipo_analise),
    'GRUPO': pd.Series(grupo_analise),
    'PERIODOS': pd.Series(periodo_analise),
    'META_PCD_50%': pd.Series(evolucao_pcd),
    'META_IMPACTO_150ANO': pd.Series(evolucao_impacto),
    })

In [ ]:
df_indicadores.sort_values(by=['GRUPO'], ascending=True).reset_index(drop=True)

# Conferência em detalhes

In [ ]:
inicio = 2021
final  = 2022
tipo   = 'artigos'
grupo  = 'permanentes'
lista_nomes_docentes = 'lista_docentes_permanentes.csv'
# lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)

In [ ]:
# df_impacto_docente[:60]

## Conferência dos achados de nomes de discentes

In [ ]:
def mostrar_partes_achadas(lista_discentes, lista_autores, verbose=False):
    partes_achadas=[]
    discentes_achados=[]
    erros=[]
    if verbose == True:
        print(f'{padronizar_titulo(lista_autores).lower()}')
    try:
        ## Buscar pelas partes de nomes de autor em cada linha de autores de artigo
        for nome in lista_discentes:
            sobrenome, partenome1, partenome2, partenome3 = quebrar_partesnomes(nome)
            if verbose == True:
                print(f'{sobrenome}, {partenome1} {partenome2} {partenome3}')
            strbusca_partes = compilar_partes(sobrenome, partenome1, partenome2, partenome3)
            try:
                achados_partes = re.search(strbusca_partes, padronizar_titulo(lista_autores).lower())
                if achados_partes.span() !=None:
                    partes_achadas.append(achados_partes.groups())
                    discentes_achados.append(nome)
            except Exception as e:
                # print(e)
                pass

        ## Buscar pelas iniciais de partes de nomes de autor, que seguem um sobrenome em cada linha de autores de artigo
        for nome in lista_discentes:
            sobrenome, inicial1, inicial2, inicial3 = quebrar_iniciais(nome)
            if verbose == True:
                print(f'{sobrenome}, {inicial1} {inicial2} {inicial3}')
            strbusca_iniciais = compilar_iniciais(sobrenome, inicial1, inicial2, inicial3)
            try:
                achados_iniciais = re.search(strbusca_iniciais, padronizar_titulo(lista_autores).lower())
                if achados_iniciais.span() !=None and achados_iniciais.groups() not in discentes_achados:
                    partes_achadas.append(achados_iniciais.groups())
                    discentes_achados.append(nome)
            except Exception as e:
                # print(e)
                pass
    except Exception as e:
        # print(f'Erro ao buscar partes de nome; {e}')
        print(e)
    
    return partes_achadas, discentes_achados

def listar_achados(docente):
    lista_csv = ler_artigostodosperiodos()
    lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes = montar_listas(lista_csv)

    df_filtrado = df_impacto_docente[(df_impacto_docente.DOCENTE==docente)]

    lista_indices = df_filtrado['INDICES_ARTIGOS'].values.tolist()[0]
    n=100
    print('-'*n)
    print(f'\nDocente: {docente} | {len(lista_indices)} Publicações identificadas para no período [{inicio} a {final}]')
    print()
    for indice in lista_indices:
        print('-'*n)
        print(f'Índice da publicação: {indice}')
        print(df_public.iloc[indice].values[:3])
        lista_autores = padronizar_titulo(df_public.iloc[indice].values[4])
        partes_achadas, discentes_achados = mostrar_partes_achadas(lista_discentes, lista_autores, verbose=False)
        print()
        print(lista_autores)
        print('\nPartes de nomes de alunos encontrados na lista de autores:')
        print(partes_achadas)
        print('\nNomes de alunos considerados como encontrados na lista de autores:')
        print(discentes_achados)

def separar_iniciais(nome):
    import re
    letras_duasconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2}$')            # Duas Letras consoantes maiúsculas juntas do início ao final da string
    letras_tresconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3}$')            # Três Letras consoantes maiúsculas juntas do início ao final da string
    letras_duasconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2},$')       # Duas Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    letras_tresconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3},$')       # Três Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    partes_nome=[]
    for j in nome.split(' '):
        div_ltrduasconsnts = letras_duasconsnts.findall(j)
        div_ltrtriplicadas = letras_tresconsnts.findall(j)
        div_ltrduasconsntsvirg = letras_duasconsntsvirg.findall(j)
        div_ltrtresconsntsvirg = letras_tresconsntsvirg.findall(j)
        if div_ltrduasconsnts or div_ltrtriplicadas:
            iniciais_separadas = ' '.join(x for x in j)
            partes_nome.append(iniciais_separadas)
        elif div_ltrduasconsntsvirg or div_ltrtresconsntsvirg:
            iniciais_separadas = ' '.join(x for x in j[:-1])
            partes_nome.append(iniciais_separadas+',')
        else:
            partes_nome.append(j)
    nome_separado = ' '.join(x for x in partes_nome).strip()
    return nome_separado

In [ ]:
docente = 'Olindo Assis Martins Filho'
listar_achados(docente)

# Outras funcionalidades

    Formatar tempo (h:min:seg)

In [ ]:
def tempo(start, end):
    t=end-start

    tempo = timedelta(
        weeks   = t//(3600*24*7),
        days    = t//(3600*24),
        seconds = t,
        minutes = t//(60),
        hours   = t//(3600),
        microseconds=t//1000000,
        )
    fmt='{H:2}:{M:02}:{S:02}'
    return strfdelta(tempo)


def horas(segundos): 
    return time.strftime("%H:%M:%S", time.gmtime(segundos)) 


def dias_horas_minutos(td):
    x = (td.days, td.seconds//3600, (td.seconds//60)%60, td.seconds)
    return x #(days, hrs, mins, seconds)


def strfdelta(tdelta, fmt='{H:02}h {M:02}m {S:02}s', inputtype='timedelta'):
    """Convert a datetime.timedelta object or a regular number to a custom-formatted string, 
    just like the stftime() method does for datetime.datetime objects.

    The fmt argument allows custom formatting to be specified.  Fields can 
    include seconds, minutes, hours, days, and weeks.  Each field is optional.

    Some examples:
        '{D:02}d {H:02}h {M:02}m {S:02}s' --> '05d 08h 04m 02s' (default)
        '{W}w {D}d {H}:{M:02}:{S:02}'     --> '4w 5d 8:04:02'
        '{D:2}d {H:2}:{M:02}:{S:02}'      --> ' 5d  8:04:02'
        '{H}h {S}s'                       --> '72h 800s'

    The inputtype argument allows tdelta to be a regular number instead of the  
    default, which is a datetime.timedelta object.  Valid inputtype strings: 
        's', 'seconds', 
        'm', 'minutes', 
        'h', 'hours', 
        'd', 'days', 
        'w', 'weeks'
    """

    # Convert tdelta to integer seconds.
    if inputtype == 'timedelta':
        remainder = int(tdelta.total_seconds())
    elif inputtype in ['s', 'seconds']:
        remainder = int(tdelta)
    elif inputtype in ['m', 'minutes']:
        remainder = int(tdelta)*60
    elif inputtype in ['h', 'hours']:
        remainder = int(tdelta)*3600
    elif inputtype in ['d', 'days']:
        remainder = int(tdelta)*86400
    elif inputtype in ['w', 'weeks']:
        remainder = int(tdelta)*604800

    f = Formatter()
    desired_fields = [field_tuple[1] for field_tuple in f.parse(fmt)]
    possible_fields = ('W', 'D', 'H', 'M', 'S')
    constants = {'W': 604800, 'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    values = {}
    
    for field in possible_fields:
        if field in desired_fields and field in constants:
            values[field], remainder = divmod(remainder, constants[field])
    
    return f.format(fmt, **values)


# print (timedelta(days=365, hours=8, minutes=15))
# print ("   Hoje é: " + str(date.today()))
# print ("Agora são: " + str(datetime.now()))
# print ("Um ano no futuro estaremos em:" + str(dt.today() + timedelta(days=365)))
# hoje = date.today()
# print(hoje)
# hora = dt.now()
# print(hora)
# dias_ano = date(hoje.year, 1, 1)
# if dias_ano < hoje:
#     print ("Decoridos %d dias do ano" % ((hoje - dias_ano).days))
    
# from datetime import datetime
# now= datetime.now() #get the current date and time

# #%c - local date and time, %x-local's date, %X- local's time
# print(now.strftime("%c"))
# print(now.strftime("%x"))
# print(now.strftime("%X"))

# ##### Time Formatting ####
# #%I/%H - 12/24 Hour, %M - minute, %S - second, %p - local's AM/PM
# print(now.strftime("%I:%M:%S %p")) # 12-Hour:Minute:Second:AM
# print(now.strftime("%H:%M")) # 24-Hour:Minute